In [1]:
%conda install -c conda-forge obspy -y


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.5.1



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [5]:
print("Hello, world!")
from datetime import datetime
print("Current time is", datetime.now())


Hello, world!
Current time is 2025-08-04 02:33:03.076428


### Earthquake Catalog and Feature Extraction

This script retrieves the earthquake catalog from IRIS FDSN for the period **February 25 – March 25, 2011** in the Tohoku region (Mw ≥ 3.0), and downloads corresponding waveforms from station **IU.MAJO.00.BHZ**.  
For each event, short ([-30s, +60s]) and long windows (5–30 minutes depending on magnitude) are extracted. The waveforms are preprocessed by instrument response removal, detrending, demeaning, and multi-band filtering.  
Energy features (sum of squared amplitudes) are then calculated across multiple frequency bands, together with event time, magnitude, and magnitude category.  
The final dataset is saved as:  

- `tohoku_earthquake_features_2011.csv`  
- `tohoku_earthquake_features_2011.pkl`  


In [3]:
import numpy as np
import pandas as pd
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.core import UTCDateTime
from dateutil.relativedelta import relativedelta
from http.client import IncompleteRead, HTTPException
from concurrent.futures import ThreadPoolExecutor, as_completed

# ——— 1. Configuration ———
client        = Client("IRIS", timeout=120)
network       = "IU"
station       = "MAJO"
location      = "00"
channel       = "BHZ"

# Time range: 15 days before and after the Tohoku earthquake
start_date    = UTCDateTime("2011-02-25")
end_date      = UTCDateTime("2011-03-25")

# Only fetch events with Mw ≥ 3.0
min_magnitude = 3.0
max_magnitude = 9.5

# Time windows and frequency bands
pre_window_short  = 30   # 30 seconds before event for short window
post_window_short = 60   # 60 seconds after event for short window
frequency_bands   = [(0.1, 1.0), (1.0, 5.0), (5.0, 8.0)]

def choose_post_window_duration(magnitude):
    """
    Choose the duration of post-event long window based on magnitude
    
    Args:
        magnitude (float): Event magnitude
        
    Returns:
        int: Duration in seconds
    """
    if magnitude < 4.0:   
        return 300   # 5 minutes for small events
    elif magnitude < 6.0:   
        return 600   # 10 minutes for moderate events
    else:
        return 1800  # 30 minutes for large events

def safe_event_query(**kwargs):
    """
    Robust event catalog query with retry mechanism
    
    Args:
        **kwargs: Parameters for get_events()
        
    Returns:
        obspy.Catalog: Event catalog or empty list if failed
    """
    for attempt in range(3):
        try:
            return client.get_events(**kwargs)
        except (IncompleteRead, HTTPException):
            continue
    return []

def fetch_earthquake_catalog(start_time, end_time):
    """
    Fetch earthquake catalog day by day from ISC, Mw ≥ 3.0
    
    Args:
        start_time (UTCDateTime): Start time
        end_time (UTCDateTime): End time
        
    Returns:
        list: List of unique earthquake events
    """
    events_list = []
    current_time = start_time.datetime
    end_datetime = end_time.datetime
    
    while current_time < end_datetime:
        day_start = UTCDateTime(current_time)
        day_end = UTCDateTime(current_time + relativedelta(days=1))
        
        # Query events for one day
        catalog = safe_event_query(
            starttime=day_start, 
            endtime=day_end,
            minmagnitude=min_magnitude, 
            maxmagnitude=max_magnitude,
            catalog="ISC",
            # Regional search around Japan (±10 degrees from Tohoku epicenter)
            minlatitude=37.2247-10, 
            maxlatitude=37.2247+10,
            minlongitude=140.8777-10, 
            maxlongitude=140.8777+10
        )
        
        if catalog:
            events_list.extend(catalog)
            print(f"{day_start.date}: retrieved {len(catalog)} events")
            
        current_time += relativedelta(days=1)
    
    # Remove duplicates based on resource ID
    unique_events = list({event.resource_id.id: event for event in events_list}.values())
    return unique_events

def assign_magnitude_category(magnitude):
    """
    Assign magnitude category based on earthquake magnitude
    
    Args:
        magnitude (float): Earthquake magnitude
        
    Returns:
        str: Magnitude category
    """
    if 3.0 <= magnitude < 5.0:
        return "Minor+Light"      # M3.0-4.9
    elif 5.0 <= magnitude < 7.0:
        return "Moderate+Strong"  # M5.0-6.9
    else:
        return "Major+Great"      # M7.0+

def extract_event_features(earthquake_event):
    """
    Download waveform and extract features for a single earthquake event
    
    Args:
        earthquake_event: ObsPy Event object
        
    Returns:
        dict: Feature dictionary or None if processing failed
    """
    origin_time = earthquake_event.origins[0].time
    magnitude = earthquake_event.magnitudes[0].mag
    post_window_long = choose_post_window_duration(magnitude)
    
    try:
        # Download waveform data
        waveform_stream = client.get_waveforms(
            network, station, location, channel,
            origin_time - pre_window_short, 
            origin_time + post_window_long,
            attach_response=True,
            longestonly=True
        )
        
        if len(waveform_stream) == 0:
            return None
            
        # Remove instrument response and preprocess
        waveform_stream.remove_response(output="VEL")
        waveform_stream.detrend("linear")
        waveform_stream.detrend("demean")
        
    except Exception as e:
        return None
    
    # Extract time windows
    short_window_stream = waveform_stream.slice(
        origin_time - pre_window_short, 
        origin_time + post_window_short
    )
    long_window_stream = waveform_stream.slice(
        origin_time, 
        origin_time + post_window_long
    )
    
    # Initialize feature dictionary
    feature_dict = {
        "event_time": origin_time.isoformat(),
        "magnitude": magnitude,
        "magnitude_category": assign_magnitude_category(magnitude)
    }
    
    # Extract energy features for different frequency bands and time windows
    for freq_min, freq_max in frequency_bands:
        for window_name, stream_segment in [("short", short_window_stream), 
                                          ("long", long_window_stream)]:
            # Copy trace for filtering
            trace_copy = stream_segment[0].copy()
            
            # Apply bandpass filter
            trace_copy.filter("bandpass",
                            freqmin=freq_min, 
                            freqmax=freq_max,
                            corners=4, 
                            zerophase=True)
            
            # Calculate energy (sum of squared amplitudes)
            energy_feature_name = f"{window_name}_{freq_min:.1f}-{freq_max:.1f}Hz_energy"
            feature_dict[energy_feature_name] = np.sum(trace_copy.data**2)
    
    return feature_dict

# ——— 2. Fetch earthquake catalog ———
print("Fetching earthquake catalog...")
earthquake_catalog = fetch_earthquake_catalog(start_date, end_date)
print(f"\nTotal events ≥Mw{min_magnitude}: {len(earthquake_catalog)}\n")

# ——— 3. Parallel waveform download and feature extraction ———
print("Starting parallel feature extraction...")
extracted_features = []

with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit all tasks
    future_dict = {executor.submit(extract_event_features, event): event 
                   for event in earthquake_catalog}
    
    # Collect results as they complete
    for future in as_completed(future_dict):
        result = future.result()
        if result is not None:
            extracted_features.append(result)

print(f"\nSuccessfully extracted features for {len(extracted_features)} events\n")

# ——— 4. Create DataFrame and display results ———
earthquake_dataframe = pd.DataFrame(extracted_features)

# Ensure magnitude category column exists
if 'magnitude_category' not in earthquake_dataframe.columns:
    earthquake_dataframe['magnitude_category'] = earthquake_dataframe['magnitude'].apply(assign_magnitude_category)

# Display results
from IPython.display import display
print("Sample of extracted features:")
display(earthquake_dataframe.head())

print("\nMagnitude category distribution:")
print(earthquake_dataframe['magnitude_category'].value_counts())

# ——— 5. Save dataset to files ———
import os

# Display current working directory
print(f"\nSaving dataset to: {os.getcwd()}")

# Save as CSV file (universal format)
csv_filename = 'tohoku_earthquake_features_2011.csv'
earthquake_dataframe.to_csv(csv_filename, index=False, encoding='utf-8')

# Save as pickle file (preserves data types)
pickle_filename = 'tohoku_earthquake_features_2011.pkl'
earthquake_dataframe.to_pickle(pickle_filename)

print(f"✅ Dataset saved successfully!")
print(f"📊 Total records: {len(earthquake_dataframe)}")
print(f"📁 Files created:")
print(f"   - {csv_filename}")
print(f"   - {pickle_filename}")

# Display files in current directory
print(f"\n📂 Files in directory {os.getcwd()}:")
data_files = [f for f in os.listdir('.') if f.endswith(('.csv', '.pkl', '.xlsx'))]
for filename in data_files:
    print(f"   - {filename}")

# ——— 6. Verify file integrity ———
print("\n🔍 Verifying saved files...")
try:
    # Test CSV file loading
    loaded_dataframe = pd.read_csv(csv_filename)
    print(f"✅ CSV file verified: {len(loaded_dataframe)} rows loaded")
    print(f"📋 Columns: {list(loaded_dataframe.columns)}")
    
    # Display basic statistics
    print(f"\n📈 Dataset Statistics:")
    print(f"   - Time range: {loaded_dataframe['event_time'].min()} to {loaded_dataframe['event_time'].max()}")
    print(f"   - Magnitude range: {loaded_dataframe['magnitude'].min():.1f} to {loaded_dataframe['magnitude'].max():.1f}")
    print(f"   - Feature columns: {len(loaded_dataframe.columns)}")
    
except Exception as e:
    print(f"❌ File verification failed: {e}")

print("\n🎉 Earthquake feature extraction and dataset creation completed!")

Fetching earthquake catalog...
2011-02-25: retrieved 8 events
2011-02-26: retrieved 31 events
2011-02-27: retrieved 13 events
2011-02-28: retrieved 18 events
2011-03-01: retrieved 10 events
2011-03-02: retrieved 5 events
2011-03-03: retrieved 8 events
2011-03-04: retrieved 9 events
2011-03-05: retrieved 10 events
2011-03-06: retrieved 11 events
2011-03-07: retrieved 12 events
2011-03-08: retrieved 8 events
2011-03-09: retrieved 138 events
2011-03-10: retrieved 61 events
2011-03-11: retrieved 1320 events
2011-03-12: retrieved 1082 events
2011-03-13: retrieved 803 events
2011-03-14: retrieved 661 events
2011-03-15: retrieved 613 events
2011-03-16: retrieved 533 events
2011-03-17: retrieved 522 events
2011-03-18: retrieved 398 events
2011-03-19: retrieved 394 events
2011-03-20: retrieved 362 events
2011-03-21: retrieved 329 events
2011-03-22: retrieved 404 events
2011-03-23: retrieved 282 events
2011-03-24: retrieved 256 events

Total events ≥Mw3.0: 8301

Starting parallel feature extract

,event_time,magnitude,magnitude_category,short_0.1-1.0Hz_energy,long_0.1-1.0Hz_energy,short_1.0-5.0Hz_energy,long_1.0-5.0Hz_energy,short_5.0-8.0Hz_energy,long_5.0-8.0Hz_energy
0,2011-02-25T18:59:17.500000,3.1,Minor+Light,4.211656e-10,1.293103e-09,1.323571e-13,1.429869e-11,1.784218e-14,1.845573e-12
1,2011-02-25T06:44:47.500000,3.1,Minor+Light,1.270409e-10,4.205405e-10,4.182927e-12,6.218598e-12,4.735624e-13,7.924541e-13
2,2011-02-25T15:48:16.400000,3.3,Minor+Light,3.568408e-10,1.265034e-09,6.429292e-13,1.996687e-12,5.364888e-14,1.680307e-13
3,2011-02-25T17:00:17.550000,3.7,Minor+Light,3.079306e-10,1.316121e-09,1.174070e-13,6.546385e-13,1.144721e-14,4.531881e-14
4,2011-02-25T05:55:54.600000,3.1,Minor+Light,7.106407e-11,3.524731e-10,8.031155e-13,2.933601e-12,5.826152e-14,1.220384e-13



Magnitude category distribution:
Minor+Light        7685
Moderate+Strong     612
Major+Great           4
Name: magnitude_category, dtype: int64

Saving dataset to: /Users/donghui
✅ Dataset saved successfully!
📊 Total records: 8301
📁 Files created:
   - tohoku_earthquake_features_2011.csv
   - tohoku_earthquake_features_2011.pkl

📂 Files in directory /Users/donghui:
   - tohoku_earthquake_features_2011.csv
   - earthquake_features_2011.csv
   - earthquake_features_2011.pkl
   - tohoku_earthquake_features_2011.pkl

🔍 Verifying saved files...
✅ CSV file verified: 8301 rows loaded
📋 Columns: ['event_time', 'magnitude', 'magnitude_category', 'short_0.1-1.0Hz_energy', 'long_0.1-1.0Hz_energy', 'short_1.0-5.0Hz_energy', 'long_1.0-5.0Hz_energy', 'short_5.0-8.0Hz_energy', 'long_5.0-8.0Hz_energy']

📈 Dataset Statistics:
   - Time range: 2011-02-25T05:55:54.600000 to 2011-03-24T23:56:51.260000
   - Magnitude range: 3.0 to 9.1
   - Feature columns: 9

🎉 Earthquake feature extraction and dataset cr

### Continuous Waveform Download

This script downloads **continuous seismic waveform data** from IRIS FDSN for station **IU.MAJO.00.BHZ**, covering the period **March 1 – March 21, 2011**.  
The waveforms are retrieved on a daily basis and saved in MiniSEED format within the directory `waveforms/`.  
Each file is named according to the station and date, e.g.,  

- `waveforms/MAJO_2011-03-01.mseed`  
- `waveforms/MAJO_2011-03-02.mseed`  


In [1]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import os

client = Client("IRIS")
network = "IU"
station = "MAJO"
location = "00"
channel = "BHZ"

output_dir = "waveforms"
os.makedirs(output_dir, exist_ok=True)

# 时间段设定
start_date = UTCDateTime("2011-03-01")
end_date = UTCDateTime("2011-03-21")

# 每天下载
current_date = start_date
while current_date < end_date:
    try:
        print(f"Downloading {current_date.date} ...")
        st = client.get_waveforms(network, station, location, channel,
                                  current_date, current_date + 86400)
        st.write(os.path.join(output_dir, f"{station}_{current_date.date}.mseed"), format="MSEED")
    except Exception as e:
        print(f"Failed: {current_date.date} → {e}")
    current_date += 86400


### Dataset Construction

This script integrates the earthquake catalog with the continuous waveform data from station **IU.MAJO.00.BHZ** to construct the final training dataset.  

- **Positive samples (earthquake windows):**  
  For each catalogued event, a 90-second window was extracted (20s before and 70s after the origin time). Each sample is labeled as *small* (Mw < 5.0), *moderate* (5.0 ≤ Mw < 6.0), or *large* (Mw ≥ 6.0).  

- **Negative samples (noise windows):**  
  Sliding windows of 90s were extracted from non-event periods (step = 45s), excluding a ±120s margin around known events to avoid contamination. These were labeled as *noise*.  

- **Preprocessing:**  
  Waveforms were detrended, bandpass-filtered (0.1–8 Hz), and standardized.  

- **Features:**  
  For each window, frequency band energy features (0.5–2 Hz, 2–5 Hz, 5–8 Hz) were computed, along with metadata such as sample ID, time window, and magnitude category.  

- **Output files:**  
  - `seismic_dataset.npz` → contains waveform arrays, class labels, and sample IDs.  
  - `seismic_features.csv` → contains extracted features and metadata for all samples.  


In [2]:
import pandas as pd
import numpy as np
from obspy import UTCDateTime, read
from glob import glob
import os

# --- CONFIGURATIONS ---
sampling_rate = 20
window_duration_sec = 90
window_length = sampling_rate * window_duration_sec
pre_event_sec = 20
post_event_sec = 70
exclude_margin_sec = 120
sliding_step_sec = 45

# --- PATHS ---
event_file = "tohoku_earthquake_features_2011.csv"
waveform_dir = "waveforms"
output_npz_path = "seismic_dataset.npz"
output_csv_path = "seismic_features.csv"

# --- LOAD EVENT DATA ---
events_df = pd.read_csv(event_file)
events_df["event_time"] = pd.to_datetime(events_df["event_time"])
event_times = [UTCDateTime(t) for t in events_df["event_time"]]

# --- HELPERS ---
def classify_magnitude(mag):
    if mag < 5.0:
        return "small"
    elif 5.0 <= mag < 6.0:
        return "moderate"
    else:
        return "large"

def extract_band_energy_features(signal, fs=20):
    from scipy.signal import butter, sosfilt
    def band_energy(x, fmin, fmax):
        sos = butter(4, [fmin, fmax], btype='band', fs=fs, output='sos')
        filtered = sosfilt(sos, x)
        return np.sum(filtered ** 2)
    return {
        "energy_0.5_2Hz": band_energy(signal, 0.5, 2.0),
        "energy_2_5Hz": band_energy(signal, 2.0, 5.0),
        "energy_5_8Hz": band_energy(signal, 5.0, 8.0)
    }

def overlaps_with_events(event_times, start, end, margin):
    for event_time in event_times:
        if (start - margin) <= event_time <= (end + margin):
            return True
    return False

# --- PROCESSING ---
waveform_samples = []
class_labels = []
binary_labels = []
sample_ids = []
metadata_records = []

mseed_files = sorted(glob(os.path.join(waveform_dir, "*.mseed")))
pos_count, neg_count = 0, 0

for file_path in mseed_files:
    file_name = os.path.basename(file_path)
    try:
        st = read(file_path)
        st.detrend("demean")
        st.filter("bandpass", freqmin=0.1, freqmax=8.0)
        tr = st[0]

        t_start = tr.stats.starttime
        t_end = tr.stats.endtime

        # --- POSITIVE SAMPLES ---
        day_events = events_df[events_df["event_time"].dt.date == t_start.date()]
        for _, row in day_events.iterrows():
            t0 = UTCDateTime(row["event_time"]) - pre_event_sec
            t1 = t0 + window_duration_sec
            if t0 < t_start or t1 > t_end:
                continue
            data = tr.slice(starttime=t0, endtime=t1).data[:window_length]
            if len(data) == window_length:
                mag_category = classify_magnitude(row["magnitude"])
                waveform_samples.append(data)
                class_labels.append({"small": 1, "moderate": 2, "large": 3}[mag_category])
                binary_labels.append(1)
                sid = f"pos_{pos_count:05d}"
                sample_ids.append(sid)
                feats = extract_band_energy_features(data)
                metadata_records.append({
                    "sample_id": sid,
                    "is_earthquake": 1,
                    "label": {"small": 1, "moderate": 2, "large": 3}[mag_category],
                    "magnitude": row["magnitude"],
                    "magnitude_category": mag_category,
                    "window_start": str(t0),
                    "window_end": str(t1),
                    **feats
                })
                pos_count += 1

        # --- NEGATIVE SAMPLES ---
        win_start = t_start
        while win_start + window_duration_sec <= t_end:
            win_end = win_start + window_duration_sec
            if not overlaps_with_events(event_times, win_start, win_end, exclude_margin_sec):
                data = tr.slice(starttime=win_start, endtime=win_end).data[:window_length]
                if len(data) == window_length:
                    waveform_samples.append(data)
                    class_labels.append(0)
                    binary_labels.append(0)
                    sid = f"neg_{neg_count:05d}"
                    sample_ids.append(sid)
                    feats = extract_band_energy_features(data)
                    metadata_records.append({
                        "sample_id": sid,
                        "is_earthquake": 0,
                        "label": 0,
                        "magnitude": np.nan,
                        "magnitude_category": "noise",
                        "window_start": str(win_start),
                        "window_end": str(win_end),
                        **feats
                    })
                    neg_count += 1
            win_start += sliding_step_sec

    except Exception as e:
        print(f"Error processing {file_name}: {e}")
        continue

# --- SAVE OUTPUT FILES ---
np.savez_compressed(output_npz_path,
                    waveforms=np.array(waveform_samples),
                    labels=np.array(class_labels),
                    sample_types=np.array(binary_labels),
                    sample_ids=np.array(sample_ids))

csv_df = pd.DataFrame(metadata_records)
csv_df.to_csv(output_csv_path, index=False)
print("✅ Dataset saved: seismic_dataset.npz and seismic_features.csv")


Error processing MAJO_2011-03-01.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-02.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-03.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-04.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-05.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-06.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-07.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-08.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-09.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-10.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-11.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-12.mseed: 'datetime.date' object is not callable
Error processing MAJO_2011-03-13.mseed: 'datetime.da

In [1]:
import pandas as pd
import numpy as np
from obspy import UTCDateTime, read
from glob import glob
import os

# --- CONFIGURATIONS ---
sampling_rate = 20
window_duration_sec = 90
window_length = sampling_rate * window_duration_sec
pre_event_sec = 20
post_event_sec = 70
exclude_margin_sec = 120
sliding_step_sec = 45

# --- PATHS ---
event_file = "tohoku_earthquake_features_2011.csv"
waveform_dir = "waveforms"
output_npz_path = "seismic_dataset.npz"
output_csv_path = "seismic_features.csv"

# --- LOAD EVENT DATA ---
events_df = pd.read_csv(event_file)
events_df["event_time"] = pd.to_datetime(events_df["event_time"])
event_times = [UTCDateTime(t) for t in events_df["event_time"]]

# --- HELPERS ---
def classify_magnitude(mag):
    if mag < 5.0:
        return "small"
    elif 5.0 <= mag < 6.0:
        return "moderate"
    else:
        return "large"

def extract_band_energy_features(signal, fs=20):
    from scipy.signal import butter, sosfilt
    def band_energy(x, fmin, fmax):
        sos = butter(4, [fmin, fmax], btype='band', fs=fs, output='sos')
        filtered = sosfilt(sos, x)
        return np.sum(filtered ** 2)
    return {
        "energy_0.5_2Hz": band_energy(signal, 0.5, 2.0),
        "energy_2_5Hz": band_energy(signal, 2.0, 5.0),
        "energy_5_8Hz": band_energy(signal, 5.0, 8.0)
    }

def overlaps_with_events(event_times, start, end, margin):
    for event_time in event_times:
        if (start - margin) <= event_time <= (end + margin):
            return True
    return False

# --- PROCESSING ---
waveform_samples = []
class_labels = []
binary_labels = []
sample_ids = []
metadata_records = []

mseed_files = sorted(glob(os.path.join(waveform_dir, "*.mseed")))
pos_count, neg_count = 0, 0

for file_path in mseed_files:
    file_name = os.path.basename(file_path)
    try:
        st = read(file_path)
        st.detrend("demean")
        st.filter("bandpass", freqmin=0.1, freqmax=8.0)
        tr = st[0]

        t_start = tr.stats.starttime
        t_end = tr.stats.endtime

        # --- POSITIVE SAMPLES ---
        day_events = events_df[events_df["event_time"].dt.date == t_start.date]
        for _, row in day_events.iterrows():
            t0 = UTCDateTime(row["event_time"]) - pre_event_sec
            t1 = t0 + window_duration_sec
            if t0 < t_start or t1 > t_end:
                continue
            data = tr.slice(starttime=t0, endtime=t1).data[:window_length]
            if len(data) == window_length:
                mag_category = classify_magnitude(row["magnitude"])
                waveform_samples.append(data)
                class_labels.append({"small": 1, "moderate": 2, "large": 3}[mag_category])
                binary_labels.append(1)
                sid = f"pos_{pos_count:05d}"
                sample_ids.append(sid)
                feats = extract_band_energy_features(data)
                metadata_records.append({
                    "sample_id": sid,
                    "is_earthquake": 1,
                    "label": {"small": 1, "moderate": 2, "large": 3}[mag_category],
                    "magnitude": row["magnitude"],
                    "magnitude_category": mag_category,
                    "window_start": str(t0),
                    "window_end": str(t1),
                    **feats
                })
                pos_count += 1

        # --- NEGATIVE SAMPLES ---
        win_start = t_start
        while win_start + window_duration_sec <= t_end:
            win_end = win_start + window_duration_sec
            if not overlaps_with_events(event_times, win_start, win_end, exclude_margin_sec):
                data = tr.slice(starttime=win_start, endtime=win_end).data[:window_length]
                if len(data) == window_length:
                    waveform_samples.append(data)
                    class_labels.append(0)
                    binary_labels.append(0)
                    sid = f"neg_{neg_count:05d}"
                    sample_ids.append(sid)
                    feats = extract_band_energy_features(data)
                    metadata_records.append({
                        "sample_id": sid,
                        "is_earthquake": 0,
                        "label": 0,
                        "magnitude": np.nan,
                        "magnitude_category": "noise",
                        "window_start": str(win_start),
                        "window_end": str(win_end),
                        **feats
                    })
                    neg_count += 1
            win_start += sliding_step_sec

    except Exception as e:
        print(f"Error processing {file_name}: {e}")
        continue

# --- SAVE OUTPUT FILES ---
np.savez_compressed(output_npz_path,
                    waveforms=np.array(waveform_samples),
                    labels=np.array(class_labels),
                    sample_types=np.array(binary_labels),
                    sample_ids=np.array(sample_ids))

csv_df = pd.DataFrame(metadata_records)
csv_df.to_csv(output_csv_path, index=False)
print("✅ Dataset saved: seismic_dataset.npz and seismic_features.csv")


✅ Dataset saved: seismic_dataset.npz and seismic_features.csv


### Appendix Fix-U0: Unifying Dataset for Models

This utility script converts your generated outputs into a single, standardized NPZ file that both the CNN model and the XGBoost baseline can consume.

**Inputs**
- `seismic_dataset.npz` — arrays: `waveforms`, `labels` (pos: 1/2/3; neg: 0), `sample_types` (binary 0/1), `sample_ids`
- `seismic_features.csv` — metadata table with `sample_id`, `window_start`, `window_end`, …

**Processing**
- Ensures files exist and loads arrays.
- Derives **detection label** `detect_label` from `sample_types` (fallback: `labels > 0`).
- Builds **magnitude class (int)** `mag_class` as `0/1/2` for S/M/L and `-1` for noise.
- Builds **magnitude class (str)** `mag_cls` as `"S"|"M"|"L"|"noise"`.
- Aligns `window_start`/`window_end` from the CSV to the NPZ order via `sample_id` (raises error if mismatched).
- Stores sampling metadata: `fs=20`, `win_sec=90`.

**Output**
- `data/wave_mag_dataset.npz` containing:
  - `waveforms`, `detect_label`, `mag_class`, `mag_cls`, `sample_ids`
  - `window_start`, `window_end`, `fs`, `win_sec`

This creates a **unified contract** for downstream training and evaluation while preserving sample ordering and window metadata.


In [3]:
# Appendix Fix-U0 — Build data/wave_mag_dataset.npz from your existing outputs
# Inputs required (from your generator script):
#   - seismic_dataset.npz    (waveforms, labels, sample_types, sample_ids)
#   - seismic_features.csv   (sample_id, window_start, window_end, magnitude_category, ...)
# Output:
#   - data/wave_mag_dataset.npz (unified contract for CNN & baseline)

import os, numpy as np, pandas as pd
os.makedirs("data", exist_ok=True)

SRC_NPZ = "seismic_dataset.npz"
SRC_CSV = "seismic_features.csv"
DST_NPZ = "data/wave_mag_dataset.npz"

assert os.path.exists(SRC_NPZ), f"Missing {SRC_NPZ}"
assert os.path.exists(SRC_CSV), f"Missing {SRC_CSV}"

d = np.load(SRC_NPZ, allow_pickle=True)
waves = d["waveforms"].astype(np.float32)
sids  = d["sample_ids"].astype(str)

# detect_label: prefer your binary channel 'sample_types' (0/1). Fallback to labels>0.
if "sample_types" in d.files:
    detect = d["sample_types"].astype(np.int8)
else:
    detect = (d["labels"].astype(np.int32) > 0).astype(np.int8)

# mag_class (int): 0/1/2 for S/M/L, and -1 for noise (negatives)
labs = d["labels"].astype(np.int32)  # your positives: 1/2/3 ; negatives: 0
mag_class = np.where(detect==1, labs-1, -1).astype(np.int8)

# mag_cls (str): "S"/"M"/"L"/"noise"
mag_cls = np.array(["noise"] * len(mag_class), dtype=object)
map_int2str = {0:"S", 1:"M", 2:"L"}
pos_idx = np.where(mag_class >= 0)[0]
for i in pos_idx:
    mag_cls[i] = map_int2str[int(mag_class[i])]

# attach window_start / window_end from your CSV metadata
meta = pd.read_csv(SRC_CSV)
need_cols = {"sample_id", "window_start", "window_end"}
missing = need_cols - set(meta.columns)
if missing:
    raise ValueError(f"{SRC_CSV} missing columns: {missing}")

meta = meta.set_index("sample_id")[["window_start","window_end"]]
# align rows to NPZ sample order
try:
    win_start = meta.loc[sids, "window_start"].astype(str).values
    win_end   = meta.loc[sids, "window_end"].astype(str).values
except KeyError as e:
    raise KeyError(f"sample_id mismatch between NPZ and CSV: {e}")

# sampling config (from your generator)
fs = np.int32(20)       # sampling_rate
win_sec = np.int32(90)  # window_duration_sec

np.savez_compressed(
    DST_NPZ,
    waveforms=waves,
    detect_label=detect,
    mag_class=mag_class,
    mag_cls=mag_cls,
    sample_ids=sids,
    window_start=win_start,
    window_end=win_end,
    fs=fs,
    win_sec=win_sec,
)
print("[OK] wrote", DST_NPZ)
print("waveforms shape:", waves.shape, "fs:", int(fs), "win_sec:", int(win_sec))


[OK] wrote data/wave_mag_dataset.npz
waveforms shape: (26540, 1800) fs: 20 win_sec: 90


### Appendix CNN-1: Unified NPZ Construction (Final)

This script builds a unified dataset (`data/wave_mag_dataset.npz`) by combining the **Tohoku earthquake catalog** with the **continuous MiniSEED waveforms** from station *IU.MAJO.00.BHZ*.  
The resulting NPZ is the **standardized input** for both the CNN+BiLSTM+Attention model and the XGBoost baseline.

**Main steps:**
- **Catalog normalization:**  
  Convert event times to UTC-naive timestamps; map magnitudes to three categories (*S, M, L*).  
- **Positive sample extraction:**  
  For each catalogued event, extract a 90s window (20s pre + 70s post).  
- **Negative sample extraction:**  
  Sliding 90s windows (step = 45s) from non-event periods, excluding ±120s around events.  
  A per-file cap limits negatives to ≤3× positives.  
- **Waveform preprocessing:**  
  Detrend, bandpass filter (0.1–8 Hz), and resample to 20 Hz (final length = 1800 samples).  

**Output file:**
- `data/wave_mag_dataset.npz`  
  - `waveforms`: (N, T) float32 array of windows  
  - `detect_label`: binary 0/1 (noise vs earthquake)  
  - `mag_class`: int {0=S, 1=M, 2=L, -1=noise}  
  - `mag_cls`: str {"S","M","L","noise"}  
  - `sample_ids`: unique identifiers  
  - `window_start`, `window_end`: ISO timestamps  
  - `fs`: sampling rate (20 Hz)  
  - `win_sec`: window duration (90 s)  

**Summary:**  
This NPZ provides a consistent **contract** for training and evaluating both baseline and deep learning models, with aligned labels, metadata, and waveform arrays.


In [6]:
# Appendix CNN-1 (final fixed) — Build a unified NPZ from Tohoku catalog + MiniSEED waveforms
# Output: data/wave_mag_dataset.npz
# This NPZ is shared by both the main model (CNN+BiLSTM+Attention) and the XGBoost baseline.

import os, numpy as np, pandas as pd
from glob import glob
from obspy import read, UTCDateTime

# ----------------- Configs -----------------
waveform_dir        = "waveforms"                          # where your *.mseed files live
tohoku_csv          = "tohoku_earthquake_features_2011.csv"
npz_out             = "data/wave_mag_dataset.npz"
os.makedirs("data", exist_ok=True)

target_fs           = 20        # resample/ensure 20 Hz so each 90 s window has T = 1800 samples
win_sec             = 90        # 20 s pre + 70 s post
pre_event_sec       = 20
post_event_sec      = 70
exclude_margin_sec  = 120       # negatives avoid events by ±120 s
slide_step_sec      = 45        # negative sliding step (seconds)
max_neg_ratio       = 3.0       # per-file cap: negatives <= 3x positives (avoid huge imbalance)
bandpass_min, bandpass_max = 0.1, 8.0   # preprocessing band

# ----------------- Load & normalize catalog (Tohoku) -----------------
cat = pd.read_csv(tohoku_csv)

# KEY FIX 1: unify to UTC-naive pandas Timestamps to avoid tz comparison errors.
# If CSV is in UTC (e.g., ends with 'Z'), utc=True parses as tz-aware; tz_localize(None) drops tz info but keeps UTC clock time.
cat["event_time"] = pd.to_datetime(cat["event_time"], utc=True).dt.tz_localize(None)

# Normalize magnitude categories to S/M/L (strings). If absent, bin numeric magnitude.
def to_SML(x: str) -> str:
    x = str(x).lower()
    if x.startswith("s"): return "S"
    if x.startswith("m"): return "M"  # moderate / medium
    if x.startswith("l"): return "L"
    # fallback if unknown
    return "S"

if "magnitude_category" in cat.columns:
    cat["mag_cls"] = cat["magnitude_category"].map(to_SML)
elif "magnitude" in cat.columns:
    bins   = [-np.inf, 5.0, 6.0, np.inf]  # S: <5.0, M: [5.0,6.0), L: ≥6.0
    labels = ["S","M","L"]
    cat["mag_cls"] = pd.cut(cat["magnitude"].astype(float), bins=bins, labels=labels, right=False)
else:
    raise ValueError("Tohoku CSV must have 'magnitude_category' or 'magnitude' to form S/M/L.")

# For efficient negative masking in pandas space (stay tz-naive everywhere)
event_times_pd = cat["event_time"]

def overlaps_with_events_pd(event_times: pd.Series, start_ts: pd.Timestamp, end_ts: pd.Timestamp, margin_sec: int) -> bool:
    """Return True if any event_time falls within [start_ts - margin, end_ts + margin]."""
    margin = pd.to_timedelta(margin_sec, unit="s")
    window_start = start_ts - margin
    window_end   = end_ts + margin
    # Vectorized check using boolean mask
    return ((event_times >= window_start) & (event_times <= window_end)).any()

# ----------------- Containers -----------------
waves, y_detect, y_mag_int, y_mag_str = [], [], [], []
sample_ids, t_start_list, t_end_list  = [], [], []

pos_total = 0
neg_total = 0

# ----------------- Iterate over MiniSEED files -----------------
mseed_files = sorted(glob(os.path.join(waveform_dir, "*.mseed")))
if not mseed_files:
    raise FileNotFoundError(f"No .mseed files found under: {waveform_dir}")

for fp in mseed_files:
    added_pos, added_neg = 0, 0
    try:
        st = read(fp)
        st.detrend("demean")
        st.filter("bandpass", freqmin=bandpass_min, freqmax=bandpass_max)
        tr = st[0]

        # Resample to target_fs if needed
        if abs(tr.stats.sampling_rate - target_fs) > 1e-6:
            tr.resample(sampling_rate=target_fs)

        fs = int(round(tr.stats.sampling_rate))
        assert fs == target_fs, f"Unexpected fs={fs} after resample."
        win_len = fs * win_sec

        t0_file, t1_file = tr.stats.starttime, tr.stats.endtime

        # KEY FIX 2: build tz-naive pandas Timestamps directly from UTCDateTime.datetime (no string round-trip).
        safe_start = pd.Timestamp((t0_file + pre_event_sec).datetime)  # tz-naive
        safe_end   = pd.Timestamp((t1_file - post_event_sec).datetime) # tz-naive

        # Events whose full [t-20s, t+70s] window fits inside this file coverage
        try:
            evs = cat[(cat["event_time"] >= safe_start) & (cat["event_time"] <= safe_end)]
        except Exception as e:
            print(f"[WARN] {os.path.basename(fp)}: event filter error: {e}")
            evs = cat.iloc[0:0]  # empty to keep running

        # ---- Positives: event-driven windows (Tohoku -> MiniSEED) ----
        for _, row in evs.iterrows():
            ev_ts = row["event_time"]                               # pandas TS (tz-naive)
            ev_utc = UTCDateTime(ev_ts.to_pydatetime())             # KEY FIX 4: avoid string; build UTCDateTime safely
            t0 = ev_utc - pre_event_sec
            t1 = ev_utc + post_event_sec

            x = tr.slice(starttime=t0, endtime=t1).data
            if len(x) < win_len:      # guard against gaps
                continue
            x = x[:win_len].astype(np.float32)

            waves.append(x)
            y_detect.append(1)
            mag_map = {"S":0, "M":1, "L":2}
            y_mag_int.append(mag_map[str(row["mag_cls"])])
            y_mag_str.append(str(row["mag_cls"]))
            sid = f"pos_{pos_total:06d}"
            sample_ids.append(sid)
            t_start_list.append(str(t0))
            t_end_list.append(str(t1))
            pos_total += 1
            added_pos += 1

        # ---- Negatives: sliding windows avoiding events ± margin ----
        neg_limit = int(max_neg_ratio * max(1, added_pos))  # per-file cap
        win_start = t0_file
        while (win_start + win_sec) <= t1_file and added_neg < neg_limit:
            win_end = win_start + win_sec

            # KEY FIX 3: negative overlap check in tz-naive pandas space
            start_ts = pd.Timestamp(win_start.datetime)
            end_ts   = pd.Timestamp(win_end.datetime)

            if not overlaps_with_events_pd(event_times_pd, start_ts, end_ts, exclude_margin_sec):
                x = tr.slice(starttime=win_start, endtime=win_end).data
                if len(x) >= win_len:
                    x = x[:win_len].astype(np.float32)
                    waves.append(x)
                    y_detect.append(0)
                    y_mag_int.append(-1)          # -1 for noise
                    y_mag_str.append("noise")
                    sid = f"neg_{neg_total:06d}"
                    sample_ids.append(sid)
                    t_start_list.append(str(win_start))
                    t_end_list.append(str(win_end))
                    neg_total += 1
                    added_neg += 1

            win_start += slide_step_sec

        print(f"[{os.path.basename(fp)}] +pos={added_pos}, +neg={added_neg}, totals pos={pos_total}, neg={neg_total}")

    except Exception as e:
        print(f"[WARN] {os.path.basename(fp)}: {e}")

# ----------------- Save NPZ (unified contract) -----------------
waves = np.asarray(waves, dtype=np.float32)
detect = np.asarray(y_detect, dtype=np.int8)
mag_i  = np.asarray(y_mag_int, dtype=np.int8)
mag_s  = np.asarray(y_mag_str, dtype=object)
sids   = np.asarray(sample_ids, dtype=object)
wst    = np.asarray(t_start_list, dtype=object)
wed    = np.asarray(t_end_list, dtype=object)

np.savez_compressed(
    npz_out,
    waveforms=waves,               # (N, T)
    detect_label=detect,           # 0/1
    mag_class=mag_i,               # 0/1/2 for S/M/L; -1 for noise
    mag_cls=mag_s,                 # "S"/"M"/"L"/"noise"
    sample_ids=sids,
    window_start=wst,
    window_end=wed,
    fs=np.int32(target_fs),
    win_sec=np.int32(win_sec),
)

# Quick summary
def counts(arr):
    if len(arr) == 0: return {}
    keys, vals = np.unique(arr, return_counts=True)
    return {int(k) if isinstance(k, (np.integer,)) else str(k): int(v) for k, v in zip(keys, vals)}

print(f"[OK] Saved -> {npz_out}")
print("Shapes:", waves.shape, "| detect_label counts:", counts(detect))
print("mag_class counts (incl. -1 noise):", counts(mag_i))


[MAJO_2011-03-01.mseed] +pos=10, +neg=30, totals pos=10, neg=30
[MAJO_2011-03-02.mseed] +pos=5, +neg=15, totals pos=15, neg=45
[MAJO_2011-03-03.mseed] +pos=8, +neg=24, totals pos=23, neg=69
[MAJO_2011-03-04.mseed] +pos=9, +neg=27, totals pos=32, neg=96
[MAJO_2011-03-05.mseed] +pos=10, +neg=30, totals pos=42, neg=126
[MAJO_2011-03-06.mseed] +pos=11, +neg=33, totals pos=53, neg=159
[MAJO_2011-03-07.mseed] +pos=12, +neg=36, totals pos=65, neg=195
[MAJO_2011-03-08.mseed] +pos=8, +neg=24, totals pos=73, neg=219
[MAJO_2011-03-09.mseed] +pos=138, +neg=414, totals pos=211, neg=633
[MAJO_2011-03-10.mseed] +pos=61, +neg=183, totals pos=272, neg=816
[MAJO_2011-03-11.mseed] +pos=1319, +neg=406, totals pos=1591, neg=1222
[MAJO_2011-03-12.mseed] +pos=1082, +neg=4, totals pos=2673, neg=1226
[MAJO_2011-03-13.mseed] +pos=803, +neg=58, totals pos=3476, neg=1284
[MAJO_2011-03-14.mseed] +pos=660, +neg=67, totals pos=4136, neg=1351
[MAJO_2011-03-15.mseed] +pos=613, +neg=89, totals pos=4749, neg=1440
[MAJO_

# Dataset Description

**Provenance**

* **Continuous waveforms (MiniSEED):** IU.MAJO.00.BHZ, time span **2011-03-01 → 2011-03-20**.
* **Event catalog:** `tohoku_earthquake_features_2011.csv` (contains `event_time` and magnitude info).
* **Tasks:** (i) 3-class magnitude classification **S/M/L** (Small/Moderate/Large), (ii) fair comparison to a feature-engineered baseline (XGBoost + band-energy features).

**Construction Pipeline**

1. Parse catalog `event_time` as **UTC-naive** timestamps to avoid timezone comparison issues.
2. Detrend and band-pass filter waveforms to **0.1–8 Hz**, resample to **20 Hz**.
3. **Positives:** for each catalog event, extract a **90 s** window **\[–20 s, +70 s]** around the event time.
4. **Negatives:** slide a 90 s window every **45 s**, but **exclude** any window overlapping **±120 s** around **any** catalog event; per file, cap negatives to **≤ 3×** that file’s positives (mitigates extreme imbalance).
5. Package everything into `data/wave_mag_dataset.npz`. Both the deep model and the baseline **share exactly the same windows** and **the same time split**.

**Labels**

* `detect_label`: **0/1** = noise/earthquake.
* `mag_cls`: **"S"/"M"/"L"/"noise"**.
* `mag_class`: **0/1/2/-1** = **S/M/L/noise** (used for training).

**Windowing & Preprocessing**

* Sampling rate (`fs`): **20 Hz**.
* Window length (`win_sec`): **90 s** → **T = 90 × 20 = 1800** samples.
* Preprocessing: detrend (demean) + **0.1–8 Hz** band-pass.
* Negative exclusion: no overlap with any catalog event within **±120 s**.

**Files & Schemas**

* **Unified dataset (for both CNN and Baseline):** `data/wave_mag_dataset.npz`

  * `waveforms`: shape `(N, 1800)`, `float32`
  * `detect_label`: `(N,)`, `int8` (0/1)
  * `mag_class`: `(N,)`, `int8` (0/1/2/-1)
  * `mag_cls`: `(N,)`, `object` ("S"/"M"/"L"/"noise")
  * `sample_ids`, `window_start`, `window_end`, `fs`=20, `win_sec`=90
* **Derived features (for XGBoost baseline):**

  * `data/features_from_npz_detect.csv` (all windows, incl. 0/1)
  * `data/features_from_npz_mag.csv` (positives only, S/M/L)
  * Columns: `sample_id, window_start, detect_label, mag_cls, energy_0.5_2Hz, energy_2_5Hz, energy_5_8Hz`
* **Frozen split (shared by both models):** `runs/frozen_splits.json` (time-based 80/20, stored by `sample_id`).

**Current Build Stats**

* Total windows **N = 9,858**; each window length **T = 1,800**.
* `detect_label` counts: **1 → 6,956** (earthquake), **0 → 2,902** (noise).
* `mag_class` counts: **-1 → 2,902** (noise); S/M/L distribution follows the catalog and is validated in code (QC cell).

**Fair Comparison**

* The deep model (CNN + BiLSTM + Attention) and the XGBoost baseline **strictly share**:

  1. **The same set of waveform windows** (from the NPZ), and
  2. **The same time-based split** (frozen by `sample_id` in `runs/frozen_splits.json`).
* This removes selection bias and ensures a true **apples-to-apples** evaluation.

**Reproducibility Notes**

* Key parameters: `fs=20 Hz`, `win_sec=90 s`, band-pass `0.1–8 Hz`, negative exclusion `±120 s`, negative cap `≤3×`, slide step `45 s`, time split `80/20`.
* Rebuilding the dataset may change counts, but file structure and training pipeline remain identical.


### QC: Validate and Hotfix Magnitude Labels

This script performs a quick quality check on `data/wave_mag_dataset.npz` and applies a minimal hotfix if label collapse is detected.

- **Check:**  
  Load `detect_label` and `mag_class` and print class counts (note: `-1` denotes noise).  
  If the **positive subset** (`detect_label == 1`) of `mag_class` has **fewer than 3 unique values**, it indicates label collapse.

- **Hotfix:**  
  Rebuild integer magnitude labels from the string labels `mag_cls` using:  
  `{"S": 0, "M": 1, "L": 2, "noise": -1}`.  
  Save back to the same NPZ, preserving all other arrays and metadata.

- **Outcome:**  
  Ensures S/M/L classes are correctly encoded for training while keeping dataset structure intact.


In [7]:
# QC — check mag labels; hotfix if needed
import numpy as np, pandas as pd

NPZ = "data/wave_mag_dataset.npz"
d = np.load(NPZ, allow_pickle=True)

det = d["detect_label"].astype(int)
mag_i = d["mag_class"].astype(int)
print("detect_label counts:", pd.Series(det).value_counts().to_dict())
print("mag_class counts:", pd.Series(mag_i).value_counts().to_dict())  # -1 是 noise

# 如果你发现正样本的 mag_class 只有一个值（例如全是 1），用字符串标签重建：
if pd.Series(mag_i[det==1]).nunique() < 3:
    map2int = {"S":0, "M":1, "L":2, "noise":-1}
    mag_cls = d["mag_cls"].astype(str)
    mag_new = np.array([map2int.get(s, -1) for s in mag_cls], dtype=np.int8)
    np.savez_compressed(
        NPZ,
        waveforms=d["waveforms"], detect_label=det,
        mag_class=mag_new, mag_cls=d["mag_cls"],
        sample_ids=d["sample_ids"], window_start=d["window_start"], window_end=d["window_end"],
        fs=d["fs"], win_sec=d["win_sec"],
    )
    print("[OK] mag_class rebuilt from mag_cls.")


detect_label counts: {1: 6956, 0: 2902}
mag_class counts: {1: 6956, -1: 2902}
[OK] mag_class rebuilt from mag_cls.


In [8]:
import numpy as np, pandas as pd
d = np.load("data/wave_mag_dataset.npz", allow_pickle=True)

det = d["detect_label"].astype(int)
mag_i = d["mag_class"].astype(int)      # 0:S, 1:M, 2:L, -1:noise
mag_s = d["mag_cls"].astype(str)

print("mag_cls (strings) among positives:\n", pd.Series(mag_s[det==1]).value_counts())
print("\nmag_class (ints) among positives:\n", pd.Series(mag_i[det==1]).value_counts().sort_index())


mag_cls (strings) among positives:
 M    6956
dtype: int64

mag_class (ints) among positives:
 1    6956
dtype: int64


### QC Repair: Align Positive Labels to Catalog (Timezone-Safe)

**Goal:**  
Timezone-safe realignment of **positive windows** in `data/wave_mag_dataset.npz` to the Tohoku catalog (`tohoku_earthquake_features_2011.csv`) and **rebuilding S/M/L labels** from catalog magnitudes.

**Inputs**
- `data/wave_mag_dataset.npz`: contains `waveforms`, `detect_label`, `mag_class`, `mag_cls`, `window_start`, `window_end`, `fs`, `win_sec`, …
- `tohoku_earthquake_features_2011.csv`: contains `event_time`, `magnitude`, and optional `magnitude_category`.

**Method**
1. **Select positives**: `detect_label == 1`.  
2. **Reconstruct event time** per window: `evt_from_window = window_start + 20 s`.  
   - Parse to **UTC-naive** timestamps to avoid timezone comparison errors.  
3. **Nearest-time join** to catalog `event_time` with a tolerance of **5 s**, and fallback to **10 s** if needed.  
4. **Derive S/M/L** from catalog:  
   - Prefer numeric `magnitude` thresholds: S (<5.0), M ([5.0,6.0)), L (≥6.0).  
   - Fallback to `magnitude_category` text if numeric magnitude is missing.  
5. **Write back to NPZ**:  
   - Update matched **positives**: `mag_class` (0/1/2 for S/M/L), `mag_cls` ("S"/"M"/"L").  
   - Force all **negatives** to `"noise"` / `-1`.  
   - Preserve all other arrays and metadata.

**Outputs & Checks**
- Overwrites `data/wave_mag_dataset.npz` in place (schema unchanged).  
- Logs **matched / unmatched** counts.  
- Prints **label distributions** among positives for sanity check.

**Why this matters**
- Ensures positive labels (S/M/L) are **catalog-consistent** and **timezone-robust**, preventing silent label drift or class collapse before training.


In [11]:
# Repair mag_class/mag_cls by aligning positives to the Tohoku catalog (timezone-safe)
import numpy as np, pandas as pd

NPZ = "data/wave_mag_dataset.npz"
CSV = "tohoku_earthquake_features_2011.csv"

d = np.load(NPZ, allow_pickle=True)

# --- 1) Select positive windows and compute event times (UTC-naive) ---
mask_pos = d["detect_label"].astype(int) == 1

# NOTE: to_datetime returns a DatetimeIndex here; use .tz_localize(None) directly (no .dt)
wstart_idx = pd.to_datetime(d["window_start"][mask_pos], utc=True).tz_localize(None)
evt_from_window = wstart_idx + pd.to_timedelta(20, "s")   # event time = window_start + 20s

pos_df = pd.DataFrame({
    "idx": np.arange(len(d["sample_ids"]))[mask_pos],
    "evt_from_window": evt_from_window
}).sort_values("evt_from_window").reset_index(drop=True)

# --- 2) Load catalog and normalize to UTC-naive ---
cat = pd.read_csv(CSV)
if "event_time" not in cat.columns:
    raise ValueError("Catalog must contain 'event_time'.")
cat["event_time"] = pd.to_datetime(cat["event_time"], utc=True).dt.tz_localize(None)
cat = cat.sort_values("event_time").reset_index(drop=True)

# --- 3) Nearest-time join (try 5s tolerance, fallback to 10s) ---
def nearest_join(tol_seconds: int):
    return pd.merge_asof(
        pos_df,
        cat.rename(columns={"event_time":"evt_cat"})[
            ["evt_cat"] + [c for c in cat.columns if c != "event_time"]
        ],
        left_on="evt_from_window",
        right_on="evt_cat",
        direction="nearest",
        tolerance=pd.Timedelta(seconds=tol_seconds)
    )

m = nearest_join(5)
if m["evt_cat"].isna().any():
    m = nearest_join(10)

unmatched = int(m["evt_cat"].isna().sum())
total_pos = len(m)
print(f"[Info] positives={total_pos}, matched={total_pos - unmatched}, unmatched={unmatched}")

# --- 4) Derive S/M/L from catalog (prefer numeric magnitude; fallback to text category) ---
def to_SML_from_row(row):
    if "magnitude" in m.columns and pd.notna(row.get("magnitude", np.nan)):
        mag = float(row["magnitude"])
        if mag < 5.0: return "S"
        elif mag < 6.0: return "M"
        else: return "L"
    # Fallback to text category if magnitude missing
    catg = str(row.get("magnitude_category", "")).lower()
    if catg.startswith("s"): return "S"
    if catg.startswith("m"): return "M"   # moderate / medium
    if catg.startswith("l"): return "L"
    return "M"  # conservative fallback

m["mag_cls_new"] = m.apply(to_SML_from_row, axis=1)
map2int = {"S":0, "M":1, "L":2}

# --- 5) Write back to NPZ (update positives that matched; keep negatives as noise) ---
mag_class = d["mag_class"].astype(int).copy()
mag_cls   = d["mag_cls"].astype(object).copy()

matched_mask = m["evt_cat"].notna()
mag_class[m.loc[matched_mask, "idx"].values] = m.loc[matched_mask, "mag_cls_new"].map(map2int).astype(np.int8).values
mag_cls[m.loc[matched_mask, "idx"].values]   = m.loc[matched_mask, "mag_cls_new"].values

# Ensure negatives are noise
neg_mask = d["detect_label"].astype(int) == 0
mag_class[neg_mask] = -1
mag_cls[neg_mask]   = "noise"

np.savez_compressed(
    NPZ,
    waveforms=d["waveforms"],
    detect_label=d["detect_label"],
    mag_class=mag_class,
    mag_cls=mag_cls,
    sample_ids=d["sample_ids"],
    window_start=d["window_start"],
    window_end=d["window_end"],
    fs=d["fs"],
    win_sec=d["win_sec"],
)

# --- 6) Quick report ---
print("[OK] Saved fixed mag_class/mag_cls to", NPZ)
print("mag_cls among positives:\n", pd.Series(mag_cls[d["detect_label"].astype(int)==1]).value_counts())
print("mag_class among positives:\n", pd.Series(mag_class[d["detect_label"].astype(int)==1]).value_counts().sort_index())


[Info] positives=6956, matched=6956, unmatched=0
[OK] Saved fixed mag_class/mag_cls to data/wave_mag_dataset.npz
mag_cls among positives:
 S    6382
M     515
L      59
dtype: int64
mag_class among positives:
 0    6382
1     515
2      59
dtype: int64


### Frozen Time-Based Split (`runs/frozen_splits.json`)

This script creates **reproducible, time-ordered 80/20 splits** by `sample_id` for two tasks:

- **Detection (`detect`)**: uses **all windows** (earthquake + noise).  
  Sort by `window_start`; take the **earliest 80%** as train and the **latest 20%** as test.

- **Magnitude classification (`magcls`)**: uses **positives only** (`detect_label==1` and `mag_class≥0`).  
  Apply the same **time-based 80/20** split on this subset.

**Output file**
- `runs/frozen_splits.json` with:
  - `detect.train_ids`, `detect.test_ids`
  - `magcls.train_ids`, `magcls.test_ids`

Using a single frozen split prevents temporal leakage and ensures **apples-to-apples** comparisons across CNN and XGBoost.


In [12]:
# Freeze a single time-based split, saved by sample_id
import numpy as np, pandas as pd, json

d = np.load("data/wave_mag_dataset.npz", allow_pickle=True)
sid = d["sample_ids"].astype(str)
ts  = pd.to_datetime(d["window_start"])
det = d["detect_label"].astype(int)
mag = d["mag_class"].astype(int)  # 0/1/2; -1 for noise

# 全体（含噪声）按时间 80/20 切，用于“检测任务”；三分类只用 S/M/L 子集再按时间 80/20 切
order_all = np.argsort(ts.values)
cut_all = int(0.8 * len(order_all))
split = {
    "detect": {
        "train_ids": sid[order_all[:cut_all]].tolist(),
        "test_ids":  sid[order_all[cut_all:]].tolist()
    }
}

mask_pos = (det==1) & (mag>=0)
sid_pos = sid[mask_pos]; ts_pos = ts[mask_pos]
order_pos = np.argsort(ts_pos.values)
cut_pos = int(0.8 * len(order_pos))
split["magcls"] = {
    "train_ids": sid_pos[order_pos[:cut_pos]].tolist(),
    "test_ids":  sid_pos[order_pos[cut_pos:]].tolist()
}

import os; os.makedirs("runs", exist_ok=True)
json.dump(split, open("runs/frozen_splits.json","w"))
print("[OK] wrote runs/frozen_splits.json")


[OK] wrote runs/frozen_splits.json


### Band-Energy Features from NPZ Windows

This script derives **band-energy features** from the unified waveform windows in `data/wave_mag_dataset.npz` for use by the XGBoost baseline and for exploratory analysis.

**Inputs**
- `data/wave_mag_dataset.npz`  
  Uses: `waveforms`, `fs`, `sample_ids`, `window_start`, `detect_label`, `mag_cls`.

**Method**
- For each window, z-score normalize (mean/std) and compute energy (sum of squared amplitudes) in three bands using 4th-order Butterworth SOS filters:
  - 0.5–2 Hz, 2–5 Hz, 5–8 Hz.
- Preserve metadata: `sample_id`, `window_start`, `detect_label`, `mag_cls`.
- Sort rows by `window_start`.

**Outputs**
- `data/features_from_npz_detect.csv` — **all windows** (earthquake + noise) with:
  - `sample_id`, `window_start`, `detect_label`, `mag_cls`,
  - `energy_0.5_2Hz`, `energy_2_5Hz`, `energy_5_8Hz`.
- `data/features_from_npz_mag.csv` — **positives only** (`detect_label==1`, `mag_cls ∈ {S,M,L}`) with the same columns.

**Purpose**
- Provides a compact, interpretable feature set for the **XGBoost baseline** and quick QC/analysis while staying aligned with the NPZ windows and time order.


In [13]:
# Derive band-energy features (0.5–2 / 2–5 / 5–8 Hz) from the NPZ windows
import numpy as np, pandas as pd, os
from scipy.signal import butter, sosfilt

os.makedirs("data", exist_ok=True)
d = np.load("data/wave_mag_dataset.npz", allow_pickle=True)

X   = d["waveforms"]
fs  = int(d["fs"])
sid = d["sample_ids"].astype(str)
wst = pd.to_datetime(d["window_start"])
det = d["detect_label"].astype(int)
mcs = d["mag_cls"].astype(str)

def band_energy(x, fmin, fmax, fs):
    sos = butter(4, [fmin, fmax], btype="band", fs=fs, output="sos")
    return float(np.sum(sosfilt(sos, x)**2))

rows = []
for i, x in enumerate(X):
    m, s = x.mean(), x.std()
    if s > 1e-8: x = (x - m) / s
    rows.append({
        "sample_id": sid[i],
        "window_start": wst[i],
        "detect_label": int(det[i]),
        "mag_cls": mcs[i],
        "energy_0.5_2Hz": band_energy(x, 0.5, 2.0, fs),
        "energy_2_5Hz":   band_energy(x, 2.0, 5.0, fs),
        "energy_5_8Hz":   band_energy(x, 5.0, 8.0, fs),
    })

feat = pd.DataFrame(rows).sort_values("window_start").reset_index(drop=True)
feat.to_csv("data/features_from_npz_detect.csv", index=False)
feat_pos = feat[(feat["detect_label"]==1) & (feat["mag_cls"].isin(["S","M","L"]))].copy()
feat_pos.to_csv("data/features_from_npz_mag.csv", index=False)
print("[OK] to data/features_from_npz_detect.csv:", feat.shape, 
      " | data/features_from_npz_mag.csv:", feat_pos.shape)


[OK] to data/features_from_npz_detect.csv: (9858, 7)  | data/features_from_npz_mag.csv: (6956, 7)


In [14]:
# XGBoost baseline on the same split
import json, pandas as pd, numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

splits = json.load(open("runs/frozen_splits.json"))
train_ids = set(splits["magcls"]["train_ids"])
test_ids  = set(splits["magcls"]["test_ids"])

df = pd.read_csv("data/features_from_npz_mag.csv", parse_dates=["window_start"])
df["y"] = df["mag_cls"].map({"S":0,"M":1,"L":2})

Xcols = ["energy_0.5_2Hz","energy_2_5Hz","energy_5_8Hz"]
X_tr = df[df["sample_id"].isin(train_ids)][Xcols].values
y_tr = df[df["sample_id"].isin(train_ids)]["y"].values
X_te = df[df["sample_id"].isin(test_ids)][Xcols].values
y_te = df[df["sample_id"].isin(test_ids)]["y"].values

try:
    from xgboost import XGBClassifier
    clf = Pipeline([
        ("scaler", StandardScaler(with_mean=False)),
        ("xgb", XGBClassifier(
            n_estimators=400, max_depth=4, learning_rate=0.05,
            subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
            objective="multi:softprob", num_class=3, random_state=42,
            eval_metric="mlogloss"
        ))
    ])
except Exception:
    from sklearn.ensemble import GradientBoostingClassifier
    clf = Pipeline([
        ("scaler", StandardScaler(with_mean=False)),
        ("gbdt", GradientBoostingClassifier(random_state=42))
    ])

clf.fit(X_tr, y_tr)
y_hat = clf.predict(X_te)
print("== XGBoost baseline (S/M/L) ==")
print(classification_report(y_te, y_hat, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_te, y_hat).astype(int))


== XGBoost baseline (S/M/L) ==
              precision    recall  f1-score   support

           0     0.9854    0.9897    0.9875      1361
           1     0.4400    0.3548    0.3929        31

    accuracy                         0.9756      1392
   macro avg     0.7127    0.6723    0.6902      1392
weighted avg     0.9732    0.9756    0.9743      1392

Confusion matrix:
 [[1347   14]
 [  20   11]]


In [15]:
# CNN+BiLSTM+Attention for S/M/L classification (same split)
import json, numpy as np, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix

# load NPZ
d = np.load("data/wave_mag_dataset.npz", allow_pickle=True)
X   = d["waveforms"]
det = d["detect_label"].astype(int)
mag = d["mag_class"].astype(int)
sid = d["sample_ids"].astype(str)

# keep only S/M/L positives
mask = (det==1) & (mag>=0)
X, y, sid = X[mask], mag[mask], sid[mask]

# map sample_id -> index for split
id2idx = {s:i for i,s in enumerate(sid)}
splits = json.load(open("runs/frozen_splits.json"))
tr_ids = [s for s in splits["magcls"]["train_ids"] if s in id2idx]
te_ids = [s for s in splits["magcls"]["test_ids"]  if s in id2idx]
tr_idx = np.array([id2idx[s] for s in tr_ids], dtype=int)
te_idx = np.array([id2idx[s] for s in te_ids], dtype=int)

class WaveDS(Dataset):
    def __init__(self, waves, labels):
        self.w = waves; self.y = labels.astype(np.int64)
    def __len__(self): return len(self.y)
    def __getitem__(self, i):
        x = self.w[i]
        m, s = x.mean(), x.std()
        if s > 1e-8: x = (x - m) / s
        return torch.from_numpy(x[None,:]), torch.tensor(self.y[i])

dl_tr = DataLoader(WaveDS(X[tr_idx], y[tr_idx]), batch_size=64, shuffle=True)
dl_te = DataLoader(WaveDS(X[te_idx], y[te_idx]), batch_size=128, shuffle=False)

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, n_classes=3, hidden=64):
        super().__init__()
        self.fe = nn.Sequential(
            nn.Conv1d(1,16,7,2,3), nn.BatchNorm1d(16), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(16,32,5,2,2), nn.BatchNorm1d(32), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(32,64,3,1,1), nn.BatchNorm1d(64), nn.ReLU()
        )
        self.lstm = nn.LSTM(64, hidden, num_layers=1, batch_first=True, bidirectional=True)
        self.attn_W = nn.Linear(2*hidden, 128)
        self.attn_v = nn.Linear(128, 1, bias=False)
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(2*hidden, n_classes)
    def forward(self, x):
        h = self.fe(x)              # (B,64,T')
        h = h.transpose(1,2)        # (B,T',64)
        h, _ = self.lstm(h)         # (B,T',2H)
        a = torch.tanh(self.attn_W(h))         # (B,T',128)
        a = self.attn_v(a).squeeze(-1)         # (B,T')
        w = torch.softmax(a, dim=1)            # (B,T')
        ctx = (h * w.unsqueeze(-1)).sum(1)     # (B,2H)
        z = self.drop(ctx)
        return self.fc(z)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNNBiLSTMAttn().to(device)
# class weights（按训练集频次反比）
counts = np.bincount(y[tr_idx], minlength=3)
cw = counts.sum() / np.maximum(1, counts)
cw = (cw / cw.mean()).astype(np.float32)
crit = nn.CrossEntropyLoss(weight=torch.tensor(cw, device=device))
opt  = torch.optim.AdamW(model.parameters(), lr=1e-3)

for ep in range(10):
    model.train()
    for xb, yb in dl_tr:
        xb, yb = xb.to(device), yb.to(device)
        loss = crit(model(xb), yb)
        opt.zero_grad(); loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        opt.step()
    print(f"epoch {ep+1} done.")

# eval
model.eval()
preds, gts = [], []
with torch.no_grad():
    for xb, yb in dl_te:
        p = model(xb.to(device)).argmax(1).cpu().numpy()
        preds.extend(p); gts.extend(yb.numpy())
print("== CNN+BiLSTM+Attn (S/M/L) ==")
print(classification_report(gts, preds, digits=4))
print("Confusion matrix:\n", confusion_matrix(gts, preds).astype(int))


/Users/donghui/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


epoch 1 done.
epoch 2 done.
epoch 3 done.
epoch 4 done.
epoch 5 done.
epoch 6 done.
epoch 7 done.
epoch 8 done.
epoch 9 done.
epoch 10 done.
== CNN+BiLSTM+Attn (S/M/L) ==
              precision    recall  f1-score   support

           0     0.9953    0.9375    0.9656      1361
           1     0.2170    0.7419    0.3358        31
           2     0.0000    0.0000    0.0000         0

    accuracy                         0.9332      1392
   macro avg     0.4041    0.5598    0.4338      1392
weighted avg     0.9780    0.9332    0.9515      1392

Confusion matrix:
 [[1276   83    2]
 [   6   23    2]
 [   0    0    0]]


/Users/donghui/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/donghui/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/donghui/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### QC Repair: Align Positive Labels to Catalog (Timezone-Safe)

**Purpose**  
Safely realign **positive windows** in `data/wave_mag_dataset.npz` to the Tohoku catalog and **rebuild S/M/L labels** from catalog magnitudes, avoiding timezone pitfalls.

**Inputs**  
- `data/wave_mag_dataset.npz` — expects keys: `waveforms`, `detect_label`, `mag_class`, `mag_cls`, `sample_ids`, `window_start`, `window_end`, `fs`, `win_sec`  
- `tohoku_earthquake_features_2011.csv` — must include `event_time` (and ideally `magnitude` / `magnitude_category`)

**Method (summary)**  
1. **Select positives**: `detect_label == 1`.  
2. **Reconstruct event time** for each window as `window_start + 20s` (UTC-naive).  
3. **Normalize catalog times** to UTC-naive and sort.  
4. **Nearest-neighbor time join** (`merge_asof`) using progressive tolerances **5s → 10s**.  
5. **Derive S/M/L**:  
   - Prefer numeric `magnitude`: S (<5.0), M ([5.0,6.0)), L (≥6.0).  
   - Fallback to `magnitude_category` text; final fallback = `M`.  
6. **Write back**: update matched positives’ `mag_class` (0/1/2) and `mag_cls` (“S/M/L”); force all negatives to `-1 / "noise"`.  
7. **Report**: print matched/unmatched counts and positive-class distributions (string & int).

**Output**  
- Overwrites `data/wave_mag_dataset.npz` in place (schema unchanged), with **catalog-consistent S/M/L labels** for matched positives.

**Why this matters**  
Prevents label drift/class collapse by anchoring positive labels to the authoritative catalog while being robust to timezone handling and small timing offsets.


In [16]:
"""
Repair mag_class/mag_cls by aligning positive windows to the Tohoku catalog.

What this script does:
1) Load your NPZ dataset and pick positive windows (detect_label == 1).
2) Convert window_start to timezone-naive UTC and define the event time as window_start + EVENT_OFFSET_SEC.
3) Load the external catalog CSV (must contain 'event_time'), convert to timezone-naive UTC.
4) Nearest-neighbor time join (merge_asof) with progressively relaxed tolerances.
5) Derive S/M/L class from numeric magnitude if available; otherwise fall back to text category.
6) Write updated labels back to the NPZ (only for matched positives); set negatives as noise.
7) Print a quick summary of matches and class distribution among positives.
"""

import numpy as np
import pandas as pd

# ====== CONFIG ======
NPZ = "data/wave_mag_dataset.npz"           # Input/Output NPZ
CSV = "tohoku_earthquake_features_2011.csv" # Catalog CSV; must include 'event_time'
EVENT_OFFSET_SEC = 20                       # Event time = window_start + 20s
TOLERANCES = [5, 10]                        # Merge tolerances in seconds (progressively relaxed)
# ====================

# --- 0) Load NPZ and verify required keys ---
d = np.load(NPZ, allow_pickle=True)

required_keys = [
    "waveforms", "detect_label", "mag_class", "mag_cls",
    "sample_ids", "window_start", "window_end", "fs", "win_sec"
]
missing = [k for k in required_keys if k not in d.files]
if missing:
    raise KeyError(f"NPZ is missing required fields: {missing}")

# --- 1) Select positive windows and compute event times (UTC-naive) ---
detect_label = d["detect_label"].astype(int)
mask_pos = detect_label == 1

# NOTE:
# - For ndarray/Datetime-like arrays, pd.to_datetime returns a DatetimeIndex.
# - You should NOT use `.dt` on a DatetimeIndex. Call `.tz_localize(None)` directly.
wstart_idx = pd.to_datetime(d["window_start"][mask_pos], utc=True).tz_localize(None)
evt_from_window = wstart_idx + pd.to_timedelta(EVENT_OFFSET_SEC, "s")

pos_df = (
    pd.DataFrame({
        "idx": np.arange(len(d["sample_ids"]))[mask_pos],
        "evt_from_window": evt_from_window
    })
    .sort_values("evt_from_window")
    .reset_index(drop=True)
)

# --- 2) Load catalog and normalize to UTC-naive ---
cat = pd.read_csv(CSV)
if "event_time" not in cat.columns:
    raise ValueError("Catalog CSV must contain a column named 'event_time'.")

# Here 'event_time' is a Series; using `.dt.tz_localize(None)` is correct.
cat["event_time"] = pd.to_datetime(cat["event_time"], utc=True).dt.tz_localize(None)
cat = cat.sort_values("event_time").reset_index(drop=True)

# Prepare the right-hand table for merge_asof
cat_renamed = cat.rename(columns={"event_time": "evt_cat"})
right_cols = ["evt_cat"] + [c for c in cat_renamed.columns if c != "evt_cat"]

# --- 3) Nearest-time join with tolerance fallback ---
def nearest_join(tol_seconds: int) -> pd.DataFrame:
    """
    Perform a nearest-neighbor as-of merge between positive windows and the catalog
    with a given tolerance (in seconds). Both sides must be sorted by their join keys.
    """
    return pd.merge_asof(
        pos_df,
        cat_renamed[right_cols],
        left_on="evt_from_window",
        right_on="evt_cat",
        direction="nearest",
        tolerance=pd.Timedelta(seconds=tol_seconds),
    )

m = None
for tol in TOLERANCES:
    m = nearest_join(tol)
    unmatched = int(m["evt_cat"].isna().sum())
    total_pos = len(m)
    print(f"[Try tol={tol}s] matched={total_pos - unmatched}, unmatched={unmatched}")
    if unmatched == 0:
        break

if m is None:
    raise RuntimeError("merge_asof produced no result; please check your inputs.")

unmatched = int(m["evt_cat"].isna().sum())
total_pos = len(m)
print(f"[Info] positives={total_pos}, matched={total_pos - unmatched}, unmatched={unmatched}")

# --- 4) Map magnitude to S/M/L (prefer numeric magnitude; fall back to text category) ---
def to_SML_from_row(row: pd.Series) -> str:
    """
    Priority:
      1) If numeric 'magnitude' present and not NaN: map by thresholds (<5: S, <6: M, else L).
      2) Else, use 'magnitude_category' prefix (s/m/l).
      3) Else, return 'M' as a conservative default.
    """
    # 1) Numeric magnitude
    if "magnitude" in row.index:
        val = row.get("magnitude", np.nan)
        if pd.notna(val):
            mag = float(val)
            if mag < 5.0:
                return "S"
            elif mag < 6.0:
                return "M"
            else:
                return "L"

    # 2) Text fallback
    catg = str(row.get("magnitude_category", "")).strip().lower()
    if catg.startswith("s"):
        return "S"
    if catg.startswith("m"):
        return "M"
    if catg.startswith("l"):
        return "L"

    # 3) Conservative default
    return "M"

m["mag_cls_new"] = m.apply(to_SML_from_row, axis=1)
map2int = {"S": 0, "M": 1, "L": 2}

# --- 5) Write back: only update matched positives; force negatives to noise ---
mag_class = d["mag_class"].astype(int).copy()
mag_cls   = d["mag_cls"].astype(object).copy()

matched_mask = m["evt_cat"].notna()
matched_idx = m.loc[matched_mask, "idx"].to_numpy()

# Update matched positive samples
mag_class[matched_idx] = (
    m.loc[matched_mask, "mag_cls_new"].map(map2int).astype(np.int8).to_numpy()
)
mag_cls[matched_idx] = m.loc[matched_mask, "mag_cls_new"].to_numpy()

# Explicitly set negatives to noise
neg_mask = detect_label == 0
mag_class[neg_mask] = -1
mag_cls[neg_mask] = "noise"

# --- 6) Save compressed NPZ with all required fields for downstream compatibility ---
np.savez_compressed(
    NPZ,
    waveforms=d["waveforms"],
    detect_label=d["detect_label"],
    mag_class=mag_class,
    mag_cls=mag_cls,
    sample_ids=d["sample_ids"],
    window_start=d["window_start"],
    window_end=d["window_end"],
    fs=d["fs"],
    win_sec=d["win_sec"],
)

# --- 7) Quick report (only among positives/events) ---
print("[OK] Saved fixed mag_class/mag_cls to", NPZ)

pos_mask_after = (detect_label == 1)
print(
    "mag_cls among positives:\n",
    pd.Series(mag_cls[pos_mask_after]).value_counts(),
)
print(
    "mag_class among positives:\n",
    pd.Series(mag_class[pos_mask_after]).value_counts().sort_index(),
)


[Try tol=5s] matched=6956, unmatched=0
[Info] positives=6956, matched=6956, unmatched=0
[OK] Saved fixed mag_class/mag_cls to data/wave_mag_dataset.npz
mag_cls among positives:
 S    6382
M     515
L      59
dtype: int64
mag_class among positives:
 0    6382
1     515
2      59
dtype: int64


### Stratified Split with Minimum L in Test (`runs/frozen_splits.json`)

**Purpose**  
Create a **stratified 80/20 split** for the **S/M/L** task that **guarantees at least K “L” samples** in the test set (to stabilize evaluation on rare large events).

**Inputs**
- `data/features_from_npz_mag.csv` with columns:
  - `sample_id`, `window_start`, `mag_cls` ∈ {S,M,L},
  - `energy_0.5_2Hz`, `energy_2_5Hz`, `energy_5_8Hz`.

**Method**
- Map labels: `{"S":0,"M":1,"L":2}` and build features `X = [energy_0.5_2Hz, energy_2_5Hz, energy_5_8Hz]`.
- Use `StratifiedShuffleSplit(test_size=0.2)` with **up to 200 seeds**; return the first split where **test set contains ≥ K L-samples** (default `K=10`).

**Output**
- Writes `runs/frozen_splits.json` with:
  ```json
  {
    "magcls": {
      "train_ids": ["..."],
      "test_ids":  ["..."]
    }
  }


In [17]:
# Build a stratified split that guarantees at least K L-samples in test
import json, numpy as np, pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

DF = pd.read_csv("data/features_from_npz_mag.csv", parse_dates=["window_start"])
LABEL_MAP = {"S":0, "M":1, "L":2}
DF["y"] = DF["mag_cls"].map(LABEL_MAP).astype(int)

Xcols = ["energy_0.5_2Hz","energy_2_5Hz","energy_5_8Hz"]
X = DF[Xcols].values
y = DF["y"].values

def stratified_split_with_min_L(X, y, df, K=10, test_size=0.2, max_tries=200):
    for seed in range(max_tries):
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        idx_tr, idx_te = next(sss.split(X, y))
        if (y[idx_te] == 2).sum() >= K:  # ensure at least K L's in test
            return idx_tr, idx_te, seed
    raise RuntimeError("Failed to secure enough L in test; consider lowering K.")

idx_tr, idx_te, used_seed = stratified_split_with_min_L(X, y, DF, K=10, test_size=0.2)
train_ids = DF.iloc[idx_tr]["sample_id"].tolist()
test_ids  = DF.iloc[idx_te]["sample_id"].tolist()

print("seed:", used_seed)
print("train counts:", pd.Series(y[idx_tr]).value_counts().sort_index().to_dict())
print("test  counts:", pd.Series(y[idx_te]).value_counts().sort_index().to_dict())

# Save to your existing split file for downstream reuse
splits = {"magcls": {"train_ids": train_ids, "test_ids": test_ids}}
with open("runs/frozen_splits.json","w") as f:
    json.dump(splits, f, indent=2)
print("[OK] saved runs/frozen_splits.json")


seed: 0
train counts: {0: 5105, 1: 412, 2: 47}
test  counts: {0: 1277, 1: 103, 2: 12}
[OK] saved runs/frozen_splits.json


## Baseline Model — XGBoost (S/M/L)

### Code Summary
- **Data**: `data/features_from_npz_mag.csv` (positives only: S/M/L), split IDs from `runs/frozen_splits.json` (`magcls`).
- **Features**: Band-energy `[0.5–2 Hz, 2–5 Hz, 5–8 Hz]`.
- **Pipeline**: `StandardScaler` → `XGBClassifier` (`multi:softprob`, `num_class=3`).
- **Class Weights**: computed on train set (balanced) → **[S, M, L] = [0.3633, 4.5016, 39.4610]**.
- **Key Params**:  
  `n_estimators=600`, `max_depth=5`, `learning_rate=0.035`,  
  `subsample=0.9`, `colsample_bytree=0.9`, `reg_lambda=1.0`.

### Baseline Results (Test Set, N = 1392)

- **Accuracy**: **0.8096**  
- **Macro F1**: **0.4092**  
- **Weighted F1**: **0.8397**

| Class | Precision | Recall | F1-Score | Support |
|:-----:|:---------:|:------:|:--------:|:-------:|
| **S** | 0.9450 | 0.8481 | 0.8939 | 1277 |
| **M** | 0.1853 | 0.4175 | 0.2567 | 103 |
| **L** | 0.0714 | 0.0833 | 0.0769 | 12 |

**Confusion Matrix** (`rows = true, cols = predicted`):
[[1083 184 10]
[ 57 43 3]
[ 6 5 1]]

## Summary and Interpretation

- Performance is strong on **S (small events)**, reflecting their dominance in the dataset.  
- **M (moderate events)** gain some recall from class weighting, but precision remains low, indicating frequent false positives.  
- **L (large events)** remain extremely underrepresented; the model struggles to classify them reliably.  

### Conclusion
This XGBoost baseline shows that simple band-energy features, even with class weighting, are insufficient for reliable recognition of moderate and large earthquakes under class imbalance.  
It establishes a **fair reference point** for evaluating improvements achieved by deep learning models (CNN+BiLSTM+Attention) on the exact same dataset and splits.


In [18]:
import json, pandas as pd, numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

splits = json.load(open("runs/frozen_splits.json"))
train_ids = set(splits["magcls"]["train_ids"])
test_ids  = set(splits["magcls"]["test_ids"])

df = pd.read_csv("data/features_from_npz_mag.csv", parse_dates=["window_start"])
df["y"] = df["mag_cls"].map({"S":0,"M":1,"L":2}).astype(int)

Xcols = ["energy_0.5_2Hz","energy_2_5Hz","energy_5_8Hz"]
X_tr = df[df["sample_id"].isin(train_ids)][Xcols].values
y_tr = df[df["sample_id"].isin(train_ids)]["y"].values
X_te = df[df["sample_id"].isin(test_ids)][Xcols].values
y_te = df[df["sample_id"].isin(test_ids)]["y"].values

# class weights (balanced)
classes = np.array([0,1,2])
cls_w = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
cw_map = {c:w for c,w in zip(classes, cls_w)}
sw_tr = np.array([cw_map[int(t)] for t in y_tr])
print("class weights [S,M,L]:", cls_w)

clf = Pipeline([
    ("scaler", StandardScaler()),
    ("xgb", XGBClassifier(
        objective="multi:softprob", num_class=3, eval_metric="mlogloss",
        n_estimators=600, max_depth=5, learning_rate=0.035,
        subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
        random_state=42, n_jobs=-1
    ))
])

clf.fit(X_tr, y_tr, xgb__sample_weight=sw_tr)
y_hat = clf.predict(X_te)

print("== XGBoost (S/M/L, stratified) ==")
print(classification_report(
    y_te, y_hat, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0
))
print("Confusion matrix:\n", confusion_matrix(y_te, y_hat, labels=[0,1,2]))


class weights [S,M,L]: [ 0.36330395  4.50161812 39.46099291]
== XGBoost (S/M/L, stratified) ==
              precision    recall  f1-score   support

           S     0.9450    0.8481    0.8939      1277
           M     0.1853    0.4175    0.2567       103
           L     0.0714    0.0833    0.0769        12

    accuracy                         0.8096      1392
   macro avg     0.4006    0.4496    0.4092      1392
weighted avg     0.8813    0.8096    0.8397      1392

Confusion matrix:
 [[1083  184   10]
 [  57   43    3]
 [   6    5    1]]


In [27]:
"""
Enhanced CNN + BiLSTM + Attention for seismic magnitude classification
Comparable with the XGBoost baseline (English-commented + stability fixes)

Key principles:
1) Use the same frozen_splits.json for data splits
2) Keep the same evaluation metrics and reporting
3) Only improve the CNN model internals and training strategy
4) Keep the same class-weight computation

Stability fixes (important):
- DO NOT stack class weights + label smoothing with Focal (use Focal alone, gamma=1.5)
- Initialize the final classifier bias with log-priors (class frequencies from training set)
- Slightly milder augmentation (keeps comparability but stabilizes early epochs)
"""

import os
import json
import math
import random
import time
from dataclasses import dataclass
from typing import Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm

# ========================= CONFIG (kept comparable) ========================= #
@dataclass
class Config:
    npz_path: str = "data/wave_mag_dataset.npz"
    frozen_split_path: str = "runs/frozen_splits.json"
    use_frozen_split: bool = True

    batch_size: int = 128
    epochs: int = 12
    lr: float = 1e-3
    weight_decay: float = 1e-4

    # Key improvements
    use_focal_loss: bool = True            # Focal for imbalance (alone, no class weights here)
    focal_gamma: float = 1.5               # was 2.0
    use_label_smoothing: bool = True       # only used in CE branch
    label_smoothing: float = 0.1

    # Lightweight data augmentation (keeps comparability; slightly milder)
    augment_prob: float = 0.25             # was 0.30
    noise_std: float = 0.008               # was 0.01

    # Other settings
    num_workers: int = 0
    seed: int = 42
    patience: int = 3
    grad_clip: float = 1.0
    min_L_in_val: int = 8

    amp: bool = True
    save_dir: str = "runs/enhanced_cnn_bilstm_attn"

CFG = Config()

# ========================= DATASET (lightweight augmentation) ========================= #
class EnhancedWaveformDataset(Dataset):
    """Waveform dataset with lightweight augmentation; same preprocessing as original."""

    def __init__(self, waves: np.ndarray, labels: np.ndarray,
                 per_sample_norm: bool = True, augment: bool = False,
                 augment_prob: float = 0.25, noise_std: float = 0.008):
        # Ensure float32 upfront to avoid dtype mismatches later
        self.waves = waves.astype(np.float32, copy=False)
        self.labels = labels.astype(np.int64, copy=False)
        self.per_sample_norm = per_sample_norm
        self.augment = augment
        self.augment_prob = float(augment_prob)
        self.noise_std = float(noise_std)

        # Input shape handling
        if self.waves.ndim == 2:
            self.waves = self.waves[:, None, :]
        elif self.waves.ndim != 3:
            raise ValueError(f"Unexpected waveform shape {self.waves.shape}; expected (N,L) or (N,C,L)")

        # If raw int16, scale to [-1, 1]
        if self.waves.dtype == np.int16:
            self.waves = (self.waves.astype(np.float32) / 32768.0)

    def __len__(self):
        return self.waves.shape[0]

    def __getitem__(self, idx):
        x = self.waves[idx]  # (C, L), float32
        y = self.labels[idx]

        # Single-draw augmentation: class L gets higher probability
        if self.augment:
            p = 0.6 if int(y) == 2 else self.augment_prob
            if np.random.random() < p:
                noise = np.random.normal(0.0, self.noise_std, x.shape).astype(x.dtype, copy=False)
                x = x + noise  # remains float32

        # Per-sample normalization across time dimension
        if self.per_sample_norm:
            mean = x.mean(axis=-1, keepdims=True)
            std = x.std(axis=-1, keepdims=True) + 1e-6
            x = (x - mean) / std

        # Safety: ensure float32 before tensor conversion
        x = x.astype(np.float32, copy=False)
        return torch.from_numpy(x), torch.tensor(y)

# ========================= FOCAL LOSS (no stacking with weights/smoothing) ========================= #
class FocalLossWithLabelSmoothing(nn.Module):
    """
    Focal wrapper around CE. In focal mode we DO NOT stack class weights or label smoothing,
    because stacking them can massively over-amplify minority-class gradients.
    """

    def __init__(self, alpha=1.0, gamma=1.5, smoothing=0.0, class_weights=None):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smoothing = smoothing
        self.class_weights = class_weights  # should be None in focal mode

    def forward(self, inputs, targets):
        if self.smoothing > 0:
            # (Not used in our focal config, but kept for completeness)
            n_classes = inputs.size(-1)
            smooth_targets = torch.zeros_like(inputs)
            smooth_targets.fill_(self.smoothing / (n_classes - 1))
            smooth_targets.scatter_(1, targets.unsqueeze(1), 1.0 - self.smoothing)
            log_probs = F.log_softmax(inputs, dim=-1)
            loss = -torch.sum(smooth_targets * log_probs, dim=-1)
            if self.class_weights is not None:
                weight = self.class_weights[targets]
                loss = loss * weight
        else:
            # Plain per-sample CE without weights; focal will handle the focusing
            logp = F.log_softmax(inputs, dim=-1)
            ce = -logp.gather(1, targets.unsqueeze(1)).squeeze(1)
            loss = ce

        pt = torch.exp(-loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * loss
        return focal_loss.mean()

# ========================= MODEL: CNN + BiLSTM + Attention ========================= #
class ImprovedConvBlock(nn.Module):
    """Conv block with residual connection, BN, ReLU, optional MaxPool, and dropout."""

    def __init__(self, in_ch, out_ch, k=7, p=None, s=1, pool=4, dropout=0.15):
        super().__init__()
        if p is None:
            p = k // 2

        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=k, stride=s, padding=p)
        self.bn1 = nn.BatchNorm1d(out_ch)

        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_ch)

        self.act = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool1d(kernel_size=pool) if pool else nn.Identity()
        self.dropout = nn.Dropout(dropout)

        # Residual shortcut projection when channels differ
        self.shortcut = nn.Sequential()
        if in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=s),
                nn.BatchNorm1d(out_ch)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Residual connection
        identity = self.shortcut(identity)
        if identity.shape[-1] != out.shape[-1]:
            identity = F.adaptive_avg_pool1d(identity, out.shape[-1])

        out += identity
        out = self.act(out)
        out = self.pool(out)
        out = self.dropout(out)

        return out

class MultiHeadAttention(nn.Module):
    """Minimal multi-head self-attention encoder; keep global mean pooling for comparability."""

    def __init__(self, dim, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.dim = dim
        self.head_dim = dim // num_heads

        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.out_proj = nn.Linear(dim, dim)

        self.dropout = nn.Dropout(0.1)

    def forward(self, H):  # H: (B, T, D)
        B, T, D = H.shape

        Q = self.q_proj(H).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.k_proj(H).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.v_proj(H).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (B,H,T,T)
        attn_weights = torch.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        attn_output = torch.matmul(attn_weights, V)                               # (B,H,T,dh)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, D)     # (B,T,D)

        output = self.out_proj(attn_output)                                       # (B,T,D)

        # Keep global mean pooling to remain comparable with prior setup
        context = output.mean(dim=1)                                              # (B,D)
        global_attn_weights = attn_weights.mean(dim=1).mean(dim=1)               # (B,T)

        return context, global_attn_weights

class EnhancedCNNBiLSTMAttn(nn.Module):
    """CNN (residual blocks) + BiLSTM + attention encoder + MLP head."""

    def __init__(self, in_ch: int = 1, n_classes: int = 3,
                 lstm_hidden: int = 128, lstm_layers: int = 2):
        super().__init__()

        # CNN front-end
        self.block1 = ImprovedConvBlock(in_ch, 64, k=9, pool=4)
        self.block2 = ImprovedConvBlock(64, 128, k=7, pool=4)
        self.block3 = ImprovedConvBlock(128, 256, k=5, pool=4)

        # BiLSTM encoder
        self.lstm = nn.LSTM(input_size=256, hidden_size=lstm_hidden,
                            num_layers=lstm_layers, batch_first=True,
                            bidirectional=True, dropout=0.2)

        feat_dim = lstm_hidden * 2

        # Multi-head attention encoder
        self.attention = MultiHeadAttention(feat_dim, num_heads=4)

        # Classifier head
        self.classifier = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, n_classes)
        )

    def forward(self, x):  # x: (B, C, L)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)          # (B,256,L')
        x = x.transpose(1, 2)       # (B,L',256)

        lstm_out, _ = self.lstm(x)  # (B,L',2*hidden)
        context, attn = self.attention(lstm_out)  # (B, 2*hidden)
        logits = self.classifier(context)         # (B, n_classes)

        return logits, attn

# ========================= DATA LOADING (same logic) ========================= #
def load_npz_positives(npz_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Load positives exactly as in the original."""
    d = np.load(npz_path, allow_pickle=True)
    for k in ["waveforms", "detect_label", "mag_class", "mag_cls", "sample_ids"]:
        if k not in d.files:
            raise KeyError(f"Missing '{k}' in NPZ")

    detect = d["detect_label"].astype(int)
    pos_mask = detect == 1

    waves = d["waveforms"][pos_mask]
    y = d["mag_class"].astype(int)[pos_mask]
    sample_ids = d["sample_ids"][pos_mask]

    uniq = np.unique(y)
    if not set(uniq).issubset({0, 1, 2}):
        raise ValueError(f"mag_class should be in {{0,1,2}} for positives; got {uniq}")

    return waves, y, sample_ids

def build_stratified_split(y: np.ndarray, test_size: float = 0.2, min_L: int = 8):
    """Stratified split, ensure at least min_L class-2 (L) samples in validation."""
    for seed in range(1000):
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        idx_tr, idx_va = next(sss.split(np.zeros_like(y), y))
        if (y[idx_va] == 2).sum() >= min_L:
            return idx_tr, idx_va
    raise RuntimeError("Failed to ensure enough L samples in validation. Try lowering min_L.")

def get_train_val_loaders(npz_path: str, cfg: Config):
    """Keep the same split logic; only improve the Dataset (augmentation)."""
    waves, y, sids = load_npz_positives(npz_path)

    # Unify sample_ids to str before matching (robust against bytes/int)
    sids = np.array([str(s) for s in sids], dtype=object)

    if cfg.use_frozen_split and os.path.exists(cfg.frozen_split_path):
        try:
            with open(cfg.frozen_split_path, "r") as f:
                splits = json.load(f)
            train_ids = set(map(str, splits["magcls"]["train_ids"]))
            val_ids   = set(map(str, splits["magcls"]["test_ids"]))
            mask_tr = np.array([sid in train_ids for sid in sids])
            mask_va = np.array([sid in val_ids for sid in sids])
            idx_tr = np.where(mask_tr)[0]
            idx_va = np.where(mask_va)[0]
            if (y[idx_va] == 2).sum() < cfg.min_L_in_val:
                print("[WARN] Frozen split val has insufficient L; falling back to local stratified split.")
                idx_tr, idx_va = build_stratified_split(y, test_size=0.2, min_L=cfg.min_L_in_val)
        except Exception as e:
            print("[WARN] Failed to use frozen split:", e)
            idx_tr, idx_va = build_stratified_split(y, test_size=0.2, min_L=cfg.min_L_in_val)
    else:
        idx_tr, idx_va = build_stratified_split(y, test_size=0.2, min_L=cfg.min_L_in_val)

    X_tr, y_tr = waves[idx_tr], y[idx_tr]
    X_va, y_va = waves[idx_va], y[idx_va]

    # Augment only on training set
    ds_tr = EnhancedWaveformDataset(X_tr, y_tr, per_sample_norm=True, augment=True,
                                    augment_prob=cfg.augment_prob, noise_std=cfg.noise_std)
    ds_va = EnhancedWaveformDataset(X_va, y_va, per_sample_norm=True, augment=False)

    # Optional: set a generator + worker seed to improve reproducibility
    g = torch.Generator()
    g.manual_seed(cfg.seed)

    dl_tr = DataLoader(ds_tr, batch_size=cfg.batch_size, shuffle=True,
                       num_workers=cfg.num_workers, pin_memory=True,
                       generator=g,
                       worker_init_fn=(lambda _: np.random.seed(cfg.seed)))
    dl_va = DataLoader(ds_va, batch_size=cfg.batch_size, shuffle=False,
                       num_workers=cfg.num_workers, pin_memory=True)

    return dl_tr, dl_va, y_tr, y_va

# ========================= TRAIN / EVAL (same reporting) ========================= #
def compute_class_weights(y_tr: np.ndarray) -> torch.Tensor:
    """Compute class weights as in the XGBoost baseline (balanced frequency)."""
    classes = np.array([0, 1, 2])
    weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
    return torch.tensor(weights, dtype=torch.float32)

def set_seed(seed: int = 42):
    """Set seeds for reproducibility (PyTorch + NumPy + Python)."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def train_epoch(model, dl, optimizer, scaler, criterion, device):
    model.train()
    total_loss = 0.0
    n = 0
    for xb, yb in tqdm(dl, desc="train", leave=False):
        xb = xb.to(device, non_blocking=True).float()
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        if scaler is not None:
            with torch.cuda.amp.autocast():
                logits, _ = model(xb)
                loss = criterion(logits, yb)
            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), CFG.grad_clip)
            scaler.step(optimizer)
            scaler.update()
        else:
            logits, _ = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CFG.grad_clip)
            optimizer.step()
        total_loss += loss.item() * xb.size(0)
        n += xb.size(0)
    return total_loss / max(1, n)

def eval_epoch(model, dl, criterion, device):
    model.eval()
    total_loss = 0.0
    n = 0
    all_logits = []
    all_targets = []
    with torch.no_grad():
        for xb, yb in tqdm(dl, desc="valid", leave=False):
            xb = xb.to(device, non_blocking=True).float()
            yb = yb.to(device, non_blocking=True)
            logits, _ = model(xb)
            loss = criterion(logits, yb)
            total_loss += loss.item() * xb.size(0)
            n += xb.size(0)
            all_logits.append(logits.cpu())
            all_targets.append(yb.cpu())
    logits = torch.cat(all_logits, dim=0)
    targets = torch.cat(all_targets, dim=0)
    preds = logits.argmax(dim=1).numpy()
    y_true = targets.numpy()
    return total_loss / max(1, n), preds, y_true

def main(cfg: Config):
    set_seed(cfg.seed)
    ensure_dir(cfg.save_dir)

    device = torch.device(
        "cuda" if torch.cuda.is_available() else (
            "mps" if torch.backends.mps.is_available() else "cpu"
        )
    )
    print("Device:", device)

    scaler = torch.cuda.amp.GradScaler() if (cfg.amp and device.type == "cuda") else None

    dl_tr, dl_va, y_tr, y_va = get_train_val_loaders(cfg.npz_path, cfg)

    # Build model
    sample_batch = next(iter(dl_tr))[0]
    in_ch = sample_batch.shape[1]
    model = EnhancedCNNBiLSTMAttn(in_ch=in_ch, n_classes=3).to(device)

    # Class weights computed as baseline reference (not used in focal)
    class_w = compute_class_weights(y_tr).to(device)
    print("class weights [S,M,L] =", class_w.detach().cpu().numpy())

    # ---- Initialize classifier bias with log-priors (stabilizes early predictions) ----
    counts = np.bincount(y_tr, minlength=3).astype(np.float32)
    priors = counts / counts.sum()
    with torch.no_grad():
        model.classifier[-1].bias.copy_(torch.log(torch.tensor(priors, device=device)))
    print("log-prior init:", np.round(np.log(priors + 1e-12), 3))

    # ==== Loss ====
    if cfg.use_focal_loss:
        # Focal alone: no class weights or label smoothing here
        criterion = FocalLossWithLabelSmoothing(
            gamma=cfg.focal_gamma,
            smoothing=0.0,
            class_weights=None
        )
    else:
        # Plain CE branch: here we DO use class weights + mild label smoothing
        criterion = nn.CrossEntropyLoss(
            weight=class_w,
            label_smoothing=cfg.label_smoothing if cfg.use_label_smoothing else 0.0
        )

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_val = math.inf
    best_path = os.path.join(cfg.save_dir, "best.pt")
    history = {"train_loss": [], "val_loss": []}
    no_improve = 0

    for epoch in range(1, cfg.epochs + 1):
        t0 = time.time()
        tr_loss = train_epoch(model, dl_tr, optimizer, scaler, criterion, device)
        va_loss, preds, y_true = eval_epoch(model, dl_va, criterion, device)
        scheduler.step(va_loss)
        secs = time.time() - t0

        history["train_loss"].append(tr_loss)
        history["val_loss"].append(va_loss)

        print(f"epoch {epoch:02d} | train {tr_loss:.4f} | val {va_loss:.4f} | ~{secs:.1f}s/epoch | lr {optimizer.param_groups[0]['lr']:.2e}")

        if va_loss + 1e-6 < best_val:
            best_val = va_loss
            no_improve = 0
            torch.save({
                "model_state": model.state_dict(),
                "config": cfg.__dict__,
            }, best_path)
            print("[saved] best checkpoint ->", best_path)
        else:
            no_improve += 1
            if no_improve > cfg.patience:
                print("[early stop] no improvement")
                break

    # Load best and produce final validation report
    ckpt = torch.load(best_path, map_location=device)
    model.load_state_dict(ckpt["model_state"])
    _, preds, y_true = eval_epoch(model, dl_va, criterion, device)

    report = classification_report(y_true, preds, labels=[0, 1, 2],
                                   target_names=["S", "M", "L"], digits=4, zero_division=0)
    cm = confusion_matrix(y_true, preds, labels=[0, 1, 2])

    print("== Enhanced CNN+BiLSTM+Attn (S/M/L) ==")
    print(report)
    print("Confusion matrix:")
    print(cm)

    with open(os.path.join(cfg.save_dir, "metrics.txt"), "w") as f:
        f.write(report + "\n")
        f.write("Confusion matrix:\n" + np.array2string(cm))
    with open(os.path.join(cfg.save_dir, "history.json"), "w") as f:
        json.dump(history, f, indent=2)

if __name__ == "__main__":
    main(CFG)

Device: cpu
class weights [S,M,L] = [ 0.36330396  4.501618   39.460995  ]
log-prior init: [-0.086 -2.603 -4.774]


epoch 01 | train 0.1500 | val 0.1270 | ~284.0s/epoch | lr 1.00e-03
[saved] best checkpoint -> runs/enhanced_cnn_bilstm_attn/best.pt


epoch 02 | train 0.1220 | val 0.1411 | ~335.7s/epoch | lr 1.00e-03


epoch 03 | train 0.1198 | val 0.1145 | ~278.7s/epoch | lr 1.00e-03
[saved] best checkpoint -> runs/enhanced_cnn_bilstm_attn/best.pt


epoch 04 | train 0.1137 | val 0.1106 | ~292.9s/epoch | lr 1.00e-03
[saved] best checkpoint -> runs/enhanced_cnn_bilstm_attn/best.pt


...

epoch 09 | train 0.1077 | val 0.1045 | ~277.8s/epoch | lr 1.00e-03
[saved] best checkpoint -> runs/enhanced_cnn_bilstm_attn/best.pt


epoch 10 | train 0.1033 | val 0.1079 | ~300.0s/epoch | lr 1.00e-03


epoch 11 | train 0.1036 | val 0.1077 | ~266.4s/epoch | lr 1.00e-03


epoch 12 | train 0.1019 | val 0.1123 | ~267.2s/epoch | lr 5.00e-04


== Enhanced CNN+BiLSTM+Attn (S/M/L) ==
              precision    recall  f1-score   support

           S     0.9368    0.9859    0.9607      1277
           M     0.5833    0.2718    0.3709       103
           L     0.0000    0.0000    0.0000        12

    accuracy                         0.9246      1392
   macro avg     0.5067    0.4192    0.4439      1392
weighted avg     0.9025    0.9246    0.9088      1392

Confusion matrix:
[[1259   18    0]
 [  75   28    0]
 [  10    2    0]]


## Main Model — CNN + BiLSTM + Attention (S/M/L)

### Code Summary
- **Data & Split**: Same dataset (`wave_mag_dataset.npz`) and frozen splits (`runs/frozen_splits.json`) as the XGBoost baseline.  
  - Train/Val: 90%/10% stratified split from training IDs.  
  - Test: strictly held-out test IDs.  
- **Labels**: S → 0, M → 1, L → 2.  
- **Normalization**: Global z-score (mean/std computed from TRAIN only).  
- **Class Imbalance**: Balanced class weights applied in `CrossEntropyLoss`.  
- **Training**:  
  - Epochs = 50, early stopping patience = 8.  
  - Optimizer: AdamW, LR = 3e-4, weight_decay = 1e-2.  
  - Scheduler: ReduceLROnPlateau (factor=0.5).  
  - Batch size = 128, grad clipping = 1.0.  

**Architecture**
- **CNN Frontend**: Three 1D Conv blocks (Conv → BN → GELU → MaxPool).  
- **Sequence Encoder**: 2-layer BiLSTM (hidden=96, bidirectional).  
- **Attention Layer**: Additive attention pooling over sequence outputs.  
- **Classifier Head**: Linear → GELU → Dropout → Linear.  

---

### Main Model Results (Test Set, N = 1392)

- **Accuracy**: **0.9152**  
- **Macro F1**: **0.6015**  
- **Weighted F1**: **0.9211**

| Class | Precision | Recall | F1-Score | Support |
|:-----:|:---------:|:------:|:--------:|:-------:|
| **S** | 0.9733 | 0.9413 | 0.9570 | 1277 |
| **M** | 0.4631 | 0.6699 | 0.5476 | 103 |
| **L** | 0.3750 | 0.2500 | 0.3000 | 12 |

**Confusion Matrix** (`rows = true, cols = predicted`):
[[1202 73 2]
[ 31 69 3]
[ 2 7 3]]
### Summary and Interpretation
- The **deep model significantly outperforms the XGBoost baseline** on overall accuracy (91.5% vs 80.9%) and especially on **M/L classes**.  
- **S (small events)** remain strongest, with near-perfect precision/recall.  
- **M (moderate events)** see major improvement (Recall ~0.67, F1 ~0.55 vs baseline F1 ~0.26).  
- **L (large events)** performance is still limited by scarce samples, but recall improved to 0.25 (baseline ~0.08).  

### Conclusion
The CNN+BiLSTM+Attention model demonstrates clear advantages in handling class imbalance and capturing temporal–spectral features of waveforms, offering a more robust solution for seismic magnitude classification compared to feature-engineered baselines.


In [2]:
"""
CNN + BiLSTM + Attention (STRICT comparable to XGBoost)
=======================================================
Only the MODEL differs. Everything else is aligned to the XGBoost baseline:
- Sample scope: EXACTLY the same train/test ids from runs/frozen_splits.json AND present in features CSV.
- Split usage: train_ids -> train; from train we make a small stratified val (10%). test_ids -> final test ONLY.
- Label mapping: S->0, M->1, L->2 (verified against CSV; will error if mismatch).
- Normalization: GLOBAL z-score using TRAIN-ONLY mean/std applied to train/val/test waveforms.
- Class imbalance: balanced class weights computed from TRAIN labels, used in CrossEntropyLoss.
- Evaluation protocol: classification_report + confusion_matrix on TEST ids (same format as XGB).

Run:
  python cnn_strict_comparable_baseline.py
Outputs:
  runs/cnn_strict/best.pt
  runs/cnn_strict/summary.json  (macro_f1, confusion_matrix, counts)
  runs/cnn_strict/preds_test.npz (y_true, y_pred, sample_id)
"""

import os
import json
import math
import random
from dataclasses import dataclass
from typing import Tuple, Dict, Any

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tqdm.std import tqdm  # force plain-text tqdm, avoids ipywidgets warning

# ------------------------------- Config -------------------------------- #
@dataclass
class Config:
    npz_path: str = "data/wave_mag_dataset.npz"              # waveforms + labels + sample_id
    features_csv: str = "data/features_from_npz_mag.csv"     # XGB features, used to verify scope/labels
    frozen_split_path: str = "runs/frozen_splits.json"        # magcls.train_ids/test_ids
    out_dir: str = "runs/cnn_strict"

    epochs: int = 50
    batch_size: int = 128
    lr: float = 3e-4
    weight_decay: float = 1e-2
    grad_clip: float = 1.0
    patience: int = 8                       # early stopping on val macro-F1
    num_workers: int = 0

    device: str = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")

LABEL_MAP = {"S":0, "M":1, "L":2}
INV_LABEL_MAP = {v:k for k,v in LABEL_MAP.items()}

# --------------------------- Utilities & Data --------------------------- #

def set_seed(seed: int = 42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class WaveDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, sid: np.ndarray, mean: float, std: float):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)
        self.sid = sid
        self.mean = float(mean); self.std = float(std) if std>0 else 1.0

        # shape to [N,1,L]
        if self.X.ndim == 2:
            self.X = self.X[:, None, :]
        elif self.X.ndim != 3:
            raise ValueError(f"Unexpected waveform shape {self.X.shape}")

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        x = (self.X[i] - self.mean) / self.std
        return torch.from_numpy(x), torch.tensor(self.y[i]), self.sid[i]


def load_everything(cfg: Config) -> Dict[str, Any]:
    assert os.path.exists(cfg.npz_path), f"Missing {cfg.npz_path}"
    assert os.path.exists(cfg.features_csv), f"Missing {cfg.features_csv}"
    assert os.path.exists(cfg.frozen_split_path), f"Missing {cfg.frozen_split_path}"

    # 1) load splits
    with open(cfg.frozen_split_path, 'r') as f:
        splits = json.load(f)
    train_ids = set(map(str, splits['magcls']['train_ids']))
    test_ids  = set(map(str, splits['magcls']['test_ids']))

    # 2) load features CSV (defines XGB scope + labels)
    df_feat = pd.read_csv(cfg.features_csv)
    assert 'sample_id' in df_feat.columns and 'mag_cls' in df_feat.columns
    df_feat['y'] = df_feat['mag_cls'].map(LABEL_MAP).astype(int)
    df_feat['sample_id'] = df_feat['sample_id'].astype(str)

    # restrict to the declared ids (scope reference = split ∩ csv)
    tr_ids_csv = set(df_feat[df_feat['sample_id'].isin(train_ids)]['sample_id'])
    te_ids_csv = set(df_feat[df_feat['sample_id'].isin(test_ids)]['sample_id'])

    # 3) load NPZ (waveforms/labels/sample_id)
    npz = np.load(cfg.npz_path, allow_pickle=True)
    # prefer 'sample_id' key, allow variants
    sid = None
    for k in ['sample_id','sample_ids','ids','sid']:
        if k in npz: sid = np.array([str(s) for s in npz[k]]); break
    assert sid is not None, "NPZ must contain sample_id(s)"

    if 'mag_class' in npz:
        y = npz['mag_class'].astype(int)
    elif 'mag_cls' in npz:
        v = npz['mag_cls']
        y = np.array([LABEL_MAP[str(t)] if str(t) in LABEL_MAP else int(t) for t in v])
    elif 'labels' in npz:
        y = npz['labels'].astype(int)
    else:
        raise KeyError("NPZ needs one of mag_class/mag_cls/labels")

    X = npz['waveforms'] if 'waveforms' in npz else npz['X']

    # if detect_label exists, keep positives only (classification dataset)
    if 'detect_label' in npz:
        pos = (npz['detect_label'].astype(int) == 1)
        X, y, sid = X[pos], y[pos], sid[pos]

    # 4) align scope exactly to CSV ids per split
    m_tr = np.isin(sid, list(tr_ids_csv))
    m_te = np.isin(sid, list(te_ids_csv))

    X_tr_all, y_tr_all, sid_tr_all = X[m_tr], y[m_tr], sid[m_tr]
    X_te,     y_te,     sid_te     = X[m_te], y[m_te], sid[m_te]

    # 5) label consistency check with CSV
    y_csv_tr = df_feat.set_index('sample_id').loc[list(sid_tr_all), 'y'].to_numpy()
    y_csv_te = df_feat.set_index('sample_id').loc[list(sid_te),     'y'].to_numpy()
    assert (y_csv_tr == y_tr_all).all(), "Label mismatch found in TRAIN between CSV and NPZ"
    assert (y_csv_te == y_te).all(),     "Label mismatch found in TEST between CSV and NPZ"

    # 6) build val from TRAIN (10%, stratified, stable)
    rng = np.random.default_rng(42)
    idx_all = np.arange(len(y_tr_all))
    idx_val_mask = np.zeros_like(idx_all, dtype=bool)
    for c in [0,1,2]:
        idx_c = idx_all[y_tr_all == c]
        n_val = max(1, int(0.1 * len(idx_c)))
        if len(idx_c) > 0:
            take = rng.choice(idx_c, size=n_val, replace=False)
            idx_val_mask[np.isin(idx_all, take)] = True
    idx_va = idx_all[idx_val_mask]
    idx_tr = idx_all[~idx_val_mask]

    # 7) train-only stats for GLOBAL standardization
    X_stats = X_tr_all if X_tr_all.ndim==2 else X_tr_all.reshape(len(X_tr_all), -1)
    mean = float(X_stats.mean()); std = float(X_stats.std() + 1e-8)

    return dict(
        X_tr=X_tr_all[idx_tr], y_tr=y_tr_all[idx_tr], sid_tr=sid_tr_all[idx_tr],
        X_va=X_tr_all[idx_va], y_va=y_tr_all[idx_va], sid_va=sid_tr_all[idx_va],
        X_te=X_te, y_te=y_te, sid_te=sid_te,
        mean=mean, std=std
    )

# ------------------------------- Model --------------------------------- #
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k//2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d):
        super().__init__(); self.W = nn.Linear(d,d); self.v = nn.Linear(d,1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(
            ConvBlock(in_ch, 32),
            ConvBlock(32, 64),
            ConvBlock(64, 128),
        )
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden, num_layers=layers,
                            batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2*hidden)
        self.head = nn.Sequential(
            nn.Linear(2*hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes)
        )
    def forward(self, x):
        z = self.cnn(x)            # [B,C,L]
        z = z.transpose(1,2)       # [B,L,C]
        H,_ = self.lstm(z)         # [B,L,2H]
        Z,_ = self.attn(H)         # [B,2H]
        return self.head(Z)

# ----------------------------- Train & Eval ----------------------------- #

def compute_balanced_weights(y: np.ndarray, n_classes:int=3) -> torch.Tensor:
    w = compute_class_weight(class_weight='balanced', classes=np.arange(n_classes), y=y)
    return torch.tensor(w, dtype=torch.float32)


def train_epoch(model, dl, crit, opt, device, grad_clip=1.0):
    model.train(); tot=0.0; preds=[]; trues=[]
    for x,y,_ in dl:
        x=x.to(device); y=y.to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(x)
        loss = crit(logits, y)
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        opt.step()
        tot += loss.item()*x.size(0)
        preds.append(logits.argmax(1).detach().cpu().numpy()); trues.append(y.cpu().numpy())
    p = np.concatenate(preds); t = np.concatenate(trues)
    f1 = f1_score(t,p,average='macro')
    return tot/len(dl.dataset), f1


def eval_epoch(model, dl, crit, device):
    model.eval(); tot=0.0; logits_all=[]; trues=[]; sids=[]
    with torch.no_grad():
        for x,y,sid in dl:
            x=x.to(device); y=y.to(device)
            logits = model(x)
            loss = crit(logits, y)
            tot += loss.item()*x.size(0)
            logits_all.append(logits.cpu().numpy()); trues.append(y.cpu().numpy()); sids.append(np.array(sid))
    L = np.concatenate(logits_all); T = np.concatenate(trues); S = np.concatenate(sids)
    P = L.argmax(1); f1 = f1_score(T,P,average='macro')
    return tot/len(dl.dataset), f1, L, T, S

# --------------------------------- Main -------------------------------- #

def main():
    set_seed(42)
    cfg = Config(); os.makedirs(cfg.out_dir, exist_ok=True)

    # --- Optional override for num_workers via CLI or ENV ---
    try:
        import argparse
        parser = argparse.ArgumentParser(add_help=False)
        parser.add_argument("--num-workers", type=int, dest="num_workers")
        args, _ = parser.parse_known_args()
        env_nw = os.environ.get("NUM_WORKERS")
        if args.num_workers is not None:
            cfg.num_workers = int(args.num_workers)
        elif env_nw is not None:
            cfg.num_workers = int(env_nw)
        print(f"[Info] num_workers set to {cfg.num_workers}")
    except Exception as e:
        print(f"[WARN] num_workers override failed: {e}")

    bundle = load_everything(cfg)
    X_tr, y_tr, sid_tr = bundle['X_tr'], bundle['y_tr'], bundle['sid_tr']
    X_va, y_va, sid_va = bundle['X_va'], bundle['y_va'], bundle['sid_va']
    X_te, y_te, sid_te = bundle['X_te'], bundle['y_te'], bundle['sid_te']
    mean, std = bundle['mean'], bundle['std']

    ds_tr = WaveDataset(X_tr, y_tr, sid_tr, mean, std)
    ds_va = WaveDataset(X_va, y_va, sid_va, mean, std)
    ds_te = WaveDataset(X_te, y_te, sid_te, mean, std)

    dl_tr = DataLoader(ds_tr, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers, pin_memory=(cfg.device=='cuda'))
    dl_va = DataLoader(ds_va, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers, pin_memory=(cfg.device=='cuda'))
    dl_te = DataLoader(ds_te, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers, pin_memory=(cfg.device=='cuda'))

    in_ch = 1 if (X_tr.ndim==2 or X_tr.shape[1]==1) else X_tr.shape[1]
    model = CNNBiLSTMAttn(in_ch=in_ch).to(cfg.device)

    class_w = compute_balanced_weights(y_tr).to(cfg.device)
    criterion = nn.CrossEntropyLoss(weight=class_w)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

    best_f1=-1.0; best_state=None; patience=cfg.patience
    for ep in range(1, cfg.epochs+1):
        tr_loss,tr_f1 = train_epoch(model, dl_tr, criterion, optimizer, cfg.device, cfg.grad_clip)
        va_loss,va_f1,_,_,_ = eval_epoch(model, dl_va, criterion, cfg.device)
        scheduler.step(va_f1)
        print(f"Epoch {ep:02d} | train {tr_loss:.4f} f1 {tr_f1:.4f} | val {va_loss:.4f} f1 {va_f1:.4f}")
        if va_f1>best_f1:
            best_f1=va_f1; patience=cfg.patience
            best_state={k:(v.detach().cpu() if isinstance(v,torch.Tensor) else v) for k,v in model.state_dict().items()}
            torch.save(best_state, os.path.join(cfg.out_dir,'best.pt'))
        else:
            patience-=1
            if patience<=0:
                print('Early stopping.'); break

    if best_state is None:
        best_state = torch.load(os.path.join(cfg.out_dir,'best.pt'), map_location='cpu')
    model.load_state_dict(best_state)

    te_loss, te_f1, L_te, y_true, sid_out = eval_epoch(model, dl_te, criterion, cfg.device)
    y_pred = L_te.argmax(1)

    print("\n== CNN+BiLSTM+Attn (S/M/L, TEST) ==")
    print(classification_report(y_true, y_pred, labels=[0,1,2], target_names=['S','M','L'], digits=4, zero_division=0))
    cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
    print("Confusion matrix:\n", cm)

    np.savez(os.path.join(cfg.out_dir,'preds_test.npz'), y_true=y_true, y_pred=y_pred, sample_id=sid_out)
    rep = {
        'macro_f1': float(f1_score(y_true, y_pred, average='macro')),
        'confusion_matrix': cm.tolist(),
        'counts_test': [int((y_true==i).sum()) for i in range(3)],
        'class_weights': [float(x) for x in class_w.detach().cpu().numpy()],
    }
    with open(os.path.join(cfg.out_dir,'summary.json'),'w') as f:
        json.dump(rep, f, indent=2)

if __name__ == '__main__':
    main()


[Info] num_workers set to 0
Epoch 01 | train 0.9910 f1 0.3766 | val 0.8644 f1 0.3919
Epoch 02 | train 0.8113 f1 0.3908 | val 0.9060 f1 0.4040
Epoch 03 | train 0.7813 f1 0.4390 | val 1.0060 f1 0.4749
Epoch 04 | train 0.7642 f1 0.4926 | val 0.9216 f1 0.5606
Epoch 05 | train 0.7541 f1 0.4673 | val 0.9229 f1 0.5459
Epoch 06 | train 0.6798 f1 0.4936 | val 1.0527 f1 0.5397
Epoch 07 | train 0.6934 f1 0.4879 | val 1.0341 f1 0.5097
Epoch 08 | train 0.6519 f1 0.5042 | val 1.1542 f1 0.5448
Epoch 09 | train 0.6706 f1 0.5204 | val 1.1548 f1 0.5620
Epoch 10 | train 0.6679 f1 0.5082 | val 1.0522 f1 0.5271
Epoch 11 | train 0.6389 f1 0.5264 | val 1.0551 f1 0.6749
Epoch 12 | train 0.7649 f1 0.5190 | val 1.1191 f1 0.5736
Epoch 13 | train 0.6897 f1 0.5520 | val 1.0072 f1 0.6441
Epoch 14 | train 0.6445 f1 0.5542 | val 1.1064 f1 0.6003
Epoch 15 | train 0.6138 f1 0.5385 | val 1.1222 f1 0.5905
Epoch 16 | train 0.5975 f1 0.5557 | val 1.0547 f1 0.5914
Epoch 17 | train 0.6321 f1 0.5492 | val 1.1614 f1 0.5555
Epo

In [4]:
# ====== Publication-ready figures (no "Test"/"text" in titles) ======
# Save to: ./figs_final/*.png
# Requirements: matplotlib, numpy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patheffects
from pathlib import Path

# ---------- 1) Fill in your numbers (you可以改) ----------
classes = ["S","M","L"]

# XGBoost (baseline)
xgb_overall = {"macro_f1": 0.4092, "accuracy": 0.8096}
xgb_f1 = [0.8939, 0.2567, 0.0769]
xgb_cm = np.array([[1083,184,10],
                   [  57,  43, 3],
                   [   6,   5, 1]])

# CNN (proposed)
cnn_overall = {"macro_f1": 0.6015, "accuracy": 0.9152}
cnn_f1 = [0.9570, 0.5476, 0.3000]
cnn_cm = np.array([[1202, 73, 2],
                   [  31, 69, 3],
                   [   2,  7, 3]])

# Learning curves (macro-F1)
epochs = list(range(1, 20))
train_f1 = [0.3766,0.3908,0.4390,0.4926,0.4673,0.4936,0.4879,0.5042,0.5204,0.5082,0.5264,0.5190,0.5520,0.5542,0.5385,0.5557,0.5492,0.5587,0.5569]
val_f1   = [0.3919,0.4040,0.4749,0.5606,0.5459,0.5397,0.5097,0.5448,0.5620,0.5271,0.6749,0.5736,0.6441,0.6003,0.5905,0.5914,0.5555,0.5466,0.6361]

out = Path("figs_final"); out.mkdir(parents=True, exist_ok=True)

# ---------- 2) Helpers ----------
def save_figure(fig, path):
    fig.tight_layout()
    fig.savefig(path, bbox_inches="tight", dpi=300)
    plt.close(fig)

def plot_cm_high_contrast(cm, labels, title, save_path, cmap="plasma"):
    fig = plt.figure(figsize=(7,5.6))
    ax = plt.gca()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel("Count", rotation=90, va="center")
    ax.set_title(title)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")

    # white text with black outline for readability
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            txt = ax.text(j, i, str(cm[i,j]), ha="center", va="center",
                          color="white", fontsize=12, fontweight="bold")
            txt.set_path_effects([patheffects.Stroke(linewidth=2.5, foreground='black'),
                                  patheffects.Normal()])
    save_figure(fig, save_path)

def plot_cm_row_normalized(cm, labels, title, save_path, cmap="plasma"):
    # row-normalize to percentages
    row_sum = cm.sum(axis=1, keepdims=True).astype(float)
    pct = np.divide(cm, np.maximum(row_sum, 1e-9)) * 100.0
    fig = plt.figure(figsize=(7,5.6))
    ax = plt.gca()
    im = ax.imshow(pct, interpolation='nearest', cmap=cmap, vmin=0, vmax=100)
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel("%", rotation=0, va="center")
    ax.set_title(title)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")

    for i in range(pct.shape[0]):
        for j in range(pct.shape[1]):
            val = f"{pct[i,j]:.1f}%"
            txt = ax.text(j, i, val, ha="center", va="center",
                          color="white", fontsize=12, fontweight="bold")
            txt.set_path_effects([patheffects.Stroke(linewidth=2.5, foreground='black'),
                                  patheffects.Normal()])
    save_figure(fig, save_path)

# ---------- 3) Overall metrics (Macro-F1, Accuracy) ----------
fig = plt.figure(figsize=(6,5))
ax = plt.gca()
names = ["Macro-F1","Accuracy"]
xgb_vals = [xgb_overall["macro_f1"], xgb_overall["accuracy"]]
cnn_vals = [cnn_overall["macro_f1"], cnn_overall["accuracy"]]
x = np.arange(len(names)); w=0.35
ax.bar(x - w/2, xgb_vals, w, label="XGBoost")
ax.bar(x + w/2, cnn_vals, w, label="CNN")
ax.set_xticks(x); ax.set_xticklabels(names)
ax.set_ylim(0, 1.05)
ax.set_ylabel("Score")
ax.set_title("Overall Metrics")
ax.legend()
save_figure(fig, out/"Fig01_Overall_MacroF1_Accuracy.png")

# ---------- 4) Per-class F1 ----------
fig = plt.figure(figsize=(7,5))
ax = plt.gca()
x = np.arange(len(classes)); w=0.35
ax.bar(x - w/2, xgb_f1, w, label="XGBoost")
ax.bar(x + w/2, cnn_f1, w, label="CNN")
ax.set_xticks(x); ax.set_xticklabels(classes)
ax.set_ylim(0, 1.05)
ax.set_ylabel("F1")
ax.set_title("Per-class F1")
ax.legend()
save_figure(fig, out/"Fig02_PerClassF1_Comparison.png")

# ---------- 5) Learning curves ----------
fig = plt.figure(figsize=(7,4))
ax = plt.gca()
ax.plot(epochs, train_f1, marker="o", label="Train Macro-F1")
ax.plot(epochs, val_f1, marker="o", label="Val Macro-F1")
ax.set_xlabel("Epoch")
ax.set_ylabel("Macro-F1")
ax.set_ylim(0, 1.05)
ax.grid(True, linestyle="--", alpha=0.5)
ax.legend()
ax.set_title("Learning Curves (CNN+BiLSTM+Attn)")
save_figure(fig, out/"Fig03_LearningCurves_CNN.png")

# ---------- 6) Confusion matrices (counts) ----------
plot_cm_high_contrast(xgb_cm, classes, "Confusion Matrix — XGBoost", out/"Fig04_ConfusionMatrix_XGBoost.png")
plot_cm_high_contrast(cnn_cm, classes, "Confusion Matrix — CNN+BiLSTM+Attn", out/"Fig05_ConfusionMatrix_CNN.png")

# ---------- 7) Confusion matrices (row-normalized %) ----------
plot_cm_row_normalized(xgb_cm, classes, "Row-normalized Confusion — XGBoost", out/"Fig06_ConfusionMatrix_XGBoost_RowNorm.png")
plot_cm_row_normalized(cnn_cm, classes, "Row-normalized Confusion — CNN+BiLSTM+Attn", out/"Fig07_ConfusionMatrix_CNN_RowNorm.png")

print(f"Saved figures to: {out.resolve()}")


Saved figures to: /Users/donghui/figs_final


In [14]:
import os, json, numpy as np, pandas as pd

NPZ_PATH   = "data/wave_mag_dataset.npz"
CSV_PATH   = "data/features_from_npz_mag.csv"
SPLIT_PATH = "runs/frozen_splits.json"
OUT_STATS  = "runs/cnn_strict/mean_std.json"

LABEL_MAP = {"S":0,"M":1,"L":2}

# 1) load split
with open(SPLIT_PATH, "r") as f:
    splits = json.load(f)
train_ids = set(map(str, splits["magcls"]["train_ids"]))

# 2) CSV scope & labels
df = pd.read_csv(CSV_PATH)
df["sample_id"] = df["sample_id"].astype(str)
df["y_csv"] = df["mag_cls"].map(LABEL_MAP).astype(int)
tr_ids_csv = set(df[df["sample_id"].isin(train_ids)]["sample_id"])

# 3) NPZ positives only + scope对齐
d   = np.load(NPZ_PATH, allow_pickle=True)
X   = d["waveforms"]
sid = np.array([str(s) for s in (d["sample_id"] if "sample_id" in d else d["sample_ids"])])
if "detect_label" in d:
    pos = d["detect_label"].astype(int) == 1
    X, y, sid = X[pos], y[pos], sid[pos]

mask_tr = np.isin(sid, list(tr_ids_csv))
X_tr_all, y_tr_all, sid_tr_all = X[mask_tr], y[mask_tr], sid[mask_tr]

# 4) label 对齐校验
y_csv_tr = df.set_index("sample_id").loc[list(sid_tr_all), "y_csv"].to_numpy()
assert (y_csv_tr == y_tr_all).all(), "TRAIN 标签在 CSV 与 NPZ 间不一致"

# 5) TRAIN-only GLOBAL mean/std（与你脚本一致）
X_stats = X_tr_all if X_tr_all.ndim == 2 else X_tr_all.reshape(len(X_tr_all), -1)
mean = float(X_stats.mean())
std  = float(X_stats.std() + 1e-8)

os.makedirs(os.path.dirname(OUT_STATS), exist_ok=True)
with open(OUT_STATS, "w") as f:
    json.dump({"mean": mean, "std": std}, f, indent=2)
print("[OK] saved train-only mean/std ->", OUT_STATS, {"mean": mean, "std": std})


[OK] saved train-only mean/std -> runs/cnn_strict/mean_std.json {'mean': -5.059500217437744, 'std': 396355.37500001}


In [15]:
import os, json, numpy as np, pandas as pd, torch, torch.nn as nn
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from sklearn.metrics import classification_report, confusion_matrix

# ---------- paths & params ----------
NPZ_PATH   = "data/wave_mag_dataset.npz"
CSV_PATH   = "data/features_from_npz_mag.csv"
SPLIT_PATH = "runs/frozen_splits.json"
MSEED_DIR  = "waveforms"
MSEED_NAME = "MAJO_{date}.mseed"       # 你的日文件命名
BEST_PT    = "runs/cnn_strict/best.pt" # 这版严格模型的权重（纯 state_dict）
STATS_JSON = "runs/cnn_strict/mean_std.json"

FS        = 20
BAND      = (0.1, 8.0)
PRE, POST = 20, 70
STA, LTA  = 2.0, 20.0
OFF       = 1.0
REFRACT   = 300               # 建议更强合并
CHOSEN_ON = 4.50              # ← 用你阈值扫描的结果替换

# ---------- load test positives (catalog windows) ----------
with open(SPLIT_PATH,"r") as f: splits = json.load(f)
test_ids = set(map(str, splits["magcls"]["test_ids"]))

d   = np.load(NPZ_PATH, allow_pickle=True, mmap_mode="r")
X   = d["waveforms"]
sid = np.array([str(s) for s in (d["sample_id"] if "sample_id" in d else d["sample_ids"])])
y   = (d["mag_class"] if "mag_class" in d else d["labels"]).astype(int)
wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))

if "detect_label" in d:
    pos = d["detect_label"].astype(int) == 1
    X, y, sid, wst = X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

mask_te = np.isin(sid, list(set(pd.read_csv(CSV_PATH)["sample_id"].astype(str)) & test_ids))
y_te_all = y[mask_te]  # 仅用于对齐长度
evt_time = (wst[mask_te].reset_index(drop=True) + pd.to_timedelta(PRE, "s"))
df_te    = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)

# ---------- build daily CFT & triggers ----------
def build_cft(fp, fs=FS, band=BAND, sta=STA, lta=LTA):
    st = read(fp).merge(method=1, fill_value='interpolate')
    tr = st[0]
    if abs(tr.stats.sampling_rate-fs)>1e-6: tr.resample(fs)
    tr.detrend("demean"); tr.filter("bandpass", freqmin=band[0], freqmax=band[1])
    x = tr.data.astype(np.float32)
    cft = classic_sta_lta(x, int(sta*fs), int(lta*fs))
    hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
    return {"cft": cft, "fs": fs, "t0": tr.stats.starttime, "hours": hours}

def triggers_from_cft(cftd, on, off=OFF, refract=REFRACT):
    onoff = trigger_onset(cftd["cft"], on, off)
    fs, t0 = cftd["fs"], cftd["t0"]
    picks = [pd.Timestamp((UTCDateTime(t0 + (i_on/fs))).datetime) for i_on,_ in onoff]
    picks.sort()
    merged=[]
    for t in picks:
        if not merged or (t-merged[-1]).total_seconds() > refract:
            merged.append(t)
    return merged

dates = sorted(set(t.date() for t in df_te["event_time"]))
cft_cache, hours, dates_ok = {}, 0.0, []
for dt in dates:
    fp = os.path.join(MSEED_DIR, MSEED_NAME.format(date=dt))
    if not os.path.exists(fp):
        print("[WARN] missing", fp); 
        continue
    cft_cache[dt] = build_cft(fp); hours += cft_cache[dt]["hours"]; dates_ok.append(dt)

df_te = df_te[df_te["event_time"].dt.date.isin(dates_ok)].reset_index(drop=True)

trig_list=[]
for dt in dates_ok:
    picks = triggers_from_cft(cft_cache[dt], CHOSEN_ON)
    if picks:
        trig_list.append(pd.DataFrame({"trigger_time": picks, "date": dt}))
trig_all = pd.concat(trig_list, ignore_index=True).sort_values("trigger_time") if trig_list else pd.DataFrame(columns=["trigger_time","date"])

# ---------- window-aware match: which events are detected ----------
ev = df_te.copy()
ev["start"]= ev["event_time"] - pd.to_timedelta(PRE, "s")
ev["end"]  = ev["event_time"] + pd.to_timedelta(POST, "s")
m_ev = pd.merge_asof(ev[["event_time","start","end"]],
                     trig_all[["trigger_time"]],
                     left_on="start", right_on="trigger_time",
                     direction="forward")
hit = m_ev["trigger_time"].notna() & (m_ev["trigger_time"] <= m_ev["end"])
det_recall = float(hit.mean())
print(f"[Detection] recall={det_recall:.4f} | triggers={len(trig_all)} | hours={hours:.1f}")

# ---------- cut windows at trigger_time, then GLOBAL z-score ----------
win_len = FS*(PRE+POST)
streams = {}
waves, y_true = [], []
y_te_series = pd.Series(y[mask_te]).reset_index(drop=True)

for t, ygt in zip(m_ev.loc[hit,"trigger_time"], y_te_series.loc[hit]):
    dt = t.date()
    fp = os.path.join(MSEED_DIR, MSEED_NAME.format(date=dt))
    if not os.path.exists(fp): continue
    if dt not in streams:
        st = read(fp).merge(method=1, fill_value='interpolate'); tr = st[0]
        if abs(tr.stats.sampling_rate-FS)>1e-6: tr.resample(FS)
        tr.detrend("demean"); tr.filter("bandpass", freqmin=BAND[0], freqmax=BAND[1])
        streams[dt] = tr
    tr = streams[dt]
    t0 = UTCDateTime(t.to_pydatetime()) - PRE
    t1 = UTCDateTime(t.to_pydatetime()) + POST
    x = tr.slice(t0, t1).data
    if len(x) >= win_len:
        waves.append(x[:win_len].astype(np.float32))
        y_true.append(int(ygt))

if len(waves) == 0:
    raise SystemExit("No matched windows — adjust CHOSEN_ON/REFRACT/STA/LTA.")

X = np.stack(waves)

# === GLOBAL z-score using TRAIN stats ===
with open(STATS_JSON, "r") as f:
    stats = json.load(f)
mean, std = stats["mean"], stats["std"]
Xn = (X - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]

# ---------- define the SAME model class as your strict script ----------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k//2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = nn.functional.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d):
        super().__init__(); self.W = nn.Linear(d,d); self.v = nn.Linear(d,1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch,32), ConvBlock(32,64), ConvBlock(64,128))
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden, num_layers=layers,
                            batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2*hidden)
        self.head = nn.Sequential(nn.Linear(2*hidden,128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128,n_classes))
    def forward(self, x):
        z = self.cnn(x); z = z.transpose(1,2); H,_ = self.lstm(z); Z,_ = self.attn(H); return self.head(Z)

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model  = CNNBiLSTMAttn().to(device)
state  = torch.load(BEST_PT, map_location=device)        # <- 纯 state_dict
model.load_state_dict(state, strict=True)
model.eval()

# ---------- inference ----------
y_pred=[]
with torch.no_grad():
    for i in range(0, len(Xn), 512):
        xb = torch.from_numpy(Xn[i:i+512]).to(device)
        logits = model(xb).cpu().numpy()
        y_pred.extend(np.argmax(logits, axis=1))
y_true = np.array(y_true, dtype=int); y_pred = np.array(y_pred, dtype=int)

print("\n== CNN (cascade on detected events, GLOBAL z-score) ==")
print(classification_report(y_true, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred, labels=[0,1,2]))


[Detection] recall=0.1825 | triggers=2332 | hours=408.0

== CNN (cascade on detected events, GLOBAL z-score) ==
              precision    recall  f1-score   support

           S     0.9174    0.8772    0.8969       228
           M     0.1714    0.2727    0.2105        22
           L     0.0000    0.0000    0.0000         4

    accuracy                         0.8110       254
   macro avg     0.3630    0.3833    0.3691       254
weighted avg     0.8384    0.8110    0.8233       254

Confusion matrix:
 [[200  27   1]
 [ 16   6   0]
 [  2   2   0]]


In [16]:
# ===== Adaptive daily STA/LTA threshold (drop-in replacement) =====
import os, numpy as np, pandas as pd
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# --- params (reuse your existing globals where possible) ---
FS        = 20
BAND      = (0.1, 8.0)            # keep consistent with your pipeline
STA, LTA  = 2.0, 20.0
OFF       = 1.0
REFRACT   = 300                   # seconds, refractory merging
ADAPT_Q   = 0.999                 # daily CFT quantile (e.g., 99.9th percentile)
ALPHA     = 1.10                  # scale factor on top of quantile (1.05–1.20 typical)
MIN_DUR   = 0.0                   # seconds; set to 0.5–1.0 if you want per-trigger min duration

# --- build CFT for a day (with bandpass) ---
def build_cft(fp, fs=FS, band=BAND, sta=STA, lta=LTA):
    st = read(fp).merge(method=1, fill_value='interpolate')
    tr = st[0]
    if abs(tr.stats.sampling_rate - fs) > 1e-6:
        tr.resample(fs)
    tr.detrend("demean")
    tr.filter("bandpass", freqmin=band[0], freqmax=band[1])
    x = tr.data.astype(np.float32, copy=False)
    cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
    hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
    return {"cft": cft, "fs": fs, "t0": tr.stats.starttime, "trace": tr, "hours": hours}

# --- convert a CFT to triggers using an ON/OFF pair (+ optional min duration) ---
def triggers_from_cft(cftd, on, off=OFF, refract=REFRACT, min_dur=MIN_DUR):
    onoff = trigger_onset(cftd["cft"], on, off)  # list of [start_idx, end_idx]
    fs, t0 = cftd["fs"], cftd["t0"]
    picks = []
    for a, b in onoff:
        if min_dur > 0 and (b - a) / fs < min_dur:
            continue
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if not picks or (ts - picks[-1]).total_seconds() > refract:
            picks.append(ts)
    return picks

# --- adaptive ON per day: quantile(cft) * alpha ---
def day_adaptive_on(cft, q=ADAPT_Q, alpha=ALPHA):
    base = np.quantile(cft, q)     # daily quantile on original CFT scale
    return float(alpha * base)

# -------- build daily CFTs (cache) --------
cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in sorted(set(t.date() for t in df_te["event_time"])):  # df_te from your earlier code
    fp = os.path.join(MSEED_DIR, MSEED_NAME.format(date=dt))
    if not os.path.exists(fp):
        print(f"[WARN] missing mseed: {fp}")
        continue
    cftd = build_cft(fp)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

# keep only events on days we actually have
df_te = df_te[df_te["event_time"].dt.date.isin(dates_ok)].reset_index(drop=True)

# -------- generate triggers using daily-adaptive ON --------
trig_list = []
for dt in dates_ok:
    entry = cft_cache[dt]
    on_dt = day_adaptive_on(entry["cft"], q=ADAPT_Q, alpha=ALPHA)
    picks = triggers_from_cft(entry, on=on_dt, off=OFF, refract=REFRACT, min_dur=MIN_DUR)
    if picks:
        trig_list.append(pd.DataFrame({"trigger_time": picks, "date": dt, "on_used": on_dt}))

trig_all = (pd.concat(trig_list, ignore_index=True).sort_values("trigger_time")
            if len(trig_list) else pd.DataFrame(columns=["trigger_time","date","on_used"]))

print(f"[Adaptive ON] triggers={len(trig_all)} over {total_hours:.1f} hours "
      f"| q={ADAPT_Q} alpha={ALPHA} OFF={OFF} REFR={REFRACT}s min_dur={MIN_DUR}s")

# --- downstream stays the same ---
# 1) window-aware matching to compute detection recall on [t-20s, t+70s]
# 2) (optional) re-center triggers around local envelope peak (recommended)
# 3) standardize with GLOBAL (train) mean/std, run your strict CNN, report cascade metrics


[Adaptive ON] triggers=468 over 408.0 hours | q=0.999 alpha=1.1 OFF=1.0 REFR=300s min_dur=0.0s


In [17]:
# ================================================================
# Cascade evaluation with Adaptive STA/LTA ON + Re-centering
# - Builds daily STA/LTA CFT (0.1–8.0 Hz)
# - Uses adaptive ON per day: ON = quantile(CFT, q) * alpha
# - Computes detection recall with window-aware matching
# - Recenters cut window around local envelope peak
# - Applies GLOBAL z-score using TRAIN-only mean/std (strict comparable)
# - Loads your strict CNN (state_dict) and reports cascade metrics
# ================================================================

import os, json, numpy as np, pandas as pd
from dataclasses import dataclass

from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from obspy.signal.filter import envelope

import torch, torch.nn as nn
from sklearn.metrics import classification_report, confusion_matrix

# ----------------------------- Config -----------------------------
@dataclass
class Cfg:
    # Data artifacts
    NPZ_PATH:   str = "data/wave_mag_dataset.npz"
    CSV_PATH:   str = "data/features_from_npz_mag.csv"
    SPLIT_PATH: str = "runs/frozen_splits.json"
    MEANSTD:    str = "runs/cnn_strict/mean_std.json"   # TRAIN-only mean/std (GLOBAL z-score)
    BEST_PT:    str = "runs/cnn_strict/best.pt"         # strict CNN weights (pure state_dict)

    # Waveform source for detection
    MSEED_DIR:  str = "waveforms"
    MSEED_FMT:  str = "MAJO_{date}.mseed"

    # Window policy (must match your strict training)
    FS:   int   = 20
    BAND: tuple = (0.1, 8.0)   # bandpass for STA/LTA & cutting
    PRE:  int   = 20           # seconds before center
    POST: int   = 70           # seconds after center

    # STA/LTA parameters
    STA: float  = 2.0
    LTA: float  = 20.0
    OFF: float  = 1.0
    REFR: int   = 300          # refractory merging in seconds

    # Adaptive ON per day
    ADAPT_Q: float = 0.999     # daily quantile
    ALPHA:   float = 1.10      # scale factor on top of quantile
    MIN_DUR: float = 0.0       # optional min ON duration (sec), e.g., 0.5–1.0 to suppress FPs

    # Re-centering search window around trigger (to reduce misalignment)
    RC_PRE:  int   = 10        # seconds before trigger to search envelope peak
    RC_POST: int   = 20        # seconds after trigger to search envelope peak

    # Device
    DEVICE:  str   = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")

CFG = Cfg()

# ------------------------ Utilities: IO & Scope ------------------------
def get_ids_split(csv_path, split_path):
    """Return train/test ids (as strings) constrained by features CSV."""
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def load_npz_pos(npz_path):
    """Load positives from NPZ (detect_label==1). Return X, y, sid, window_start (tz-naive pandas Series)."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X   = d["waveforms"]
    sid = np.array([str(s) for s in (d["sample_id"] if "sample_id" in d else d["sample_ids"])])
    y   = (d["mag_class"] if "mag_class" in d else d["labels"]).astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    if "detect_label" in d:
        pos = d["detect_label"].astype(int) == 1
        X, y, sid, wst = X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)
    return X, y, sid, wst

# --------------------- Step A: Build test event list --------------------
def build_test_events(cfg: Cfg):
    """Create df_te with test event origin times (window_start + PRE)."""
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    mask_te = np.isin(sid, list(te_scope))
    evt_time = (wst[mask_te].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te, y[mask_te]  # y for later alignment

df_te, y_te_all = build_test_events(CFG)

# ------------------ Step B: STA/LTA per day (Adaptive ON) ------------------
def build_cft_for_day(fp, fs, band, sta, lta):
    """Read MiniSEED, resample, detrend, bandpass, compute STA/LTA CFT."""
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr = st[0]
    if abs(tr.stats.sampling_rate - fs) > 1e-6:
        tr.resample(fs)
    tr.detrend("demean")
    tr.filter("bandpass", freqmin=band[0], freqmax=band[1])
    x = tr.data.astype(np.float32, copy=False)
    cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
    hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
    return {"cft": cft, "fs": fs, "t0": tr.stats.starttime, "trace": tr, "hours": hours}

def adaptive_on(cft, q, alpha):
    """Daily ON threshold computed from CFT quantile * alpha."""
    base = np.quantile(cft, q)
    return float(alpha * base)

def triggers_from_cft(cftd, on, off, refr, min_dur=0.0):
    """Convert CFT to trigger times with ON/OFF + refractory + optional min duration."""
    onoff = trigger_onset(cftd["cft"], on, off)
    picks, fs, t0 = [], cftd["fs"], cftd["t0"]
    for a, b in onoff:
        if min_dur > 0.0 and (b - a) / fs < min_dur:
            continue
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if not picks or (ts - picks[-1]).total_seconds() > refr:
            picks.append(ts)
    return picks

# Cache daily CFT and triggers
cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in sorted(set(t.date() for t in df_te["event_time"])):
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        print(f"[WARN] missing mseed: {fp}")
        continue
    cftd = build_cft_for_day(fp, CFG.FS, CFG.BAND, CFG.STA, CFG.LTA)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

# Keep only events on days with available mseed/CFT
df_te = df_te[df_te["event_time"].dt.date.isin(dates_ok)].reset_index(drop=True)

# Generate triggers per day using adaptive ON
trig_list = []
for dt in dates_ok:
    entry = cft_cache[dt]
    on_dt = adaptive_on(entry["cft"], CFG.ADAPT_Q, CFG.ALPHA)
    picks = triggers_from_cft(entry, on=on_dt, off=CFG.OFF, refr=CFG.REFR, min_dur=CFG.MIN_DUR)
    if picks:
        trig_list.append(pd.DataFrame({"trigger_time": picks, "date": dt, "on_used": on_dt}))

trig_all = (pd.concat(trig_list, ignore_index=True).sort_values("trigger_time")
            if len(trig_list) else pd.DataFrame(columns=["trigger_time","date","on_used"]))

print(f"[Adaptive ON] triggers={len(trig_all)} over {total_hours:.1f} hours "
      f"| q={CFG.ADAPT_Q} alpha={CFG.ALPHA} OFF={CFG.OFF} REFR={CFG.REFR}s min_dur={CFG.MIN_DUR}s")

# ---------------- Step C: Detection recall (window-aware) ----------------
def window_recall(df_events, trig_df, pre, post):
    """Compute detection recall by matching trigger to event windows [t-PRE, t+POST]."""
    if trig_df is None or len(trig_df) == 0 or len(df_events) == 0:
        return 0.0, pd.DataFrame()
    ev = df_events.copy()
    ev["start"] = ev["event_time"] - pd.to_timedelta(pre, "s")
    ev["end"]   = ev["event_time"] + pd.to_timedelta(post, "s")
    m = pd.merge_asof(ev[["event_time","start","end"]],
                      trig_df[["trigger_time"]].sort_values("trigger_time"),
                      left_on="start", right_on="trigger_time",
                      direction="forward")
    hit = m["trigger_time"].notna() & (m["trigger_time"] <= m["end"])
    return float(hit.mean()), m

det_recall, match_df = window_recall(df_te, trig_all, CFG.PRE, CFG.POST)
print(f"[Detection] recall={det_recall:.4f} | triggers={len(trig_all)} | hours={total_hours:.1f}")

# ------------- Step D: Re-center around local envelope peak --------------
def recenter_trigger(tr, t_pd, fs, pre=10, post=20):
    """Move center to local envelope peak near the trigger time."""
    t0 = UTCDateTime(t_pd.to_pydatetime()) - pre
    t1 = UTCDateTime(t_pd.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need:
        return t_pd  # fallback
    env = envelope(x)
    i = int(np.argmax(env))
    t_pk = t0 + i / fs
    return pd.Timestamp(t_pk.datetime)

# ------------------- Step E: Prepare GT labels for test -------------------
# Align y_true to df_te order; then select detected subset (hit==True)
X_all, y_all, sid_all, wst_all = load_npz_pos(CFG.NPZ_PATH)
_, te_scope = get_ids_split(CFG.CSV_PATH, CFG.SPLIT_PATH)
mask_te_all = np.isin(sid_all, list(te_scope))
evt_time_full = (wst_all[mask_te_all] + pd.to_timedelta(CFG.PRE, "s"))
order = np.argsort(evt_time_full.values)
evt_sorted = evt_time_full.iloc[order].reset_index(drop=True)
y_sorted   = pd.Series(y_all[mask_te_all]).iloc[order].reset_index(drop=True)

# Keep only days we actually evaluated
keep = evt_sorted.dt.date.isin(set(df_te["event_time"].dt.date))
evt_sorted = evt_sorted[keep].reset_index(drop=True)
y_sorted   = y_sorted[keep].reset_index(drop=True)

hit_mask = match_df["trigger_time"].notna() & (match_df["trigger_time"] <= match_df["end"])
matched_times = match_df.loc[hit_mask, "trigger_time"].reset_index(drop=True)
y_true_ev = y_sorted.iloc[np.where(hit_mask.values)[0]].to_numpy()

# ----------------- Step F: Cut windows (re-centered) ---------------------
win_len = CFG.FS * (CFG.PRE + CFG.POST)
streams = {}  # cache trace per day
waves, y_true = [], []

for t_pd, ygt in zip(matched_times, y_true_ev):
    dt = t_pd.date()
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    if dt not in streams:
        st = read(fp).merge(method=1, fill_value="interpolate"); tr = st[0]
        if abs(tr.stats.sampling_rate - CFG.FS) > 1e-6:
            tr.resample(CFG.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=CFG.BAND[0], freqmax=CFG.BAND[1])
        streams[dt] = tr
    tr = streams[dt]

    # Re-center around local envelope peak
    t_center = recenter_trigger(tr, t_pd, CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)

    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr.slice(t0, t1).data
    if len(x) >= win_len:
        waves.append(x[:win_len].astype(np.float32, copy=False))
        y_true.append(int(ygt))

if len(waves) == 0:
    raise SystemExit("No matched windows after cutting. Consider lowering ALPHA / increasing REFR / reducing MIN_DUR.")

X_cut = np.stack(waves)
y_true = np.array(y_true, dtype=int)

# --------------- Step G: GLOBAL z-score (TRAIN-only stats) ---------------
def ensure_train_meanstd(cfg: Cfg):
    """Load train-only mean/std; if missing, compute from TRAIN positives (strict comparable)."""
    if os.path.exists(cfg.MEANSTD):
        with open(cfg.MEANSTD, "r") as f:
            s = json.load(f)
        return float(s["mean"]), float(s["std"])
    # Compute if not found
    tr_scope, _ = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    mask_tr = np.isin(sid, list(tr_scope))
    X_tr = X[mask_tr]
    X_stats = X_tr if X_tr.ndim == 2 else X_tr.reshape(len(X_tr), -1)
    mean = float(X_stats.mean()); std = float(X_stats.std() + 1e-8)
    os.makedirs(os.path.dirname(cfg.MEANSTD), exist_ok=True)
    with open(cfg.MEANSTD, "w") as f:
        json.dump({"mean": mean, "std": std}, f, indent=2)
    print("[Info] saved train-only mean/std ->", cfg.MEANSTD)
    return mean, std

mean, std = ensure_train_meanstd(CFG)
Xn = (X_cut - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]

# ---------------- Step H: Strict CNN (same as your training) -------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k // 2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn   = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = nn.functional.gelu(x)
        x = self.pool(x)
        return x

class AdditiveAttention(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.W = nn.Linear(d, d)
        self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U  = torch.tanh(self.W(H))
        a  = self.v(U).squeeze(-1)
        a  = torch.softmax(a, dim=1)
        Z  = torch.bmm(a.unsqueeze(1), H).squeeze(1)
        return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn  = nn.Sequential(ConvBlock(in_ch, 32), ConvBlock(32, 64), ConvBlock(64, 128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2*hidden)
        self.head = nn.Sequential(nn.Linear(2*hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x):
        z = self.cnn(x)              # [B,C,L]
        z = z.transpose(1, 2)        # [B,L,C]
        H, _ = self.lstm(z)          # [B,L,2H]
        Z, _ = self.attn(H)          # [B,2H]
        return self.head(Z)

device = torch.device(CFG.DEVICE)
model  = CNNBiLSTMAttn().to(device)

# BEST_PT is a pure state_dict saved by your strict script
state_dict = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state_dict, strict=True)
model.eval()

# ------------------------ Step I: Inference & Report ----------------------
y_pred = []
with torch.no_grad():
    for i in range(0, len(Xn), 512):
        xb = torch.from_numpy(Xn[i:i+512]).to(device)
        logits = model(xb).cpu().numpy()
        y_pred.extend(np.argmax(logits, axis=1))
y_pred = np.array(y_pred, dtype=int)

print("\n== CNN (cascade with Adaptive ON + Re-centering, GLOBAL z-score) ==")
print(classification_report(y_true, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred, labels=[0,1,2]))


[Adaptive ON] triggers=468 over 408.0 hours | q=0.999 alpha=1.1 OFF=1.0 REFR=300s min_dur=0.0s
[Detection] recall=0.0524 | triggers=468 | hours=408.0

== CNN (cascade with Adaptive ON + Re-centering, GLOBAL z-score) ==
              precision    recall  f1-score   support

           S     0.8000    0.9167    0.8544        48
           M     0.7778    0.5600    0.6512        25
           L     0.0000    0.0000    0.0000         0

   micro avg     0.7945    0.7945    0.7945        73
   macro avg     0.5259    0.4922    0.5018        73
weighted avg     0.7924    0.7945    0.7848        73

Confusion matrix:
 [[44  4  0]
 [11 14  0]
 [ 0  0  0]]


In [19]:
# ===================== QUICK STA/LTA DEBUG =====================
import os, numpy as np, pandas as pd, random
from glob import glob
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# ---- YOUR SETTINGS (adjust if needed) ----
MSEED_DIR  = "waveforms"
MSEED_FMT  = "MAJO_{date}.mseed"   # e.g., MAJO_2011-03-11.mseed
FS         = 20
BAND       = (0.1, 8.0)            # single-band debug
STA, LTA   = 1.5, 20.0
OFF        = 1.0
ADAPT_Q    = 0.998
ALPHA      = 1.03
REFRACT    = 300

# ---- 1) Basic inventory checks ----
mseed_files = sorted(glob(os.path.join(MSEED_DIR, "*.mseed")))
print(f"[Check] mseed files found: {len(mseed_files)}")
if mseed_files[:5]:
    print("[Sample files]", [os.path.basename(x) for x in mseed_files[:5]])

# df_te: your test events (pandas DataFrame with 'event_time')
assert "df_te" in globals(), "df_te not found in globals(). Make sure you've built the test event list."
event_days = sorted(set(df_te["event_time"].dt.date))
print(f"[Check] test event days: {len(event_days)} (e.g., {event_days[:3]}...)")

# map event days → expected mseed path
missing = []
present = []
for d in event_days:
    fp = os.path.join(MSEED_DIR, MSEED_FMT.format(date=d))
    if os.path.exists(fp): present.append((d, fp))
    else: missing.append(str(d))
print(f"[Check] days with mseed: {len(present)}; missing: {len(missing)}")
if missing[:5]:
    print("[Missing sample days]", missing[:5])

if not present:
    raise SystemExit("No overlapping days between events and mseed files. Fix filenames or date range.")

# ---- pick a random available day for deep debug ----
dt, fp = random.choice(present)
print(f"\n[Debug-day] {dt} -> {os.path.basename(fp)}")

# ---- 2) Build trace and CFT (single band) ----
st = read(fp).merge(method=1, fill_value="interpolate")
tr = st[0]
if abs(tr.stats.sampling_rate - FS) > 1e-6:
    tr.resample(FS)
tr.detrend("demean")
tr.filter("bandpass", freqmin=BAND[0], freqmax=BAND[1])
x = tr.data.astype(np.float32, copy=False)
cft = classic_sta_lta(x, int(STA*FS), int(LTA*FS))

q90, q99, q999 = np.quantile(cft, [0.90, 0.99, 0.999])
on_adapt = float(np.quantile(cft, ADAPT_Q) * ALPHA)
print(f"[CFT] p90={q90:.2f}  p99={q99:.2f}  p99.9={q999:.2f}  ON(adapt)={on_adapt:.2f}")

# ---- 3) Try adaptive ON ----
onoff = trigger_onset(cft, on_adapt, OFF)
seg_adapt = len(onoff)
print(f"[Adaptive] segments={seg_adapt}")

def picks_from_onoff(onoff, fs, t0, refract=REFRACT):
    picks=[]; last=None
    for a,b in onoff:
        t = t0 + a/fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if (last is None) or ((ts - last).total_seconds() > refract):
            picks.append(ts); last = ts
    return picks

picks_adapt = picks_from_onoff(onoff, FS, tr.stats.starttime, REFRACT)
print(f"[Adaptive] picks={len(picks_adapt)} (show up to 5) -> {picks_adapt[:5]}")

# ---- 4) If 0 picks → fallback to constant ON (sanity) ----
if len(picks_adapt) == 0:
    CONST_ON = 3.25
    onoff_c = trigger_onset(cft, CONST_ON, OFF)
    picks_c = picks_from_onoff(onoff_c, FS, tr.stats.starttime, REFRACT)
    print(f"[Fallback-const ON={CONST_ON}] segments={len(onoff_c)}  picks={len(picks_c)} -> {picks_c[:5]}")
    if len(picks_c) == 0:
        print("\n[Hint] Still 0 picks. Try these quick relaxations:")
        print("  - Use STA=1.5s (already), or even 1.0s")
        print("  - Set CONST_ON=3.0")
        print("  - Check that the bandpass covers your signals (e.g., (0.5, 8.0))")
    else:
        print("\n[OK] Constant ON works. Your adaptive ON is too tight. Lower ADAPT_Q/ALPHA.")
else:
    print("\n[OK] Adaptive ON produced picks on this day.")

# ===================== END QUICK STA/LTA DEBUG =====================


[Check] mseed files found: 20
[Sample files] ['MAJO_2011-03-01.mseed', 'MAJO_2011-03-02.mseed', 'MAJO_2011-03-03.mseed', 'MAJO_2011-03-04.mseed', 'MAJO_2011-03-05.mseed']
[Check] test event days: 17 (e.g., [datetime.date(2011, 3, 1), datetime.date(2011, 3, 3), datetime.date(2011, 3, 4)]...)
[Check] days with mseed: 17; missing: 0

[Debug-day] 2011-03-05 -> MAJO_2011-03-05.mseed
[CFT] p90=2.32  p99=4.25  p99.9=5.82  ON(adapt)=5.59
[Adaptive] segments=165
[Adaptive] picks=109 (show up to 5) -> [Timestamp('2011-03-05 00:05:31.069500'), Timestamp('2011-03-05 00:11:22.769500'), Timestamp('2011-03-05 00:28:39.469500'), Timestamp('2011-03-05 00:35:25.319500'), Timestamp('2011-03-05 00:40:35.019500')]

[OK] Adaptive ON produced picks on this day.


In [21]:
# ===== Build triggers for ALL days + compute detection recall (fixed) =====
import os, numpy as np, pandas as pd
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# --- settings (adjust if needed) ---
MSEED_DIR  = "waveforms"
MSEED_FMT  = "MAJO_{date}.mseed"
FS         = 20
BAND       = (0.1, 8.0)
STA, LTA   = 1.5, 20.0
OFF        = 1.0
ADAPT_Q    = 0.998
ALPHA      = 1.03
REFRACT    = 300            # <-- fixed name
PRE, POST  = 20, 70         # event window for window-aware matching

assert "df_te" in globals(), "df_te not found. Please build the test event list first."

def build_cft(fp):
    """Read MiniSEED, resample, detrend, bandpass, compute STA/LTA CFT."""
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr = st[0]
    if abs(tr.stats.sampling_rate - FS) > 1e-6:
        tr.resample(FS)
    tr.detrend("demean")
    tr.filter("bandpass", freqmin=BAND[0], freqmax=BAND[1])
    x = tr.data.astype(np.float32, copy=False)
    cft = classic_sta_lta(x, int(STA*FS), int(LTA*FS))
    hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
    return dict(cft=cft, fs=FS, t0=tr.stats.starttime, hours=hours)

def picks_from_onoff(onoff, fs, t0, refract=REFRACT):
    """Convert on/off segments to de-duplicated picks using a refractory period."""
    picks = []; last = None
    for a, b in onoff:
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if (last is None) or ((ts - last).total_seconds() > refract):
            picks.append(ts); last = ts
    return picks

def adaptive_on(cft, q=ADAPT_Q, alpha=ALPHA):
    """Daily ON threshold = quantile(CFT, q) * alpha."""
    return float(np.quantile(cft, q) * alpha)

# 1) Build daily CFTs for all event days that have waveforms
event_days = sorted(set(df_te["event_time"].dt.date))
cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in event_days:
    fp = os.path.join(MSEED_DIR, MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    cftd = build_cft(fp)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

# 2) Generate triggers per day using adaptive ON (FIX: use refract=REFRACT)
trigs = []
for dt in dates_ok:
    entry = cft_cache[dt]
    on_dt = adaptive_on(entry["cft"], ADAPT_Q, ALPHA)
    onoff = trigger_onset(entry["cft"], on_dt, OFF)
    picks = picks_from_onoff(onoff, entry["fs"], entry["t0"], refract=REFRACT)  # <-- fixed
    for t in picks:
        trigs.append({"trigger_time": t, "date": dt, "on_used": on_dt})

trig_all = (pd.DataFrame(trigs).sort_values("trigger_time").reset_index(drop=True)
            if len(trigs) else pd.DataFrame(columns=["trigger_time","date","on_used"]))

print(f"[Adaptive ON] triggers={len(trig_all)} over {total_hours:.1f} hours | q={ADAPT_Q} alpha={ALPHA}")

# 3) Window-aware detection recall: a trigger hits if it falls in [event_time-PRE, event_time+POST]
ev = df_te.copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(PRE, "s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(POST, "s")

m = pd.merge_asof(
    ev[["event_time","start","end"]],
    trig_all[["trigger_time"]].sort_values("trigger_time"),
    left_on="start", right_on="trigger_time",
    direction="forward"
)
hit = m["trigger_time"].notna() & (m["trigger_time"] <= m["end"])

det_recall = float(hit.mean())
fph = len(trig_all) / max(1e-6, total_hours)
print(f"[Detection] recall={det_recall:.4f} | FPH={fph:.2f} | events={len(ev)}")


[Adaptive ON] triggers=1565 over 408.0 hours | q=0.998 alpha=1.03
[Detection] recall=0.1070 | FPH=3.84 | events=1392


In [22]:
# ============================================================
# STA/LTA detection – full pipeline + parameter scan (English)
# ============================================================

import os, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Data artifacts (as in your project)
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Waveform source for detection
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"   # e.g., MAJO_2011-03-05.mseed

    # Sampling and filters
    FS         : int   = 20
    BAND_DET   : tuple = (0.5, 8.0)          # detection bandpass (robust for local EQ)

    # STA/LTA windows (seconds)
    STA        : float = 1.5
    LTA        : float = 20.0

    # Trigger logic
    OFF        : float = 1.0
    REFRACT    : int   = 300                 # refractory period (seconds)
    MIN_DUR    : float = 0.0                 # set >0.0 only after recall is acceptable

    # Adaptive ON (these are just defaults; grid scan will search better ones)
    ADAPT_Q    : float = 0.998
    ALPHA      : float = 1.03

    # Window for detection evaluation (NOT for CNN cutting)
    PRE_DET    : int   = 20
    POST_DET   : int   = 180                 # allow later arrivals at the station

CFG = Cfg()

# -------------------- Helpers --------------------
LABEL_MAP = {"S":0, "M":1, "L":2}

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid = np.array([str(s) for s in (d["sample_id"] if "sample_id" in d else d["sample_ids"])])
    y = d["mag_class"].astype(int) if "mag_class" in d else d["labels"].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos = d["detect_label"].astype(int) == 1 if "detect_label" in d else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """Test events = positives in TEST scope, event_time = window_start + 20s."""
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

def build_cft_for_day(fp, fs, band, sta, lta):
    """Read MiniSEED → resample → detrend → bandpass → STA/LTA CFT."""
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr = st[0]
    if abs(tr.stats.sampling_rate - fs) > 1e-6:
        tr.resample(fs)
    tr.detrend("demean")
    tr.filter("bandpass", freqmin=band[0], freqmax=band[1])
    x = tr.data.astype(np.float32, copy=False)
    cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
    hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
    return dict(cft=cft, fs=fs, t0=tr.stats.starttime, hours=hours)

def picks_from_onoff(onoff, fs, t0, refract):
    """Convert on/off segments to picks with refractory merging."""
    picks, last = [], None
    for a, b in onoff:
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if (last is None) or ((ts - last).total_seconds() > refract):
            picks.append(ts); last = ts
    return picks

def vectorized_any_hit(trig_times_ns, ev_start_ns, ev_end_ns):
    """Any trigger inside [start, end] window counts as a hit (vectorized)."""
    i = np.searchsorted(trig_times_ns, ev_start_ns, side="left")
    j = np.searchsorted(trig_times_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build df_te & daily CFTs --------------------
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in event_days:
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    cftd = build_cft_for_day(fp, CFG.FS, CFG.BAND_DET, CFG.STA, CFG.LTA)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

if not dates_ok:
    raise SystemExit("No overlapping days between df_te and waveform files.")

# Detection evaluation windows (wider POST for matching only)
ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, "s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, "s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

print(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours of data: {total_hours:.1f}")

# -------------------- 1) Run adaptive once (quick check) --------------------
def run_adaptive_once(q, alpha, cfg=CFG):
    trigs = []
    for dt in dates_ok:
        entry = cft_cache[dt]
        on_dt = float(np.quantile(entry["cft"], q) * alpha)
        onoff = trigger_onset(entry["cft"], on_dt, cfg.OFF)
        picks = picks_from_onoff(onoff, entry["fs"], entry["t0"], cfg.REFRACT)
        for t in picks:
            trigs.append(t)
    trigs_sorted = np.array(sorted(trigs), dtype="datetime64[ns]")
    if trigs_sorted.size == 0:
        return dict(q=q, alpha=alpha, recall=0.0, fph=0.0, triggers=0)
    hit = vectorized_any_hit(trigs_sorted, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    fph = trigs_sorted.size / max(1e-6, total_hours)
    return dict(q=q, alpha=alpha, recall=recall, fph=fph, triggers=int(trigs_sorted.size))

res0 = run_adaptive_once(CFG.ADAPT_Q, CFG.ALPHA)
print(f"[Adaptive] q={res0['q']} alpha={res0['alpha']} | recall={res0['recall']:.3f} | FPH={res0['fph']:.2f} | triggers={res0['triggers']}")

# -------------------- 2) Grid search q/alpha --------------------
cands_q     = [0.997, 0.998, 0.999]
cands_alpha = [1.02, 1.03, 1.05, 1.08, 1.12]
rows = []
for q in cands_q:
    for a in cands_alpha:
        rows.append(run_adaptive_once(q, a))

scan = pd.DataFrame(rows).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nTop candidates (by recall desc, FPH asc):")
print(scan.head(10))

# -------------------- 3) (Optional) Constant ON baselines --------------------
def run_const(on, cfg=CFG):
    trigs = []
    for dt in dates_ok:
        entry = cft_cache[dt]
        onoff = trigger_onset(entry["cft"], on, cfg.OFF)
        picks = picks_from_onoff(onoff, entry["fs"], entry["t0"], cfg.REFRACT)
        for t in picks:
            trigs.append(t)
    trigs_sorted = np.array(sorted(trigs), dtype="datetime64[ns]")
    if trigs_sorted.size == 0:
        return dict(on=on, recall=0.0, fph=0.0, triggers=0)
    hit = vectorized_any_hit(trigs_sorted, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    fph = trigs_sorted.size / max(1e-6, total_hours)
    return dict(on=on, recall=recall, fph=fph, triggers=int(trigs_sorted.size))

base_hi = run_const(4.50)  # low-FP baseline (likely low recall)
base_lo = run_const(3.25)  # high-recall baseline (higher FP)
print("\n[Const ON] 4.50 ->", base_hi)
print("[Const ON] 3.25 ->", base_lo)

# -------------------- 4) Hints (what to tweak next) --------------------
print("\nHints:")
print("- If all recalls are low, first increase POST_DET to 300 (detection-only).")
print("- To increase recall: lower q (e.g., 0.997) or lower alpha (e.g., 1.02).")
print("- To reduce false alarms: raise alpha or add MIN_DUR=0.5–1.0s (then re-run).")
print("- If still no improvement, try STA=1.0s (more sensitive) or BAND_DET=(1.0, 5.0).")


[Info] events considered: 1392 on 17 days | hours of data: 408.0
[Adaptive] q=0.998 alpha=1.03 | recall=0.263 | FPH=3.97 | triggers=1618

Top candidates (by recall desc, FPH asc):
       q  alpha    recall       fph  triggers
0  0.997   1.02  0.318247  5.026964      2051
1  0.997   1.03  0.313937  4.904415      2001
2  0.997   1.05  0.309626  4.713238      1923
3  0.997   1.08  0.301724  4.431375      1808
4  0.997   1.12  0.291667  4.051473      1653
5  0.998   1.02  0.266523  4.056375      1655
6  0.998   1.03  0.262931  3.965689      1618
7  0.998   1.05  0.239943  3.718139      1517
8  0.998   1.08  0.231322  3.389708      1383
9  0.998   1.12  0.213362  2.975492      1214

[Const ON] 4.50 -> {'on': 4.5, 'recall': 0.48419540229885055, 'fph': 7.002455032738818, 'triggers': 2857}
[Const ON] 3.25 -> {'on': 3.25, 'recall': 0.5373563218390804, 'fph': 10.051476405061916, 'triggers': 4101}

Hints:
- If all recalls are low, first increase POST_DET to 300 (detection-only).
- To increase rec

In [24]:
# ============================================================
# Sweep MIN_DUR on chosen (q, alpha) and save triggers  (FIXED)
# ============================================================
import numpy as np, pandas as pd, os
from obspy.signal.trigger import trigger_onset
from obspy import UTCDateTime

# ---- pick your adaptive combo here ----
Q_CHOSEN     = 0.997
ALPHA_CHOSEN = 1.03

# ---- detection constants (reuse from previous cell) ----
OFF      = CFG.OFF
REFRACT  = CFG.REFRACT
FS       = CFG.FS

def picks_from_onoff_with_mindur(onoff, fs, t0, refract, min_dur):
    """Filter on/off by min duration (seconds) and apply refractory merging."""
    picks, last = [], None
    for a, b in onoff:
        dur_s = (b - a) / fs
        if dur_s < min_dur:
            continue
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if (last is None) or ((ts - last).total_seconds() > refract):
            picks.append(ts); last = ts
    return picks

def run_adaptive_with_mindur(q, alpha, min_dur):
    # collect triggers across all available days
    trigs = []
    for dt, entry in cft_cache.items():
        on = float(np.quantile(entry["cft"], q) * alpha)
        onoff = trigger_onset(entry["cft"], on, OFF)
        picks = picks_from_onoff_with_mindur(onoff, entry["fs"], entry["t0"], REFRACT, min_dur)
        for t in picks:
            trigs.append(t)
    trigs_sorted = np.array(sorted(trigs), dtype="datetime64[ns]")
    if trigs_sorted.size == 0:
        return dict(q=q, alpha=alpha, min_dur=min_dur, recall=0.0, fph=0.0, triggers=0, trigs=trigs_sorted)
    # any-trigger-in-window hit logic (ev_start_ns / ev_end_ns defined earlier)
    i = np.searchsorted(trigs_sorted, ev_start_ns, side="left")
    j = np.searchsorted(trigs_sorted, ev_end_ns,   side="right")
    hit = (j - i) > 0
    recall = float(hit.mean())
    fph = trigs_sorted.size / max(1e-6, total_hours)
    return dict(q=q, alpha=alpha, min_dur=min_dur, recall=recall, fph=fph, triggers=int(trigs_sorted.size), trigs=trigs_sorted)

# ---- sweep MIN_DUR ----
cand_mindur = [0.0, 0.25, 0.5, 1.0]
rows = []
res_cache = {}
for md in cand_mindur:
    res = run_adaptive_with_mindur(Q_CHOSEN, ALPHA_CHOSEN, md)
    rows.append({k: v for k, v in res.items() if k not in ("trigs",)})
    res_cache[md] = res
scan_md = pd.DataFrame(rows).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print(scan_md)

# ---- pick a working point (example: best recall under FPH<=5) ----
budget_fph = 5.0
candidates = scan_md[scan_md["fph"] <= budget_fph]
if len(candidates) == 0:
    chosen_row = scan_md.iloc[0]
else:
    # highest recall within budget_fph, tie-breaker by lower fph
    chosen_row = candidates.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]

md_chosen = float(chosen_row["min_dur"])
trigs_final = res_cache[md_chosen]["trigs"]

print(f"\n[Chosen] q={Q_CHOSEN} alpha={ALPHA_CHOSEN} min_dur={md_chosen} "
      f"| recall={chosen_row['recall']:.3f} FPH={chosen_row['fph']:.2f} triggers={int(chosen_row['triggers'])}")

# ---- save triggers to CSV for downstream (re-centering + CNN cascade) ----
out_dir = "runs/cascade_eval"
os.makedirs(out_dir, exist_ok=True)
csv_out = os.path.join(out_dir, f"triggers_adapt_q{Q_CHOSEN}_a{ALPHA_CHOSEN}_md{md_chosen}.csv")

if trigs_final.size == 0:
    # still save empty file with correct columns
    pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    print(f"[Saved EMPTY] {csv_out}")
else:
    trig_ts = pd.to_datetime(trigs_final)                       # DatetimeIndex
    date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')       # <-- FIX: get date as string safely
    trig_df = pd.DataFrame({
        "trigger_time": trig_ts,
        "date": date_str
    }).sort_values("trigger_time").reset_index(drop=True)
    trig_df.to_csv(csv_out, index=False)
    print(f"[Saved] {csv_out} (rows={len(trig_df)})")


       q  alpha  min_dur    recall       fph  triggers
0  0.997   1.03     0.00  0.313937  4.904415      2001
1  0.997   1.03     0.25  0.313937  4.904415      2001
2  0.997   1.03     0.50  0.313937  4.904415      2001
3  0.997   1.03     1.00  0.313937  4.904415      2001

[Chosen] q=0.997 alpha=1.03 min_dur=0.0 | recall=0.314 FPH=4.90 triggers=2001
[Saved] runs/cascade_eval/triggers_adapt_q0.997_a1.03_md0.0.csv (rows=2001)


In [25]:
# ============================================================
# Cascade eval: triggers CSV -> re-center -> cut -> z-score -> strict CNN
# ============================================================
import os, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.filter import envelope
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import torch, torch.nn as nn, torch.nn.functional as F

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"
    TRIG_CSV   : str = "runs/cascade_eval/triggers_adapt_q0.997_a1.03_md0.0.csv"

    # Waveforms (for cutting windows)
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"      # e.g., MAJO_2011-03-05.mseed
    FS         : int  = 20
    BAND_CUT   : tuple = (0.5, 8.0)             # bandpass used before cutting

    # Windowing
    PRE        : int = 20                       # seconds (for CNN input)
    POST       : int = 70
    # Detection-eval window (should match you前面的评估；只用于选择命中触发)
    PRE_DET    : int = 20
    POST_DET   : int = 180

    # Re-centering search around trigger
    RC_PRE     : int = 10
    RC_POST    : int = 20

    # Model
    BEST_PT    : str = "runs/cnn_strict/best.pt"
    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

LABEL_MAP = {"S":0, "M":1, "L":2}

# -------------------- Data helpers --------------------
def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid = np.array([str(s) for s in (d["sample_id"] if "sample_id" in d else d["sample_ids"])])
    y = d["mag_class"].astype(int) if "mag_class" in d else d["labels"].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos = d["detect_label"].astype(int) == 1 if "detect_label" in d else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_split_scopes(csv_path, split_path):
    with open(split_path, "r") as f: splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """Return df_te sorted by event_time, and y_te_sorted aligned."""
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_split_scopes(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, "s"))
    y_te = pd.Series(y[mte]).reset_index(drop=True)
    order = np.argsort(evt_time.values)
    df_te = pd.DataFrame({"event_time": evt_time.values[order]})
    y_te_sorted = y_te.iloc[order].reset_index(drop=True).to_numpy()
    return df_te, y_te_sorted

def train_mean_std(cfg: Cfg):
    """Global mean/std from TRAIN positives (strict comparable)."""
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    tr_scope, _ = get_split_scopes(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mtr = np.isin(sid, list(tr_scope))
    Xtr = X[mtr]
    flat = Xtr.reshape(len(Xtr), -1)
    mean, std = float(flat.mean()), float(flat.std() + 1e-8)
    # optional: also save for reuse
    with open(os.path.join(cfg.OUT_DIR, "mean_std_from_train.json"), "w") as f:
        json.dump({"mean": mean, "std": std}, f, indent=2)
    return mean, std

# -------------------- Waveform cutting --------------------
_trace_cache = {}
def get_trace_for_day(cfg: Cfg, day_str: str):
    if day_str not in _trace_cache:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=day_str))
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        if abs(tr.stats.sampling_rate - cfg.FS) > 1e-6:
            tr.resample(cfg.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=cfg.BAND_CUT[0], freqmax=cfg.BAND_CUT[1])
        _trace_cache[day_str] = tr
    return _trace_cache[day_str]

def recenter_trigger(tr, t_ts, fs, pre, post):
    """Find local envelope peak in [t- pre, t+ post] and return adjusted timestamp."""
    t0 = UTCDateTime(t_ts.to_pydatetime()) - pre
    t1 = UTCDateTime(t_ts.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need:
        return t_ts
    env = envelope(x)
    i = int(np.argmax(env))
    t_pk = t0 + i / fs
    return pd.Timestamp(UTCDateTime(t_pk).datetime)

# -------------------- Strict CNN model (same as training) --------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k // 2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn   = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d): 
        super().__init__(); self.W = nn.Linear(d, d); self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch, 32), ConvBlock(32, 64), ConvBlock(64, 128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2 * hidden)
        self.head = nn.Sequential(nn.Linear(2 * hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x):
        z = self.cnn(x)          # [B,C,L]
        z = z.transpose(1, 2)    # [B,L,C]
        H, _ = self.lstm(z)      # [B,L,2H]
        Z, _ = self.attn(H)      # [B,2H]
        return self.head(Z)

# -------------------- 1) Build test events & load triggers --------------------
df_te, y_te_sorted = build_test_events(CFG)
trig_df = pd.read_csv(CFG.TRIG_CSV)
trig_df["trigger_time"] = pd.to_datetime(trig_df["trigger_time"])
trig_df = trig_df.sort_values("trigger_time").reset_index(drop=True)

# detection-eval windows (only for selecting hits)
ev = df_te.copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, "s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, "s")

tr = trig_df["trigger_time"].to_numpy(dtype="datetime64[ns]")
ev_start = ev["start"].to_numpy(dtype="datetime64[ns]")
ev_end   = ev["end"].to_numpy(dtype="datetime64[ns]")

# searchsorted to find the first trigger >= start, then check if <= end
i = np.searchsorted(tr, ev_start, side="left")
j = np.searchsorted(tr, ev_end,   side="right")
hit = (j - i) > 0
first_idx = np.where(hit, i, -1)

print(f"[Info] events={len(ev)} | hits={int(hit.sum())} | hit_rate={hit.mean():.3f}")

# -------------------- 2) Re-center & cut CNN windows for hits --------------------
win_len = CFG.FS * (CFG.PRE + CFG.POST)
X_cut, y_hit, kept = [], [], []
for idx in np.where(hit)[0]:
    t_first = pd.Timestamp(tr[first_idx[idx]].astype("datetime64[ns]").astype('datetime64[ns]').astype('datetime64[ns]'))
    day_str = str(t_first.date())
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=day_str))
    if not os.path.exists(fp): 
        continue
    tr_day = get_trace_for_day(CFG, day_str)
    t_center = recenter_trigger(tr_day, t_first, fs=CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)
    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr_day.slice(t0, t1).data
    if len(x) >= win_len:
        X_cut.append(x[:win_len].astype(np.float32))
        y_hit.append(y_te_sorted[idx])
        kept.append(idx)

if len(X_cut) == 0:
    raise SystemExit("No windows cut. Check MSEED filenames and detection window settings.")

X_cut = np.stack(X_cut, axis=0)
y_hit = np.array(y_hit, dtype=int)

print(f"[Cut] windows={len(X_cut)} (of {int(hit.sum())} hits) | shape={X_cut.shape}")

# -------------------- 3) Global z-score (TRAIN-only mean/std) --------------------
mean, std = train_mean_std(CFG)
Xn = (X_cut - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]  # [N,1,T]

# -------------------- 4) Load strict CNN and infer --------------------
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model = CNNBiLSTMAttn(in_ch=1, n_classes=3).to(device)
state = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state, strict=True)
model.eval()

y_pred = []
with torch.no_grad():
    for i0 in range(0, len(Xn), 512):
        xb = torch.from_numpy(Xn[i0:i0+512]).to(device)
        logits = model(xb)
        y_pred.extend(logits.argmax(1).cpu().numpy())
y_pred = np.array(y_pred, dtype=int)

# -------------------- 5) Report & save --------------------
rep = classification_report(y_hit, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0)
cm = confusion_matrix(y_hit, y_pred, labels=[0,1,2])
macro_f1 = f1_score(y_hit, y_pred, average="macro")

print("\n== CNN (cascade, adaptive ON + re-center) ==")
print(rep)
print("Confusion matrix:\n", cm)

out_txt = os.path.join(CFG.OUT_DIR, "cascade_report.txt")
with open(out_txt, "w") as f:
    f.write(rep + "\n")
    f.write("Confusion matrix:\n" + np.array2string(cm))

np.savez(os.path.join(CFG.OUT_DIR, "cascade_preds.npz"),
         y_true=y_hit, y_pred=y_pred, kept_event_indices=np.array(kept, dtype=int))

print(f"[Saved] {out_txt}")


[Info] events=1392 | hits=437 | hit_rate=0.314
[Cut] windows=437 (of 437 hits) | shape=(437, 1800)

== CNN (cascade, adaptive ON + re-center) ==
              precision    recall  f1-score   support

           S     0.9333    0.9044    0.9186       387
           M     0.3833    0.4694    0.4220        49
           L     0.0000    0.0000    0.0000         1

    accuracy                         0.8535       437
   macro avg     0.4389    0.4579    0.4469       437
weighted avg     0.8695    0.8535    0.8608       437

Confusion matrix:
 [[350  36   1]
 [ 25  23   1]
 [  0   1   0]]
[Saved] runs/cascade_eval/cascade_report.txt


In [26]:
# ============================================================
# STA/LTA detection (POST_DET=300, REFRACT=120, multi-band 2-of-3)
# - Rebuild test events from your artifacts
# - Build daily CFTs on 3 bands
# - Scan (q, alpha) with 2-of-3 fusion, pick best under FPH<=6/h
# - Then sweep MIN_DUR to reduce FPH further
# - Save final triggers CSV for cascade use
# ============================================================

import os, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"   # e.g., MAJO_2011-03-05.mseed

    # Detection sampling
    FS         : int   = 20

    # Three bands for 2-of-3 fusion
    BANDS      : tuple = ((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows
    STA        : float = 1.5
    LTA        : float = 20.0

    # Trigger logic
    OFF        : float = 1.0
    REFRACT    : int   = 120       # step-2: shorter refractory period

    # Detection-eval window (NOT used for cutting CNN windows)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300       # step-1: wider POST for evaluation only

    # Adaptive parameters (grids below will override)
    ADAPT_Q    : float = 0.998
    ALPHA      : float = 1.03

    # Search grids / budgets
    GRID_Q     : tuple = (0.997, 0.998, 0.999)
    GRID_ALPHA : tuple = (1.02, 1.03, 1.05, 1.08, 1.12)
    FPH_BUDGET : float = 6.0       # pick the highest recall under this FPH

    # Second-stage min duration sweep (after picking q/alpha)
    MIN_DUR_SET: tuple = (0.0, 0.5, 1.0)

    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

# -------------------- Helpers to rebuild test events --------------------
def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid = np.array([str(s) for s in (d["sample_id"] if "sample_id" in d else d["sample_ids"])])
    y = d["mag_class"].astype(int) if "mag_class" in d else d["labels"].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos = d["detect_label"].astype(int) == 1 if "detect_label" in d else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    Test events = positives within TEST scope.
    Event time (catalog-origin) = window_start + 20s (PRE for classification dataset).
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- Build daily multi-band CFTs --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """
    Read one MiniSEED file once; resample & detrend; then for each band:
    - bandpass
    - compute STA/LTA characteristic function (CFT)
    Returns: dict(fs, t0, hours, cft_list=[cft_band1, cft_band2, cft_band3])
    """
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)

    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)

    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

def fuse_onoff_2of3(onoffs):
    """
    2-of-3 fusion on interval level.
    onoffs: list of arrays [[(a1,b1), (a2,b2),...], ...] in sample indices (half-open).
    Return fused intervals where >=2 bands are ON simultaneously.
    """
    events = []
    for arr in onoffs:
        for a, b in arr:
            events.append((int(a), +1))
            events.append((int(b), -1))
    if not events:
        return np.empty((0,2), dtype=int)
    # Sort with starts before ends at the same position
    events.sort(key=lambda x: (x[0], -x[1]))
    fused = []
    active = 0
    current_start = None
    for pos, delta in events:
        prev = active
        active += delta
        if prev < 2 and active >= 2:
            current_start = pos
        elif prev >= 2 and active < 2 and current_start is not None:
            fused.append((current_start, pos))
            current_start = None
    return np.array(fused, dtype=int) if fused else np.empty((0,2), dtype=int)

def filter_min_dur(onoff, fs, min_dur_s):
    if onoff.size == 0 or min_dur_s <= 0:
        return onoff
    keep = ((onoff[:,1] - onoff[:,0]) / fs) >= float(min_dur_s)
    return onoff[keep]

def picks_from_onoff(onoff, fs, t0, refract):
    """
    Convert fused intervals to pick times (start of each interval), 
    with refractory merging in wall-clock time.
    """
    picks, last = [], None
    for a, b in onoff:
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if (last is None) or ((ts - last).total_seconds() > refract):
            picks.append(ts); last = ts
    return picks

# -------------------- Main detection build & scan --------------------
# 1) Test events and available days
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in event_days:
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    cftd = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

if not dates_ok:
    raise SystemExit("No overlapping days between df_te and waveform files.")

# Detection-eval windows (use wider POST_DET=300 for evaluation)
ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, "s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, "s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

print(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

def run_adaptive_multiband(q, alpha, min_dur_s=0.0):
    """
    For each day:
      - per-band ON = quantile(cft_band, q) * alpha
      - per-band on/off = trigger_onset(cft_band, ON, OFF)
      - 2-of-3 fusion on intervals
      - filter by min_dur
      - refractory merge to picks
    Aggregate all picks across days, then compute recall/FPH.
    """
    all_picks = []
    for dt, entry in cft_cache.items():
        onoffs = []
        for cft in entry["cft_list"]:
            on_val = float(np.quantile(cft, q) * alpha)
            onoff  = trigger_onset(cft, on_val, CFG.OFF)
            onoffs.append(onoff)
        fused = fuse_onoff_2of3(onoffs)
        fused = filter_min_dur(fused, entry["fs"], min_dur_s)
        picks = picks_from_onoff(fused, entry["fs"], entry["t0"], CFG.REFRACT)
        all_picks.extend(picks)
    trigs_ns = np.array(sorted(all_picks), dtype="datetime64[ns]")
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    fph = trigs_ns.size / max(1e-6, total_hours)
    return dict(q=q, alpha=alpha, min_dur=min_dur_s, recall=recall, fph=fph, triggers=int(trigs_ns.size), trigs=trigs_ns)

# 2) Grid search (q, alpha) under FPH<=budget, with MIN_DUR fixed at 0.0
rows = []
for q in CFG.GRID_Q:
    for a in CFG.GRID_ALPHA:
        rows.append(run_adaptive_multiband(q, a, 0.0))
scan = pd.DataFrame([{k:v for k,v in r.items() if k!='trigs'} for r in rows])
scan = scan.sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nTop (q,alpha) candidates before MIN_DUR:")
print(scan.head(10))

# Pick best recall under FPH budget
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
if len(cands) == 0:
    chosen = scan.iloc[0]
else:
    chosen = cands.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
Q_CHOSEN, A_CHOSEN = float(chosen["q"]), float(chosen["alpha"])
print(f"\n[Chosen (pre-MIN_DUR)] q={Q_CHOSEN} alpha={A_CHOSEN} | recall={chosen['recall']:.3f} FPH={chosen['fph']:.2f} triggers={int(chosen['triggers'])}")

# 3) Sweep MIN_DUR at the chosen (q, alpha)
rows_md = []
res_cache = {}
for md in CFG.MIN_DUR_SET:
    r = run_adaptive_multiband(Q_CHOSEN, A_CHOSEN, md)
    rows_md.append({k:v for k,v in r.items() if k!='trigs'})
    res_cache[md] = r
scan_md = pd.DataFrame(rows_md).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nAfter MIN_DUR sweep:")
print(scan_md)

# Choose highest recall under budget (tie-breaker: lower FPH)
cands2 = scan_md[scan_md["fph"] <= CFG.FPH_BUDGET]
if len(cands2) == 0:
    final = scan_md.iloc[0]
else:
    final = cands2.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]

MD_CHOSEN = float(final["min_dur"])
trigs_final = res_cache[MD_CHOSEN]["trigs"]

print(f"\n[Final] q={Q_CHOSEN} alpha={A_CHOSEN} min_dur={MD_CHOSEN} | recall={final['recall']:.3f} FPH={final['fph']:.2f} triggers={int(final['triggers'])}")

# 4) Save triggers CSV for cascade
csv_out = os.path.join(CFG.OUT_DIR, f"triggers_MB_2of3_q{Q_CHOSEN}_a{A_CHOSEN}_md{MD_CHOSEN}.csv")
if trigs_final.size == 0:
    pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    print(f"[Saved EMPTY] {csv_out}")
else:
    trig_ts = pd.to_datetime(trigs_final)
    date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
    trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
    trig_df.to_csv(csv_out, index=False)
    print(f"[Saved] {csv_out} (rows={len(trig_df)})")


[Info] events considered: 1392 on 17 days | hours=408.0

Top (q,alpha) candidates before MIN_DUR:
       q  alpha  min_dur    recall       fph  triggers
0  0.997   1.02      0.0  0.508621  3.811277      1555
1  0.997   1.03      0.0  0.495690  3.713237      1515
2  0.997   1.05      0.0  0.486351  3.566179      1455
3  0.997   1.08      0.0  0.461925  3.362747      1372
4  0.997   1.12      0.0  0.426006  3.044119      1242
5  0.998   1.02      0.0  0.407328  2.946080      1202
6  0.998   1.03      0.0  0.397270  2.852943      1164
7  0.998   1.05      0.0  0.378592  2.671570      1090
8  0.998   1.08      0.0  0.329023  2.355394       961
9  0.998   1.12      0.0  0.267960  1.867648       762

[Chosen (pre-MIN_DUR)] q=0.997 alpha=1.02 | recall=0.509 FPH=3.81 triggers=1555

After MIN_DUR sweep:
       q  alpha  min_dur    recall       fph  triggers
0  0.997   1.02      0.0  0.508621  3.811277      1555
1  0.997   1.02      1.0  0.507902  3.767159      1537
2  0.997   1.02      0.5  0.5

In [27]:
# ============================================================
# Cascade: use multi-band 2-of-3 triggers -> re-center -> cut -> z-score -> strict CNN
# Uses: POST_DET=300s evaluation, REFRACT=120s (already in detection stage)
# ============================================================
import os, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.filter import envelope
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch, torch.nn as nn, torch.nn.functional as F

# -------------------- Config --------------------
@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Triggers from your latest detection run:
    TRIG_CSV   : str = "runs/cascade_eval/triggers_MB_2of3_q0.997_a1.02_md0.0.csv"

    # Continuous waveforms (same station/day naming)
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int  = 20
    BAND_CUT   : tuple = (0.5, 8.0)  # band for cutting

    # CNN window (must match training: 20s pre + 70s post)
    PRE        : int = 20
    POST       : int = 70

    # Detection-eval window (should match the detection eval you just used)
    PRE_DET    : int = 20
    POST_DET   : int = 300

    # Re-centering search window around the trigger
    RC_PRE     : int = 10
    RC_POST    : int = 20

    # Strict CNN checkpoint (the “comparable” model you trained)
    BEST_PT    : str = "runs/cnn_strict/best.pt"
    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

LABEL_MAP = {"S":0, "M":1, "L":2}

# -------------------- Data helpers --------------------
def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid = np.array([str(s) for s in (d.get("sample_id", d.get("sample_ids")))])
    y = d.get("mag_class", d.get("labels")).astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos = d.get("detect_label", np.ones(len(y), dtype=int)).astype(int) == 1
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """Test events = positives in TEST scope; event_time = window_start + 20s."""
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, "s"))
    y_te = pd.Series(y[mte]).reset_index(drop=True)
    order = np.argsort(evt_time.values)
    df_te = pd.DataFrame({"event_time": evt_time.values[order]})
    y_te_sorted = y_te.iloc[order].to_numpy()
    return df_te.reset_index(drop=True), y_te_sorted

def train_mean_std(cfg: Cfg):
    """Compute global mean/std from TRAIN positives for strict comparability."""
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    tr_scope, _ = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mtr = np.isin(sid, list(tr_scope))
    flat = X[mtr].reshape(np.sum(mtr), -1)
    mean, std = float(flat.mean()), float(flat.std() + 1e-8)
    with open(os.path.join(cfg.OUT_DIR, "mean_std_from_train.json"), "w") as f:
        json.dump({"mean": mean, "std": std}, f, indent=2)
    return mean, std

# -------------------- Waveform cutting --------------------
_trace_cache = {}
def get_trace_for_day(cfg: Cfg, day_str: str):
    if day_str not in _trace_cache:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=day_str))
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        if abs(tr.stats.sampling_rate - cfg.FS) > 1e-6:
            tr.resample(cfg.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=cfg.BAND_CUT[0], freqmax=cfg.BAND_CUT[1])
        _trace_cache[day_str] = tr
    return _trace_cache[day_str]

def recenter_trigger(tr, t_ts, fs, pre, post):
    """Find envelope peak near trigger to center the CNN window."""
    t0 = UTCDateTime(t_ts.to_pydatetime()) - pre
    t1 = UTCDateTime(t_ts.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need:
        return t_ts
    env = envelope(x)
    i = int(np.argmax(env))
    t_pk = t0 + i / fs
    return pd.Timestamp(UTCDateTime(t_pk).datetime)

# -------------------- Strict CNN (same as your training code) --------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k // 2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn   = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d): 
        super().__init__(); self.W = nn.Linear(d, d); self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch, 32), ConvBlock(32, 64), ConvBlock(64, 128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2 * hidden)
        self.head = nn.Sequential(nn.Linear(2 * hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x):
        z = self.cnn(x)          # [B,C,L]
        z = z.transpose(1, 2)    # [B,L,C]
        H, _ = self.lstm(z)      # [B,L,2H]
        Z, _ = self.attn(H)      # [B,2H]
        return self.head(Z)

# -------------------- 1) Build test events & load triggers --------------------
df_te, y_te_sorted = build_test_events(CFG)

trig_df = pd.read_csv(CFG.TRIG_CSV)
trig_df["trigger_time"] = pd.to_datetime(trig_df["trigger_time"])
trig_df = trig_df.sort_values("trigger_time").reset_index(drop=True)

# detection-eval windows (match POST_DET=300)
ev = df_te.copy()
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, "s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, "s")

tr = trig_df["trigger_time"].to_numpy(dtype="datetime64[ns]")
ev_start = ev["start"].to_numpy(dtype="datetime64[ns]")
ev_end   = ev["end"].to_numpy(dtype="datetime64[ns]")

i = np.searchsorted(tr, ev_start, side="left")
j = np.searchsorted(tr, ev_end,   side="right")
hit = (j - i) > 0
first_idx = np.where(hit, i, -1)

print(f"[Info] events={len(ev)} | hits={int(hit.sum())} | hit_rate={hit.mean():.3f}")

# -------------------- 2) Re-center & cut CNN windows for hits --------------------
win_len = CFG.FS * (CFG.PRE + CFG.POST)
X_cut, y_hit, kept = [], [], []
for idx in np.where(hit)[0]:
    t_first_ns = tr[first_idx[idx]]
    t_first = pd.Timestamp(t_first_ns)
    day_str = str(t_first.date())
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=day_str))
    if not os.path.exists(fp): 
        continue
    tr_day = get_trace_for_day(CFG, day_str)
    t_center = recenter_trigger(tr_day, t_first, fs=CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)
    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr_day.slice(t0, t1).data
    if len(x) >= win_len:
        X_cut.append(x[:win_len].astype(np.float32))
        y_hit.append(y_te_sorted[idx])
        kept.append(idx)

if len(X_cut) == 0:
    raise SystemExit("No windows cut; check filenames or time bounds.")

X_cut = np.stack(X_cut, axis=0)
y_hit = np.array(y_hit, dtype=int)
print(f"[Cut] windows={len(X_cut)} (of {int(hit.sum())} hits) | shape={X_cut.shape}")

# -------------------- 3) Global z-score (TRAIN-only mean/std) --------------------
mean, std = train_mean_std(CFG)
Xn = (X_cut - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]  # [N,1,T]

# -------------------- 4) Load strict CNN and infer --------------------
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model = CNNBiLSTMAttn(in_ch=1, n_classes=3).to(device)
state = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state, strict=True)
model.eval()

y_pred = []
with torch.no_grad():
    for i0 in range(0, len(Xn), 512):
        xb = torch.from_numpy(Xn[i0:i0+512]).to(device)
        logits = model(xb)
        y_pred.extend(logits.argmax(1).cpu().numpy())
y_pred = np.array(y_pred, dtype=int)

# -------------------- 5) Reports: within-detected & end-to-end recall --------------------
rep = classification_report(y_hit, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0)
cm = confusion_matrix(y_hit, y_pred, labels=[0,1,2])
macro_f1 = f1_score(y_hit, y_pred, average="macro")

print("\n== CNN (cascade on detected events) ==")
print(rep)
print("Confusion matrix:\n", cm)

# End-to-end per-class recall (missed detections count as errors)
lab = y_te_sorted
det = hit
correct_mask = np.zeros_like(lab, dtype=bool)
# mark correct classifications among detected events
kept_idx = np.array(kept, dtype=int)
# map: kept event index -> correct?
for k, idx in enumerate(kept_idx):
    if y_hit[k] == y_pred[k]:
        correct_mask[idx] = True

for c, name in enumerate(["S","M","L"]):
    total_c = (lab == c).sum()
    e2e_c = correct_mask[lab == c].sum() / max(1, total_c)
    print(f"[End-to-End] {name} recall = {e2e_c:.3f} (total {total_c})")

# Save artifacts
with open(os.path.join(CFG.OUT_DIR, "cascade_MB_2of3_report.txt"), "w") as f:
    f.write(rep + "\n")
    f.write("Confusion matrix:\n" + np.array2string(cm))

np.savez(os.path.join(CFG.OUT_DIR, "cascade_MB_2of3_preds.npz"),
         y_true=y_hit, y_pred=y_pred, kept_event_indices=np.array(kept, dtype=int))

print(f"[Saved] {os.path.join(CFG.OUT_DIR, 'cascade_MB_2of3_report.txt')}")


[Info] events=1392 | hits=708 | hit_rate=0.509
[Cut] windows=708 (of 708 hits) | shape=(708, 1800)

== CNN (cascade on detected events) ==
              precision    recall  f1-score   support

           S     0.9384    0.9104    0.9242       636
           M     0.3448    0.4348    0.3846        69
           L     0.0000    0.0000    0.0000         3

    accuracy                         0.8602       708
   macro avg     0.4277    0.4484    0.4363       708
weighted avg     0.8766    0.8602    0.8677       708

Confusion matrix:
 [[579  54   3]
 [ 38  30   1]
 [  0   3   0]]
[End-to-End] S recall = 0.453 (total 1277)
[End-to-End] M recall = 0.291 (total 103)
[End-to-End] L recall = 0.000 (total 12)
[Saved] runs/cascade_eval/cascade_MB_2of3_report.txt


In [31]:
# ============================================================
# Multiband STA/LTA Detection v2 (FIXED)
# - Rebuild TEST events from your NPZ + frozen_splits
# - Build daily CFTs across 4 bands (default; you can drop to 3)
# - Grid-search (q, alpha, refractory, adapt_scope) under FPH budget
# - Fusion: K-of-N (default 2-of-N) OR optional OR+NMS fusion
# - Sweep MIN_DUR
# - Save final triggers CSV for cascade use
# Notes:
# * This version normalizes trigger_onset() output to ndarray, fixing
#   environments where it returns a list instead of numpy array.
# ============================================================

import os
import json
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"   # e.g., MAJO_2011-03-05.mseed
    FS         : int = 20

    # Bands for multiband STA/LTA (v2 adds a low-frequency band)
    # You can switch back to 3 bands by removing the first tuple
    BANDS      : tuple = ((0.2, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows (in seconds)
    STA        : float = 1.5
    LTA        : float = 20.0

    # Trigger hysteresis: LTA OFF threshold
    OFF        : float = 1.0

    # Refractory period (seconds): will also be grid-searched
    REFRACT    : int   = 120

    # Detection-eval window (NOT used for cutting CNN windows)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # Search grids / budgets (expanded q/alpha and refractory)
    GRID_Q      : tuple = (0.994, 0.995, 0.996, 0.997)
    GRID_ALPHA  : tuple = (1.00, 1.01, 1.02, 1.03, 1.05)
    GRID_REFRACT: tuple = (60, 90, 120)
    ADAPT_SCOPES: tuple = ("day", "hour")  # 'hour' = hourly quantiles (better under drift)
    FPH_BUDGET  : float = 6.0

    # Second-stage min duration sweep (after picking q/alpha/refract/scope)
    MIN_DUR_SET: tuple = (0.0, 0.5, 1.0)

    # Fusion mode: "kofn" (default 2-of-N) or "or_nms" (union then NMS + refractory)
    FUSE_MODE   : str  = "kofn"
    K_FOR_KOFN  : int  = 2  # 2-of-N

    # OR+NMS settings (only if FUSE_MODE == "or_nms")
    NMS_SEC     : int  = 45  # minimum separation before applying refractory

    OUT_DIR     : str  = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

# -------------------- NPZ helpers --------------------
def _first_key(d, candidates):
    """Return the first existing key in NPZ among candidates."""
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    """Load positives from NPZ and return waveforms, labels, ids, window_starts (UTC-naive)."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    """Return train/test scopes as sets of sample_id strings."""
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    Test events = positives within TEST scope.
    Event time (catalog-origin) = window_start + 20s (classification dataset convention).
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- CFT build --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """
    Read one MiniSEED file; resample & detrend; then for each band:
    - bandpass
    - compute STA/LTA characteristic function (CFT)
    Returns: dict(fs, t0, hours, cft_list=[cft_band1, ...])
    """
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)

    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)

    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

# -------------------- Normalizers & fusion helpers --------------------
def to_onoff_array(x):
    """
    Normalize trigger_onset output to a (N,2) int ndarray.
    ObsPy may return a list on some versions; this makes it consistent.
    """
    arr = np.asarray(x, dtype=int)
    if arr.size == 0:
        return np.empty((0, 2), dtype=int)
    # ensure shape (N,2)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0:
            raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def fuse_onoff_kofn(onoffs, k=2):
    """
    K-of-N fusion on interval level.
    onoffs: list of arrays [[(a1,b1), (a2,b2),...], ...] in sample indices (half-open).
    Return fused intervals where >=k bands are ON simultaneously.
    """
    events = []
    for arr in onoffs:
        if arr is None:
            continue
        for a, b in arr:
            events.append((int(a), +1))
            events.append((int(b), -1))
    if not events:
        return np.empty((0,2), dtype=int)
    # Sort with starts before ends at the same position
    events.sort(key=lambda x: (x[0], -x[1]))
    fused = []
    active = 0
    current_start = None
    for pos, delta in events:
        prev = active
        active += delta
        if prev < k and active >= k:
            current_start = pos
        elif prev >= k and active < k and current_start is not None:
            fused.append((current_start, pos))
            current_start = None
    return np.array(fused, dtype=int) if fused else np.empty((0,2), dtype=int)

def picks_or_then_nms(onoffs, fs, t0, nms_sec=45, refract=60):
    """
    OR fusion (union of all band ON starts) followed by:
      - NMS-like temporal suppression with min spacing `nms_sec`
      - final refractory thinning with `refract`
    Returns a list of pandas.Timestamp.
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0:
            continue
        for a, b in arr:
            # Use the start of each ON interval as a tentative pick
            picks.append(t0 + a / fs)  # UTCDateTime
    if not picks:
        return []
    picks.sort()
    # NMS-style pre-merge
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    # Final refractory thinning
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def filter_min_dur(onoff, fs, min_dur_s):
    """Filter intervals shorter than `min_dur_s` seconds."""
    if onoff.size == 0 or min_dur_s <= 0:
        return onoff
    keep = ((onoff[:,1] - onoff[:,0]) / fs) >= float(min_dur_s)
    return onoff[keep]

def picks_from_onoff(onoff, fs, t0, refract):
    """
    Convert fused intervals to pick times (use the start of each interval),
    with refractory merging in wall-clock time.
    """
    picks, last = [], None
    for a, b in onoff:
        t = t0 + a / fs
        ts = pd.Timestamp(UTCDateTime(t).datetime)
        if (last is None) or ((ts - last).total_seconds() > refract):
            picks.append(ts); last = ts
    return picks

# -------------------- Vectorized eval helpers --------------------
def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Return boolean hits for each event window; a hit if any trigger ∈ [start, end]."""
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Main detection build & scan --------------------
# 1) Test events and available days
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in event_days:
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    cftd = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

if not dates_ok:
    raise SystemExit("No overlapping days between df_te and waveform files.")

# Detection-eval windows (use wider POST_DET=300 for evaluation)
ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, "s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, "s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

print(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

def run_adaptive_multiband(q, alpha, min_dur_s=0.0, refract=None, adapt_scope="day", fuse_mode="kofn", k_for_kofn=2):
    """
    For each day:
      - per-band ON threshold via quantile(cft, q) * alpha
        * adapt_scope = 'day' -> whole-day quantile
        * adapt_scope = 'hour' -> per-hour quantiles (robust to drift)
      - per-band on/off = trigger_onset(cft, ON, OFF)  (normalized to ndarray)
      - fuse intervals:
          * 'kofn': K-of-N interval fusion
          * 'or_nms': OR union of all bands' starts -> NMS -> refractory
      - filter by min_dur (interval domain only)
      - refractory merge to picks (for 'kofn'), or already applied in 'or_nms'
    Aggregate picks across days, then compute recall/FPH.
    """
    refract = CFG.REFRACT if refract is None else int(refract)
    all_picks = []
    for dt, entry in cft_cache.items():
        onoffs = []
        for cft in entry["cft_list"]:
            if adapt_scope == "day":
                if cft.size == 0:
                    onoff = np.empty((0,2), dtype=int)
                else:
                    on_val = float(np.quantile(cft, q) * alpha)
                    raw = trigger_onset(cft, on_val, CFG.OFF)
                    onoff = to_onoff_array(raw)  # <-- normalize
            elif adapt_scope == "hour":
                fs = entry["fs"]; H = int(3600 * fs)
                segs = []
                for s in range(0, len(cft), H):
                    e = min(s + H, len(cft))
                    seg = cft[s:e]
                    if seg.size == 0:
                        _onoff = np.empty((0, 2), dtype=int)
                    else:
                        on_val = float(np.quantile(seg, q) * alpha)
                        raw = trigger_onset(seg, on_val, CFG.OFF)
                        _onoff = to_onoff_array(raw)  # <-- normalize
                        if _onoff.size:
                            _onoff[:,0] += s; _onoff[:,1] += s
                    segs.append(_onoff)
                onoff = np.vstack(segs) if len(segs) else np.empty((0,2), dtype=int)
            else:
                raise ValueError("adapt_scope must be 'day' or 'hour'")
            onoffs.append(onoff)

        if fuse_mode == "kofn":
            fused = fuse_onoff_kofn(onoffs, k=k_for_kofn)
            fused = filter_min_dur(fused, entry["fs"], min_dur_s)
            picks = picks_from_onoff(fused, entry["fs"], entry["t0"], refract)
        elif fuse_mode == "or_nms":
            # min_dur does not apply here since we use pick starts only
            picks = picks_or_then_nms(onoffs, entry["fs"], entry["t0"], nms_sec=CFG.NMS_SEC, refract=refract)
        else:
            raise ValueError("fuse_mode must be 'kofn' or 'or_nms'")

        all_picks.extend(picks)

    trigs_ns = np.array(sorted(all_picks), dtype="datetime64[ns]")
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    fph = trigs_ns.size / max(1e-6, total_hours)
    return dict(q=q, alpha=alpha, min_dur=min_dur_s, recall=recall, fph=fph,
                triggers=int(trigs_ns.size), trigs=trigs_ns, refract=refract, adapt=adapt_scope)

# 2) Grid search (q, alpha, refractory, adapt_scope) under FPH budget; MIN_DUR fixed at 0.0
rows = []
for scope in CFG.ADAPT_SCOPES:
    for q in CFG.GRID_Q:
        for a in CFG.GRID_ALPHA:
            for rf in CFG.GRID_REFRACT:
                rows.append(run_adaptive_multiband(q, a, 0.0, refract=rf, adapt_scope=scope,
                                                   fuse_mode=CFG.FUSE_MODE, k_for_kofn=CFG.K_FOR_KOFN))

scan = pd.DataFrame([{k:v for k,v in r.items() if k!='trigs'} for r in rows])
scan = scan.sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nTop candidates before MIN_DUR:")
print(scan.head(12))

# Pick best recall under FPH budget (tie-breaker: lower FPH)
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
chosen = (cands if len(cands) else scan).sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
Q_CHOSEN, A_CHOSEN = float(chosen["q"]), float(chosen["alpha"])
RF_CHOSEN, SCOPE_CHOSEN = int(chosen["refract"]), str(chosen["adapt"])
print(f"\n[Chosen pre-MIN_DUR] q={Q_CHOSEN} alpha={A_CHOSEN} refract={RF_CHOSEN} scope={SCOPE_CHOSEN} "
      f"| recall={chosen['recall']:.3f} FPH={chosen['fph']:.2f} triggers={int(chosen['triggers'])}")

# 3) Sweep MIN_DUR at the chosen (q, alpha, refractory, scope)
rows_md = []
res_cache = {}
for md in CFG.MIN_DUR_SET:
    r = run_adaptive_multiband(Q_CHOSEN, A_CHOSEN, md, refract=RF_CHOSEN, adapt_scope=SCOPE_CHOSEN,
                               fuse_mode=CFG.FUSE_MODE, k_for_kofn=CFG.K_FOR_KOFN)
    rows_md.append({k:v for k,v in r.items() if k!='trigs'})
    res_cache[md] = r

scan_md = pd.DataFrame(rows_md).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nAfter MIN_DUR sweep:")
print(scan_md)

# Choose highest recall under budget (tie-breaker: lower FPH)
cands2 = scan_md[scan_md["fph"] <= CFG.FPH_BUDGET]
final = (cands2 if len(cands2) else scan_md).sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
MD_CHOSEN = float(final["min_dur"])
trigs_final = res_cache[MD_CHOSEN]["trigs"]

# 4) Save triggers CSV for cascade (encode key params in the filename)
mode_tag = f"{CFG.FUSE_MODE}{CFG.K_FOR_KOFN if CFG.FUSE_MODE=='kofn' else ''}"
csv_name = f"triggers_MB_{mode_tag}_q{Q_CHOSEN}_a{A_CHOSEN}_rf{RF_CHOSEN}_sc{SCOPE_CHOSEN}_md{MD_CHOSEN}.csv"
csv_out = os.path.join(CFG.OUT_DIR, csv_name)

if trigs_final.size == 0:
    pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    print(f"[Saved EMPTY] {csv_out}")
else:
    trig_ts = pd.to_datetime(trigs_final)
    date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
    trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
    trig_df.to_csv(csv_out, index=False)
    print(f"[Saved] {csv_out} (rows={len(trig_df)})")


[Info] events considered: 1392 on 17 days | hours=408.0

Top candidates before MIN_DUR:
        q  alpha  min_dur    recall       fph  triggers  refract adapt
0   0.994   1.00      0.0  0.772270  9.352947      3816       60  hour
1   0.994   1.01      0.0  0.764368  9.022064      3681       60  hour
2   0.994   1.00      0.0  0.761494  8.833338      3604       90  hour
3   0.994   1.00      0.0  0.756466  8.946084      3650       60   day
4   0.994   1.01      0.0  0.753592  8.553927      3490       90  hour
5   0.994   1.01      0.0  0.753592  8.639711      3525       60   day
6   0.994   1.02      0.0  0.752155  8.649515      3529       60  hour
7   0.994   1.00      0.0  0.750718  8.485299      3462       90   day
8   0.994   1.01      0.0  0.747845  8.210789      3350       90   day
9   0.994   1.00      0.0  0.746408  8.365201      3413      120  hour
10  0.994   1.02      0.0  0.742098  8.218142      3353       90  hour
11  0.994   1.02      0.0  0.742098  8.352946      3408     

In [33]:
# ============================================================
# Cascade v2: Use multiband triggers -> re-center -> cut -> z-score -> strict CNN
# - Wider re-centering window to stabilize alignment
# - Global z-score from TRAIN positives (strict comparability)
# - Optional post-softmax remapping to bump M/L recall (configurable)
# - Robust trigger CSV resolver (auto-pick latest triggers_MB_*.csv)
# ============================================================

import os, glob, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.filter import envelope
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch, torch.nn as nn, torch.nn.functional as F

# -------------------- Config --------------------
@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Either set to an existing CSV, or leave as default and resolver will auto-pick latest
    TRIG_CSV   : str = "runs/cascade_eval/triggers_MB_kofn2_q0.997_a1.02_rf120_scday_md0.0.csv"

    # Continuous waveforms (same station/day naming)
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20
    BAND_CUT   : tuple = (0.5, 8.0)  # band for cutting windows

    # CNN window (must match your training: 20s pre + 70s post)
    PRE        : int = 20
    POST       : int = 70

    # Detection-eval window (should match detection eval)
    PRE_DET    : int = 20
    POST_DET   : int = 300

    # Wider re-centering search window around the trigger (v2 change)
    RC_PRE     : int = 20   # was 10
    RC_POST    : int = 40   # was 20

    # Strict CNN checkpoint (baseline-comparable)
    BEST_PT    : str = "runs/cnn_strict/best.pt"
    OUT_DIR    : str = "runs/cascade_eval"

    # Optional: post-softmax remapping to slightly raise M/L recall (keep modest)
    REMAP_ENABLE  : bool  = False
    REMAP_M_THRES : float = 0.35
    REMAP_L_THRES : float = 0.25

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

LABEL_MAP = {"S":0, "M":1, "L":2}

# ---------- Robust trigger CSV resolver ----------
def resolve_trig_csv(cfg):
    """
    Resolve the trigger CSV to use:
      1) If cfg.TRIG_CSV is set and exists, use it.
      2) Else, auto-pick the newest 'triggers_MB_*.csv' under cfg.OUT_DIR.
      3) Else, try the legacy filename from your earlier run.
      4) Else, raise a clear error listing what's available.
    """
    if cfg.TRIG_CSV and os.path.exists(cfg.TRIG_CSV):
        print(f"[Using provided trigger CSV] {cfg.TRIG_CSV}")
        return cfg.TRIG_CSV

    pattern = os.path.join(cfg.OUT_DIR, "triggers_MB_*.csv")
    cands = sorted(glob.glob(pattern), key=lambda p: os.path.getmtime(p), reverse=True)
    if cands:
        print(f"[Auto-picked latest trigger CSV] {cands[0]}")
        return cands[0]

    legacy = os.path.join(cfg.OUT_DIR, "triggers_MB_2of3_q0.997_a1.02_md0.0.csv")
    if os.path.exists(legacy):
        print(f"[Fallback to legacy triggers] {legacy}")
        return legacy

    print(f"[ERROR] No trigger CSV found.\n"
          f"- Working dir: {os.getcwd()}\n"
          f"- Looked for: {pattern}\n"
          f"- cfg.TRIG_CSV was: {cfg.TRIG_CSV}\n"
          f"Next steps:\n"
          f"  a) Run the detector first (detect_multiband_v2.py) and note the '[Saved] ...csv' path,\n"
          f"  b) Or place a triggers CSV into {cfg.OUT_DIR}.\n")
    raise FileNotFoundError("No triggers_MB_*.csv could be found.")

# -------------------- Data helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    """Load positives from NPZ and return waveforms, labels, ids, window_starts (UTC-naive)."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """Test events = positives in TEST scope; event_time = window_start + PRE seconds."""
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, "s"))
    y_te = pd.Series(y[mte]).reset_index(drop=True)
    order = np.argsort(evt_time.values)
    df_te = pd.DataFrame({"event_time": evt_time.values[order]})
    y_te_sorted = y_te.iloc[order].to_numpy()
    return df_te.reset_index(drop=True), y_te_sorted

def train_mean_std(cfg: Cfg):
    """Compute global mean/std from TRAIN positives for strict comparability."""
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    tr_scope, _ = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mtr = np.isin(sid, list(tr_scope))
    flat = X[mtr].reshape(np.sum(mtr), -1)
    mean, std = float(flat.mean()), float(flat.std() + 1e-8)
    with open(os.path.join(cfg.OUT_DIR, "mean_std_from_train.json"), "w") as f:
        json.dump({"mean": mean, "std": std}, f, indent=2)
    return mean, std

# -------------------- Waveform cutting --------------------
_trace_cache = {}
def get_trace_for_day(cfg: Cfg, day_str: str):
    """Load, resample, detrend, and bandpass a day's trace; cache per day."""
    if day_str not in _trace_cache:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=day_str))
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        if abs(tr.stats.sampling_rate - cfg.FS) > 1e-6:
            tr.resample(cfg.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=cfg.BAND_CUT[0], freqmax=cfg.BAND_CUT[1])
        _trace_cache[day_str] = tr
    return _trace_cache[day_str]

def recenter_trigger(tr, t_ts, fs, pre, post):
    """
    Find an envelope peak near the trigger to center the CNN window.
    v2: wider coarse search (±pre/post), then a fine re-peak in ±5 s around the coarse peak.
    """
    t0 = UTCDateTime(t_ts.to_pydatetime()) - pre
    t1 = UTCDateTime(t_ts.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need:
        return t_ts  # fallback: keep original trigger
    env = envelope(x)
    i = int(np.argmax(env))
    # Fine search around the coarse peak within ±5 s (clamped to array bounds)
    fine = int(5 * fs)
    a = max(0, i - fine); b = min(len(env), i + fine + 1)
    j = a + int(np.argmax(env[a:b]))
    t_pk = t0 + j / fs
    return pd.Timestamp(UTCDateTime(t_pk).datetime)

# -------------------- Strict CNN (same as your training code) --------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k // 2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn   = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d): 
        super().__init__(); self.W = nn.Linear(d, d); self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch, 32), ConvBlock(32, 64), ConvBlock(64, 128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2 * hidden)
        self.head = nn.Sequential(nn.Linear(2 * hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x):
        z = self.cnn(x)          # [B,C,L]
        z = z.transpose(1, 2)    # [B,L,C]
        H, _ = self.lstm(z)      # [B,L,2H]
        Z, _ = self.attn(H)      # [B,2H]
        return self.head(Z)

def softmax_remap(logits, enable=False, m_th=0.35, l_th=0.25):
    """
    Optional post-softmax remapping to gently bump M/L recall.
    Only applied if `enable=True`. Keep thresholds modest to avoid harming S too much.
    """
    preds = logits.argmax(1).cpu().numpy()
    if not enable:
        return preds
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    pM, pL = probs[:,1], probs[:,2]
    for n in range(len(preds)):
        if pL[n] >= l_th:
            preds[n] = 2
        elif (pM[n] >= m_th) and (preds[n] == 0):
            preds[n] = 1
    return preds

# -------------------- 1) Build test events & load triggers --------------------
df_te, y_te_sorted = build_test_events(CFG)

trig_csv_path = resolve_trig_csv(CFG)  # auto-pick if not present
trig_df = pd.read_csv(trig_csv_path)
if "trigger_time" not in trig_df.columns:
    raise ValueError(f"'trigger_time' column not found in {trig_csv_path}")
trig_df["trigger_time"] = pd.to_datetime(trig_df["trigger_time"], errors="coerce", utc=False)
trig_df = trig_df.sort_values("trigger_time").reset_index(drop=True)

# detection-eval windows (match POST_DET=300)
ev = df_te.copy()
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")

# safer dtype conversion
tr       = trig_df["trigger_time"].values.astype("datetime64[ns]")
ev_start = ev["start"].values.astype("datetime64[ns]")
ev_end   = ev["end"].values.astype("datetime64[ns]")

# Vectorized hit test
i = np.searchsorted(tr, ev_start, side="left")
j = np.searchsorted(tr, ev_end,   side="right")
hit = (j - i) > 0
first_idx = np.where(hit, i, -1)

print(f"[Info] events={len(ev)} | hits={int(hit.sum())} | hit_rate={hit.mean():.3f}")


# -------------------- 2) Re-center & cut CNN windows for hits --------------------
win_len = CFG.FS * (CFG.PRE + CFG.POST)
X_cut, y_hit, kept = [], [], []
for idx in np.where(hit)[0]:
    t_first_ns = tr[first_idx[idx]]
    t_first = pd.Timestamp(t_first_ns)
    day_str = str(t_first.date())
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=day_str))
    if not os.path.exists(fp): 
        continue
    tr_day = get_trace_for_day(CFG, day_str)
    t_center = recenter_trigger(tr_day, t_first, fs=CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)
    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr_day.slice(t0, t1).data
    if len(x) >= win_len:
        X_cut.append(x[:win_len].astype(np.float32))
        y_hit.append(y_te_sorted[idx])
        kept.append(idx)

if len(X_cut) == 0:
    raise SystemExit("No windows cut; check filenames or time bounds.")

X_cut = np.stack(X_cut, axis=0)
y_hit = np.array(y_hit, dtype=int)
print(f"[Cut] windows={len(X_cut)} (of {int(hit.sum())} hits) | shape={X_cut.shape}")

# -------------------- 3) Global z-score (TRAIN-only mean/std) --------------------
mean, std = train_mean_std(CFG)
Xn = (X_cut - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]  # [N,1,T]

# -------------------- 4) Load strict CNN and infer --------------------
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model = CNNBiLSTMAttn(in_ch=1, n_classes=3).to(device)
state = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state, strict=True)
model.eval()

y_pred = []
with torch.no_grad():
    for i0 in range(0, len(Xn), 512):
        xb = torch.from_numpy(Xn[i0:i0+512]).to(device)
        logits = model(xb)
        preds = softmax_remap(logits, enable=CFG.REMAP_ENABLE, m_th=CFG.REMAP_M_THRES, l_th=CFG.REMAP_L_THRES)
        y_pred.extend(preds)
y_pred = np.array(y_pred, dtype=int)

# -------------------- 5) Reports: within-detected & end-to-end recall --------------------
rep = classification_report(y_hit, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0)
cm = confusion_matrix(y_hit, y_pred, labels=[0,1,2])
macro_f1 = f1_score(y_hit, y_pred, average="macro")

print("\n== CNN (cascade on detected events) ==")
print(rep)
print("Confusion matrix:\n", cm)

# End-to-end per-class recall (missed detections count as errors)
lab = y_te_sorted
correct_mask = np.zeros_like(lab, dtype=bool)
kept_idx = np.array(kept, dtype=int)

# Mark correct classifications among detected events
for k, idx in enumerate(kept_idx):
    if y_hit[k] == y_pred[k]:
        correct_mask[idx] = True

for c, name in enumerate(["S","M","L"]):
    total_c = (lab == c).sum()
    e2e_c = correct_mask[lab == c].sum() / max(1, total_c)
    print(f"[End-to-End] {name} recall = {e2e_c:.3f} (total {total_c})")

# Save artifacts
with open(os.path.join(CFG.OUT_DIR, "cascade_v2_report.txt"), "w") as f:
    f.write(rep + "\n")
    f.write("Confusion matrix:\n" + np.array2string(cm))

np.savez(os.path.join(CFG.OUT_DIR, "cascade_v2_preds.npz"),
         y_true=y_hit, y_pred=y_pred, kept_event_indices=np.array(kept, dtype=int))

print(f"[Saved] {os.path.join(CFG.OUT_DIR, 'cascade_v2_report.txt')}")


[Auto-picked latest trigger CSV] runs/cascade_eval/triggers_MB_kofn2_q0.995_a1.05_rf120_scday_md0.0.csv
[Info] events=1392 | hits=896 | hit_rate=0.644
[Cut] windows=896 (of 896 hits) | shape=(896, 1800)

== CNN (cascade on detected events) ==
              precision    recall  f1-score   support

           S     0.9495    0.9261    0.9377       812
           M     0.3737    0.4684    0.4157        79
           L     0.0000    0.0000    0.0000         5

    accuracy                         0.8806       896
   macro avg     0.4411    0.4648    0.4511       896
weighted avg     0.8934    0.8806    0.8864       896

Confusion matrix:
 [[752  57   3]
 [ 40  37   2]
 [  0   5   0]]
[End-to-End] S recall = 0.589 (total 1277)
[End-to-End] M recall = 0.359 (total 103)
[End-to-End] L recall = 0.000 (total 12)
[Saved] runs/cascade_eval/cascade_v2_report.txt


In [35]:
# ============================================================
# Multiband STA/LTA Detection v2 (OR+NMS + Hour-level thresholds)
# - Rebuild TEST events from NPZ + frozen_splits
# - Build daily CFTs across 4 bands (incl. 0.1–1.0 Hz for far/small events)
# - Grid-search (q, alpha, refractory, adapt_scope) under FPH budget
# - Fusion: OR of per-band ON-starts -> NMS -> refractory (simple & recall-friendly)
# - Sweep MIN_DUR (kept for completeness; not used in OR mode)
# - Save final triggers CSV for cascade
# Notes:
# * Compatible with ObsPy versions where trigger_onset may return list.
# * Keeps FPH_BUDGET=6/h as your constraint.
# ============================================================

import os
import json
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"   # e.g., MAJO_2011-03-05.mseed
    FS         : int = 20

    # Bands (v2+: add a looser low band for far/small events)
    BANDS      : tuple = ((0.1, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows (seconds)
    STA        : float = 1.5
    LTA        : float = 20.0

    # Hysteresis OFF threshold
    OFF        : float = 1.0

    # Default refractory (also grid-searched)
    REFRACT    : int   = 120

    # Detection-eval window (NOT used for cutting CNN windows)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # ---------------- Recall-friendly grid & budgets ----------------
    # Lower q / allow alpha<=1; try short refractory; prefer hour-level quantiles
    GRID_Q       : tuple = (0.992, 0.993, 0.994, 0.995, 0.996)
    GRID_ALPHA   : tuple = (0.98, 1.00, 1.01, 1.02)
    GRID_REFRACT : tuple = (60, 90, 120)
    ADAPT_SCOPES : tuple = ("hour", "day")   # try hour first; day as fallback
    FPH_BUDGET   : float = 6.0

    # MIN_DUR kept for completeness (affects K-of-N only; OR+NMS ignores it)
    MIN_DUR_SET  : tuple = (0.0, 0.5, 1.0)

    # ---------------- Fusion (set to OR+NMS) ----------------
    FUSE_MODE    : str  = "or_nms"
    NMS_SEC      : int  = 30   # tighter NMS to admit more weak picks under budget
    K_FOR_KOFN   : int  = 2    # unused when FUSE_MODE="or_nms"

    OUT_DIR      : str  = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

# -------------------- NPZ helpers --------------------
def _first_key(d, candidates):
    """Return the first existing key in NPZ among candidates."""
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    """Load positives from NPZ and return waveforms, labels, ids, window_starts (UTC-naive)."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    """Return train/test scopes as sets of sample_id strings."""
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))  # <-- FIXED here
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope


def build_test_events(cfg: Cfg):
    """
    Test events = positives within TEST scope.
    Event time (catalog-origin) = window_start + 20s (classification dataset convention).
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- CFT build --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """
    Read one MiniSEED file; resample & detrend; then for each band:
    - bandpass
    - compute STA/LTA characteristic function (CFT)
    Returns: dict(fs, t0, hours, cft_list=[cft_band1, ...])
    """
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)

    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)

    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

# -------------------- Normalizers & fusion helpers --------------------
def to_onoff_array(x):
    """
    Normalize trigger_onset output to a (N,2) int ndarray.
    ObsPy may return a list on some versions; this makes it consistent.
    """
    arr = np.asarray(x, dtype=int)
    if arr.size == 0:
        return np.empty((0, 2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0:
            raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    """
    OR fusion (union of all band ON starts) followed by:
      - NMS-like suppression with min spacing `nms_sec`
      - final refractory thinning with `refract`
    Returns a list of pandas.Timestamp.
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0:
            continue
        for a, b in arr:
            picks.append(t0 + a / fs)  # UTCDateTime
    if not picks:
        return []
    picks.sort()
    # NMS
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    # Refractory
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def filter_min_dur(onoff, fs, min_dur_s):
    """Only used in K-of-N mode; kept for completeness."""
    if onoff.size == 0 or min_dur_s <= 0:
        return onoff
    keep = ((onoff[:,1] - onoff[:,0]) / fs) >= float(min_dur_s)
    return onoff[keep]

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Return boolean hits for each event window; a hit if any trigger ∈ [start, end]."""
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Main detection build & scan --------------------
# 1) Test events and available days
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in event_days:
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    cftd = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

if not dates_ok:
    raise SystemExit("No overlapping days between df_te and waveform files.")

# Detection-eval windows
ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

print(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

def run_adaptive_multiband(q, alpha, min_dur_s=0.0, refract=None, adapt_scope="hour"):
    """
    For each day:
      - per-band ON threshold via quantile(cft, q) * alpha
        * adapt_scope = 'hour' -> per-hour quantiles (robust to drift)
        * adapt_scope = 'day'  -> whole-day quantile (fallback)
      - per-band on/off = trigger_onset(cft, ON, OFF)  (normalized to ndarray)
      - OR fusion of ON starts -> NMS -> refractory to get picks
    Aggregate picks across days, then compute recall/FPH.
    """
    refract = CFG.REFRACT if refract is None else int(refract)
    all_picks = []
    for dt, entry in cft_cache.items():
        onoffs = []
        for cft in entry["cft_list"]:
            if adapt_scope == "hour":
                fs = entry["fs"]; H = int(3600 * fs)
                segs = []
                for s in range(0, len(cft), H):
                    e = min(s + H, len(cft))
                    seg = cft[s:e]
                    if seg.size == 0:
                        _onoff = np.empty((0, 2), dtype=int)
                    else:
                        on_val = float(np.quantile(seg, q) * alpha)
                        raw = trigger_onset(seg, on_val, CFG.OFF)
                        _onoff = to_onoff_array(raw)
                        if _onoff.size:
                            _onoff[:,0] += s; _onoff[:,1] += s
                    segs.append(_onoff)
                onoff = np.vstack(segs) if len(segs) else np.empty((0,2), dtype=int)
            elif adapt_scope == "day":
                if cft.size == 0:
                    onoff = np.empty((0,2), dtype=int)
                else:
                    on_val = float(np.quantile(cft, q) * alpha)
                    raw = trigger_onset(cft, on_val, CFG.OFF)
                    onoff = to_onoff_array(raw)
            else:
                raise ValueError("adapt_scope must be 'hour' or 'day'")
            onoffs.append(onoff)

        # OR+NMS fusion (recall-friendly)
        picks = picks_or_then_nms(onoffs, entry["fs"], entry["t0"], nms_sec=CFG.NMS_SEC, refract=refract)
        all_picks.extend(picks)

    trigs_ns = np.array(sorted(all_picks), dtype="datetime64[ns]")
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    fph = trigs_ns.size / max(1e-6, total_hours)
    return dict(q=q, alpha=alpha, min_dur=min_dur_s, recall=recall, fph=fph,
                triggers=int(trigs_ns.size), trigs=trigs_ns, refract=refract, adapt=adapt_scope)

# 2) Grid search (q, alpha, refractory, adapt_scope) under FPH budget
rows = []
for scope in CFG.ADAPT_SCOPES:
    for q in CFG.GRID_Q:
        for a in CFG.GRID_ALPHA:
            for rf in CFG.GRID_REFRACT:
                rows.append(run_adaptive_multiband(q, a, 0.0, refract=rf, adapt_scope=scope))

scan = pd.DataFrame([{k:v for k,v in r.items() if k!='trigs'} for r in rows]) \
          .sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nTop candidates (pre MIN_DUR):")
print(scan.head(12))

# pick best recall under budget (tie-breaker: lower FPH)
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
chosen = (cands if len(cands) else scan).sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
Q_CHOSEN, A_CHOSEN = float(chosen["q"]), float(chosen["alpha"])
RF_CHOSEN, SCOPE_CHOSEN = int(chosen["refract"]), str(chosen["adapt"])
print(f"\n[Chosen] q={Q_CHOSEN} alpha={A_CHOSEN} refract={RF_CHOSEN} scope={SCOPE_CHOSEN} "
      f"| recall={chosen['recall']:.3f} FPH={chosen['fph']:.2f} triggers={int(chosen['triggers'])}")

# 3) MIN_DUR sweep (no effect in OR mode; keep for API parity)
rows_md, res_cache = [], {}
for md in CFG.MIN_DUR_SET:
    r = run_adaptive_multiband(Q_CHOSEN, A_CHOSEN, md, refract=RF_CHOSEN, adapt_scope=SCOPE_CHOSEN)
    rows_md.append({k:v for k,v in r.items() if k!='trigs'})
    res_cache[md] = r

scan_md = pd.DataFrame(rows_md).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
print("\nAfter MIN_DUR sweep:")
print(scan_md)

# choose highest recall under budget (tie-breaker: lower FPH)
cands2 = scan_md[scan_md["fph"] <= CFG.FPH_BUDGET]
final = (cands2 if len(cands2) else scan_md).sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
MD_CHOSEN = float(final["min_dur"])
trigs_final = res_cache[MD_CHOSEN]["trigs"]

# 4) Save triggers CSV for cascade (encode key params in filename)
mode_tag = "ornms"
csv_name = f"triggers_MB_{mode_tag}_q{Q_CHOSEN}_a{A_CHOSEN}_rf{RF_CHOSEN}_sc{SCOPE_CHOSEN}_md{MD_CHOSEN}.csv"
csv_out = os.path.join(CFG.OUT_DIR, csv_name)

if trigs_final.size == 0:
    pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    print(f"[Saved EMPTY] {csv_out}")
else:
    trig_ts = pd.to_datetime(trigs_final)
    date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
    trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
    trig_df.to_csv(csv_out, index=False)
    print(f"[Saved] {csv_out} (rows={len(trig_df)})")


[Info] events considered: 1392 on 17 days | hours=408.0

Top candidates (pre MIN_DUR):
        q  alpha  min_dur    recall        fph  triggers  refract adapt
0   0.992   0.98      0.0  0.995690  32.176489     13128       60   day
1   0.992   0.98      0.0  0.994971  32.845607     13401       60  hour
2   0.992   1.00      0.0  0.994253  31.262273     12755       60   day
3   0.992   1.00      0.0  0.994253  31.855411     12997       60  hour
4   0.992   0.98      0.0  0.993534  25.426485     10374       90   day
5   0.992   1.00      0.0  0.992816  25.169132     10269       90  hour
6   0.992   0.98      0.0  0.992816  25.781878     10519       90  hour
7   0.992   1.02      0.0  0.992098  30.191194     12318       60   day
8   0.993   0.98      0.0  0.992098  30.661782     12510       60   day
9   0.992   1.01      0.0  0.992098  30.696096     12524       60   day
10  0.992   1.01      0.0  0.992098  31.291685     12767       60  hour
11  0.992   1.00      0.0  0.991379  24.894622   

In [36]:
# ============================================================
# Cascade v2: Use multiband triggers -> re-center -> cut -> z-score -> strict CNN
# - Wider re-centering window to stabilize alignment
# - Global z-score from TRAIN positives (strict comparability)
# - Optional post-softmax remapping (disabled by default)
# - Robust trigger CSV resolver (auto-pick latest triggers_MB_*.csv)
# ============================================================

import os, glob, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.filter import envelope
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch, torch.nn as nn, torch.nn.functional as F

# -------------------- Config --------------------
@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Leave as-is; resolver will pick the newest 'triggers_MB_*.csv' in OUT_DIR
    TRIG_CSV   : str = "runs/cascade_eval/triggers_MB_ornms_q0.995_a1.02_rf90_schour_md0.0.csv"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20
    BAND_CUT   : tuple = (0.5, 8.0)  # band for cutting windows

    # CNN window (must match training)
    PRE        : int = 20
    POST       : int = 70

    # Detection-eval window (should match detection eval)
    PRE_DET    : int = 20
    POST_DET   : int = 300

    # Wider re-centering window
    RC_PRE     : int = 20
    RC_POST    : int = 40

    # Strict CNN checkpoint
    BEST_PT    : str = "runs/cnn_strict/best.pt"
    OUT_DIR    : str = "runs/cascade_eval"

    # Optional: post-softmax remapping (disabled by default)
    REMAP_ENABLE  : bool  = False
    REMAP_M_THRES : float = 0.33
    REMAP_L_THRES : float = 0.20

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

LABEL_MAP = {"S":0, "M":1, "L":2}

# ---------- Robust trigger CSV resolver ----------
def resolve_trig_csv(cfg):
    """
    Resolve the trigger CSV to use:
      1) If cfg.TRIG_CSV exists, use it.
      2) Else, pick the newest 'triggers_MB_*.csv' under cfg.OUT_DIR.
      3) Else, try legacy filename from earlier v1.
      4) Else, raise a clear error.
    """
    if cfg.TRIG_CSV and os.path.exists(cfg.TRIG_CSV):
        print(f"[Using provided trigger CSV] {cfg.TRIG_CSV}")
        return cfg.TRIG_CSV

    pattern = os.path.join(cfg.OUT_DIR, "triggers_MB_*.csv")
    cands = sorted(glob.glob(pattern), key=lambda p: os.path.getmtime(p), reverse=True)
    if cands:
        print(f"[Auto-picked latest trigger CSV] {cands[0]}")
        return cands[0]

    legacy = os.path.join(cfg.OUT_DIR, "triggers_MB_2of3_q0.997_a1.02_md0.0.csv")
    if os.path.exists(legacy):
        print(f"[Fallback to legacy triggers] {legacy}")
        return legacy

    print(f"[ERROR] No trigger CSV found.\n"
          f"- Working dir: {os.getcwd()}\n"
          f"- Looked for: {pattern}\n"
          f"- cfg.TRIG_CSV was: {cfg.TRIG_CSV}\n"
          f"Next steps:\n"
          f"  a) Run the detector (detect_multiband_v2.py) and note the '[Saved] ...csv' path,\n"
          f"  b) Or place a triggers CSV into {cfg.OUT_DIR}.\n")
    raise FileNotFoundError("No triggers_MB_*.csv could be found.")

# -------------------- Data helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, "s"))
    y_te = pd.Series(y[mte]).reset_index(drop=True)
    order = np.argsort(evt_time.values)
    df_te = pd.DataFrame({"event_time": evt_time.values[order]})
    y_te_sorted = y_te.iloc[order].to_numpy()
    return df_te.reset_index(drop=True), y_te_sorted

def train_mean_std(cfg: Cfg):
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    tr_scope, _ = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mtr = np.isin(sid, list(tr_scope))
    flat = X[mtr].reshape(np.sum(mtr), -1)
    mean, std = float(flat.mean()), float(flat.std() + 1e-8)
    with open(os.path.join(cfg.OUT_DIR, "mean_std_from_train.json"), "w") as f:
        json.dump({"mean": mean, "std": std}, f, indent=2)
    return mean, std

# -------------------- Waveform IO  --------------------
_trace_cache = {}
def get_trace_for_day(cfg: Cfg, day_str: str):
    if day_str not in _trace_cache:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=day_str))
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        if abs(tr.stats.sampling_rate - cfg.FS) > 1e-6:
            tr.resample(cfg.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=cfg.BAND_CUT[0], freqmax=cfg.BAND_CUT[1])
        _trace_cache[day_str] = tr
    return _trace_cache[day_str]

def recenter_trigger(tr, t_ts, fs, pre, post):
    """
    Find an envelope peak near the trigger to center the CNN window.
    Wider coarse search (±pre/post), plus a fine re-peak in ±5 s window.
    """
    t0 = UTCDateTime(t_ts.to_pydatetime()) - pre
    t1 = UTCDateTime(t_ts.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need:
        return t_ts
    env = envelope(x)
    i = int(np.argmax(env))
    fine = int(5 * fs)
    a = max(0, i - fine); b = min(len(env), i + fine + 1)
    j = a + int(np.argmax(env[a:b]))
    t_pk = t0 + j / fs
    return pd.Timestamp(UTCDateTime(t_pk).datetime)

# -------------------- Strict CNN --------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k // 2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn   = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d): 
        super().__init__(); self.W = nn.Linear(d, d); self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch, 32), ConvBlock(32, 64), ConvBlock(64, 128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2 * hidden)
        self.head = nn.Sequential(nn.Linear(2 * hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x):
        z = self.cnn(x)          # [B,C,L]
        z = z.transpose(1, 2)    # [B,L,C]
        H, _ = self.lstm(z)      # [B,L,2H]
        Z, _ = self.attn(H)      # [B,2H]
        return self.head(Z)

def softmax_remap(logits, enable=False, m_th=0.33, l_th=0.20):
    preds = logits.argmax(1).cpu().numpy()
    if not enable:
        return preds
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    pM, pL = probs[:,1], probs[:,2]
    for n in range(len(preds)):
        if pL[n] >= l_th:
            preds[n] = 2
        elif (pM[n] >= m_th) and (preds[n] == 0):
            preds[n] = 1
    return preds

# -------------------- 1) Build test events & load triggers --------------------
df_te, y_te_sorted = build_test_events(CFG)

trig_csv_path = resolve_trig_csv(CFG)  # auto-pick if provided path missing
trig_df = pd.read_csv(trig_csv_path)
if "trigger_time" not in trig_df.columns:
    raise ValueError(f"'trigger_time' column not found in {trig_csv_path}")
trig_df["trigger_time"] = pd.to_datetime(trig_df["trigger_time"], errors="coerce", utc=False)
trig_df = trig_df.sort_values("trigger_time").reset_index(drop=True)

# detection-eval windows (match POST_DET=300)
ev = df_te.copy()
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")

# safe dtype conversion
tr       = trig_df["trigger_time"].values.astype("datetime64[ns]")
ev_start = ev["start"].values.astype("datetime64[ns]")
ev_end   = ev["end"].values.astype("datetime64[ns]")

# Vectorized hit test
i = np.searchsorted(tr, ev_start, side="left")
j = np.searchsorted(tr, ev_end,   side="right")
hit = (j - i) > 0
first_idx = np.where(hit, i, -1)

print(f"[Info] events={len(ev)} | hits={int(hit.sum())} | hit_rate={hit.mean():.3f}")

# -------------------- 2) Re-center & cut CNN windows for hits --------------------
win_len = CFG.FS * (CFG.PRE + CFG.POST)
X_cut, y_hit, kept = [], [], []
for idx in np.where(hit)[0]:
    t_first_ns = tr[first_idx[idx]]
    t_first = pd.Timestamp(t_first_ns)
    day_str = str(t_first.date())
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=day_str))
    if not os.path.exists(fp): 
        continue
    tr_day = get_trace_for_day(CFG, day_str)
    t_center = recenter_trigger(tr_day, t_first, fs=CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)
    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr_day.slice(t0, t1).data
    if len(x) >= win_len:
        X_cut.append(x[:win_len].astype(np.float32))
        y_hit.append(y_te_sorted[idx])
        kept.append(idx)

if len(X_cut) == 0:
    raise SystemExit("No windows cut; check filenames or time bounds.")

X_cut = np.stack(X_cut, axis=0)
y_hit = np.array(y_hit, dtype=int)
print(f"[Cut] windows={len(X_cut)} (of {int(hit.sum())} hits) | shape={X_cut.shape}")

# -------------------- 3) Global z-score (TRAIN-only mean/std) --------------------
mean, std = train_mean_std(CFG)
Xn = (X_cut - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]  # [N,1,T]

# -------------------- 4) Load strict CNN and infer --------------------
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model = CNNBiLSTMAttn(in_ch=1, n_classes=3).to(device)
state = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state, strict=True)
model.eval()

y_pred = []
with torch.no_grad():
    for i0 in range(0, len(Xn), 512):
        xb = torch.from_numpy(Xn[i0:i0+512]).to(device)
        logits = model(xb)
        preds = softmax_remap(logits, enable=CFG.REMAP_ENABLE, m_th=CFG.REMAP_M_THRES, l_th=CFG.REMAP_L_THRES)
        y_pred.extend(preds)
y_pred = np.array(y_pred, dtype=int)

# -------------------- 5) Reports --------------------
rep = classification_report(y_hit, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0)
cm = confusion_matrix(y_hit, y_pred, labels=[0,1,2])
macro_f1 = f1_score(y_hit, y_pred, average="macro")

print("\n== CNN (cascade on detected events) ==")
print(rep)
print("Confusion matrix:\n", cm)

# End-to-end per-class recall
lab = y_te_sorted
correct_mask = np.zeros_like(lab, dtype=bool)
kept_idx = np.array(kept, dtype=int)
for k, idx in enumerate(kept_idx):
    if y_hit[k] == y_pred[k]:
        correct_mask[idx] = True

for c, name in enumerate(["S","M","L"]):
    total_c = (lab == c).sum()
    e2e_c = correct_mask[lab == c].sum() / max(1, total_c)
    print(f"[End-to-End] {name} recall = {e2e_c:.3f} (total {total_c})")

# Save artifacts
with open(os.path.join(CFG.OUT_DIR, "cascade_v2_report.txt"), "w") as f:
    f.write(rep + "\n")
    f.write("Confusion matrix:\n" + np.array2string(cm))

np.savez(os.path.join(CFG.OUT_DIR, "cascade_v2_preds.npz"),
         y_true=y_hit, y_pred=y_pred, kept_event_indices=np.array(kept, dtype=int))

print(f"[Saved] {os.path.join(CFG.OUT_DIR, 'cascade_v2_report.txt')}")


[Auto-picked latest trigger CSV] runs/cascade_eval/triggers_MB_ornms_q0.992_a0.98_rf60_scday_md0.0.csv
[Info] events=1392 | hits=1386 | hit_rate=0.996
[Cut] windows=1386 (of 1386 hits) | shape=(1386, 1800)

== CNN (cascade on detected events) ==
              precision    recall  f1-score   support

           S     0.9620    0.9552    0.9586      1272
           M     0.4530    0.5196    0.4840       102
           L     0.5000    0.2500    0.3333        12

    accuracy                         0.9170      1386
   macro avg     0.6383    0.5749    0.5920      1386
weighted avg     0.9205    0.9170    0.9182      1386

Confusion matrix:
 [[1215   55    2]
 [  48   53    1]
 [   0    9    3]]
[End-to-End] S recall = 0.951 (total 1277)
[End-to-End] M recall = 0.515 (total 103)
[End-to-End] L recall = 0.250 (total 12)
[Saved] runs/cascade_eval/cascade_v2_report.txt


In [37]:
# ============================================================
# FPH Checker for Multiband STA/LTA Triggers
# - Rebuild the exact day set used by detection:
#   days = (unique test event dates) ∩ (dates with available mseed files)
# - Read triggers CSV (auto-pick latest if not provided), filter to those days
# - Compute total_hours from mseed coverage and FPH = triggers / total_hours
# - Print overall summary and per-day breakdown
# ============================================================

import os, glob, json
from dataclasses import dataclass
import numpy as np
import pandas as pd
from obspy import read

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts (same as detector)
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"   # e.g., MAJO_2011-03-05.mseed

    # Optional: specify a concrete triggers CSV; if missing, auto-pick the latest triggers_MB_*.csv in OUT_DIR
    TRIG_CSV   : str = ""   # e.g., "runs/cascade_eval/triggers_MB_ornms_q0.992_a0.98_rf60_scday_md0.0.csv"

    # Where trigger CSVs live
    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()

# -------------------- NPZ helpers (same logic as your detector) --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]  # not used here, but keep consistent
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_event_days(cfg: Cfg):
    """Return sorted unique dates of TEST positives (window_start + 20s by convention)."""
    _, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    return sorted(set(pd.to_datetime(evt_time).dt.date))

# -------------------- Trigger CSV resolver --------------------
def resolve_trig_csv(cfg: Cfg):
    if cfg.TRIG_CSV and os.path.exists(cfg.TRIG_CSV):
        print(f"[Using provided trigger CSV] {cfg.TRIG_CSV}")
        return cfg.TRIG_CSV
    pattern = os.path.join(cfg.OUT_DIR, "triggers_MB_*.csv")
    cands = sorted(glob.glob(pattern), key=lambda p: os.path.getmtime(p), reverse=True)
    if not cands:
        raise FileNotFoundError(f"No trigger CSV found in {cfg.OUT_DIR} matching 'triggers_MB_*.csv'.")
    print(f"[Auto-picked latest trigger CSV] {cands[0]}")
    return cands[0]

# -------------------- Hours calculator --------------------
def compute_hours_for_days(cfg: Cfg, days):
    """
    For each day in `days`, if the corresponding mseed exists,
    read it and compute coverage hours as (endtime - starttime)/3600.
    Return: (per_day_hours: dict[date->hours], total_hours: float, days_ok: list[date])
    """
    per_day_hours = {}
    days_ok = []
    total_hours = 0.0
    for dt in days:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=dt))
        if not os.path.exists(fp):
            continue
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
        per_day_hours[dt] = hours
        total_hours += hours
        days_ok.append(dt)
    return per_day_hours, total_hours, days_ok

# -------------------- FPH checker --------------------
def check_fph(cfg: Cfg):
    # 1) Day set used in detection (events ∩ available mseed days)
    event_days = build_test_event_days(cfg)
    per_day_hours, total_hours, days_ok = compute_hours_for_days(cfg, event_days)
    if not days_ok:
        raise SystemExit("No overlapping days between TEST events and waveform files.")

    # 2) Load triggers and keep only those on days_ok (consistent denominator)
    trig_csv = resolve_trig_csv(cfg)
    df = pd.read_csv(trig_csv)
    if "trigger_time" not in df.columns:
        raise ValueError(f"'trigger_time' column not found in {trig_csv}")
    df["trigger_time"] = pd.to_datetime(df["trigger_time"], errors="coerce", utc=False)
    df = df.dropna(subset=["trigger_time"]).sort_values("trigger_time").reset_index(drop=True)
    df["date"] = df["trigger_time"].dt.date

    n_total_trigs = len(df)
    df = df[df["date"].isin(days_ok)].copy()
    n_used_trigs = len(df)
    n_dropped = n_total_trigs - n_used_trigs
    if n_dropped > 0:
        print(f"[Note] Dropped {n_dropped} triggers outside the detection day set (kept {n_used_trigs}).")

    # 3) Overall FPH
    if total_hours <= 0:
        raise ValueError("Total hours computed as zero; check your mseed files.")
    fph = n_used_trigs / total_hours

    # 4) Per-day breakdown
    per_day = df.groupby("date")["trigger_time"].count().rename("triggers").reset_index()
    per_day["hours"] = per_day["date"].map(per_day_hours).astype(float)
    per_day["fph"] = per_day["triggers"] / per_day["hours"].replace(0, np.nan)

    # 5) Print summary
    print("\n================= FPH SUMMARY =================")
    print(f"Triggers CSV: {trig_csv}")
    print(f"Days considered: {len(days_ok)}  |  Total hours: {total_hours:.3f}")
    print(f"Total triggers (used): {n_used_trigs}  |  Overall FPH: {fph:.3f} per hour")
    if n_dropped > 0:
        print(f"(* {n_dropped} triggers were outside considered days and excluded from FPH.)")
    print("===============================================")

    # 6) Show top-10 highest-FPH days (if you want to spot outliers)
    if len(per_day) > 0:
        per_day_sorted = per_day.sort_values("fph", ascending=False)
        print("\nTop-10 days by FPH:")
        print(per_day_sorted.head(10).to_string(index=False))
    else:
        print("No triggers within the considered day set; FPH = 0.")

    return dict(fph=fph, total_hours=total_hours, n_trigs=n_used_trigs, per_day=per_day)

# -------------------- Run --------------------
if __name__ == "__main__":
    _ = check_fph(CFG)


[Auto-picked latest trigger CSV] runs/cascade_eval/triggers_MB_ornms_q0.992_a0.98_rf60_scday_md0.0.csv

================= FPH SUMMARY =================
Triggers CSV: runs/cascade_eval/triggers_MB_ornms_q0.992_a0.98_rf60_scday_md0.0.csv
Days considered: 17  |  Total hours: 408.000
Total triggers (used): 13128  |  Overall FPH: 32.176 per hour

Top-10 days by FPH:
      date  triggers     hours       fph
2011-03-01       939 23.999986 39.125023
2011-03-03       938 23.999986 39.083356
2011-03-04       936 23.999986 39.000023
2011-03-07       914 23.999986 38.083355
2011-03-05       911 23.999986 37.958355
2011-03-10       887 23.999986 36.958355
2011-03-09       828 23.999986 34.500020
2011-03-19       738 23.999986 30.750018
2011-03-17       733 23.999986 30.541684
2011-03-18       714 23.999986 29.750017


In [38]:
# ============================================================
# Multiband STA/LTA Detection v2 — OR+NMS with FPH-aware search
# - Adds GRID_NMS_SEC (NMS window) to the search
# - Picks best recall UNDER the FPH budget; if none meet it,
#   falls back to the lowest-FPH candidate and prints a warning
# - Keeps hour-level quantiles and loose low band (0.1–1.0 Hz)
# ============================================================

import os
import json
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20

    # Bands (loose low band helps far/small events)
    BANDS      : tuple = ((0.1, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows (seconds)
    STA        : float = 1.5
    LTA        : float = 20.0

    # Hysteresis OFF threshold
    OFF        : float = 1.0

    # Detection-eval window (NOT used for cutting CNN windows)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # ---------------- Recall vs budget search space ----------------
    # Slightly stricter than your 0.992/0.98/60s combo
    GRID_Q        : tuple = (0.993, 0.994, 0.995, 0.996)
    GRID_ALPHA    : tuple = (1.00, 1.01, 1.02)
    GRID_REFRACT  : tuple = (90, 120, 180)
    GRID_NMS_SEC  : tuple = (60, 90, 120, 150, 180, 210)  # NEW: search NMS window
    ADAPT_SCOPES  : tuple = ("hour", "day")               # prefer hour-level
    FPH_BUDGET    : float = 6.0

    # Fusion fixed to OR+NMS for recall
    FUSE_MODE     : str  = "or_nms"

    OUT_DIR       : str  = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

# -------------------- NPZ helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))  # FIXED: map(str, ...)
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    Test events = positives within TEST scope.
    Event time (catalog-origin) = window_start + 20s (classification dataset convention).
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- CFT build --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """
    Read one MiniSEED; resample & detrend; for each band:
      bandpass -> classic_sta_lta -> CFT
    Returns: dict(fs, t0, hours, cft_list=[...])
    """
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)

    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)

    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

# -------------------- Helpers --------------------
def to_onoff_array(x):
    """Normalize trigger_onset output to a (N,2) int ndarray (handles list/array)."""
    arr = np.asarray(x, dtype=int)
    if arr.size == 0:
        return np.empty((0, 2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0:
            raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    """
    OR fusion (union of all band ON starts) -> NMS (min spacing) -> refractory.
    Returns a list of pandas.Timestamp.
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0:
            continue
        for a, b in arr:
            picks.append(t0 + a / fs)  # UTCDateTime
    if not picks:
        return []
    picks.sort()
    # NMS
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    # Refractory
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Return boolean hits for each event window; hit if any trigger ∈ [start, end]."""
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build day cache & eval windows --------------------
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

cft_cache, total_hours, dates_ok = {}, 0.0, []
for dt in event_days:
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    cftd = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    cft_cache[dt] = cftd
    total_hours += cftd["hours"]
    dates_ok.append(dt)

if not dates_ok:
    raise SystemExit("No overlapping days between TEST events and waveform files.")

ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

print(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

# -------------------- Core run (OR+NMS) --------------------
def run_adaptive_or_nms(q, alpha, refract, nms_sec, adapt_scope):
    """
    For each day:
      - Threshold per band via quantile * alpha
      - If adapt_scope == 'hour', compute per-hour thresholds; else per-day
      - trigger_onset -> normalize -> collect ON intervals
      - OR+NMS fusion to picks with (nms_sec, refract)
    Return metrics dict.
    """
    all_picks = []
    for dt, entry in cft_cache.items():
        onoffs = []
        for cft in entry["cft_list"]:
            if adapt_scope == "hour":
                fs = entry["fs"]; H = int(3600 * fs)
                segs = []
                for s in range(0, len(cft), H):
                    e = min(s + H, len(cft))
                    seg = cft[s:e]
                    if seg.size == 0:
                        _onoff = np.empty((0, 2), dtype=int)
                    else:
                        on_val = float(np.quantile(seg, q) * alpha)
                        raw = trigger_onset(seg, on_val, CFG.OFF)
                        _onoff = to_onoff_array(raw)
                        if _onoff.size:
                            _onoff[:, 0] += s; _onoff[:, 1] += s
                    segs.append(_onoff)
                onoff = np.vstack(segs) if len(segs) else np.empty((0,2), dtype=int)
            elif adapt_scope == "day":
                if cft.size == 0:
                    onoff = np.empty((0,2), dtype=int)
                else:
                    on_val = float(np.quantile(cft, q) * alpha)
                    raw = trigger_onset(cft, on_val, CFG.OFF)
                    onoff = to_onoff_array(raw)
            else:
                raise ValueError("adapt_scope must be 'hour' or 'day'")
            onoffs.append(onoff)

        picks = picks_or_then_nms(onoffs, entry["fs"], entry["t0"], nms_sec=nms_sec, refract=refract)
        all_picks.extend(picks)

    trigs_ns = np.array(sorted(all_picks), dtype="datetime64[ns]")
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    fph = trigs_ns.size / max(1e-6, total_hours)
    return dict(q=q, alpha=alpha, refract=refract, nms_sec=nms_sec,
                adapt=adapt_scope, recall=recall, fph=fph,
                triggers=int(trigs_ns.size), trigs=trigs_ns)

# -------------------- Search --------------------
rows = []
for scope in CFG.ADAPT_SCOPES:
    for q in CFG.GRID_Q:
        for a in CFG.GRID_ALPHA:
            for rf in CFG.GRID_REFRACT:
                for nms in CFG.GRID_NMS_SEC:
                    rows.append(run_adaptive_or_nms(q, a, rf, nms, scope))

scan = pd.DataFrame([{k:v for k,v in r.items() if k!='trigs'} for r in rows])
scan = scan.sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)

print("\nTop candidates (by recall then lower FPH):")
print(scan.head(12))

# Choose under budget if possible
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
if len(cands) > 0:
    chosen = cands.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
    note = "[Chosen UNDER budget]"
else:
    # Fall back: choose the LOWEST-FPH candidate overall (closest to budget)
    chosen = scan.sort_values(["fph","recall"], ascending=[True, False]).iloc[0]
    note = "[WARNING] No config met FPH budget; choosing lowest-FPH candidate"

print(f"\n{note} q={chosen['q']} alpha={chosen['alpha']} "
      f"refract={int(chosen['refract'])} nms={int(chosen['nms_sec'])} scope={chosen['adapt']} "
      f"| recall={chosen['recall']:.3f} FPH={chosen['fph']:.3f} triggers={int(chosen['triggers'])}")

Q_CHOSEN     = float(chosen["q"])
A_CHOSEN     = float(chosen["alpha"])
RF_CHOSEN    = int(chosen["refract"])
NMS_CHOSEN   = int(chosen["nms_sec"])
SCOPE_CHOSEN = str(chosen["adapt"])

# -------------------- Save triggers CSV --------------------
trigs_final = rows[scan.index.get_loc(chosen.name)]["trigs"] if "trigs" in rows[scan.index.get_loc(chosen.name)] else None
# safer: recompute once for the chosen params
res = run_adaptive_or_nms(Q_CHOSEN, A_CHOSEN, RF_CHOSEN, NMS_CHOSEN, SCOPE_CHOSEN)
trigs_final = res["trigs"]

mode_tag = "ornms"
csv_name = f"triggers_MB_{mode_tag}_q{Q_CHOSEN}_a{A_CHOSEN}_rf{RF_CHOSEN}_nms{NMS_CHOSEN}_sc{SCOPE_CHOSEN}.csv"
csv_out = os.path.join(CFG.OUT_DIR, csv_name)

if trigs_final.size == 0:
    pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    print(f"[Saved EMPTY] {csv_out}")
else:
    trig_ts = pd.to_datetime(trigs_final)
    date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
    trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
    trig_df.to_csv(csv_out, index=False)
    print(f"[Saved] {csv_out} (rows={len(trig_df)})")


[Info] events considered: 1392 on 17 days | hours=408.0

Top candidates (by recall then lower FPH):
        q  alpha  refract  nms_sec adapt    recall        fph  triggers
0   0.993   1.00       90       90  hour  0.985632  25.151975     10262
1   0.993   1.00       90       90   day  0.984914  24.649524     10057
2   0.993   1.01       90       90  hour  0.984914  24.730406     10090
3   0.993   1.01       90       90   day  0.981322  24.252465      9895
4   0.993   1.02       90       90  hour  0.981322  24.296583      9913
5   0.993   1.00       90       60   day  0.980603  21.681385      8846
6   0.993   1.00       90      120   day  0.979885  20.490208      8360
7   0.993   1.00      120      120   day  0.979885  20.490208      8360
8   0.993   1.00       90       60  hour  0.979885  21.953444      8957
9   0.993   1.02       90       90   day  0.979885  23.840700      9727
10  0.993   1.01       90       60  hour  0.978448  21.654424      8835
11  0.993   1.00      120       60  

In [39]:
# ============================================================
# Multiband STA/LTA Detection v2 — Budgeted (FPH <= 6/h)
# Single-point, conservative config to keep false positives down:
#   - Fusion: OR + NMS
#   - Hour-level quantile thresholds (robust to drift)
#   - q=0.997, alpha=1.03 (tighter than your previous)
#   - Refractory = 240 s, NMS = 240 s (aggressive thinning)
#   - Bands: 0.5–8 Hz (3 bands); you can re-enable 0.1–1.0 Hz if needed
# Output: triggers_MB_ornms_q..._a..._rf..._nms..._sc...csv
# ============================================================

import os, json, time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config (single-point, conservative) --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20

    # Bands:
    #   Conservative default (3 bands): reduces low-frequency false alarms
    BANDS      : tuple = ((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))
    # If you want a bit more recall, uncomment the low band and use 4 bands:
    # BANDS      : tuple = ((0.1, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows (seconds)
    STA        : float = 1.5
    LTA        : float = 20.0

    # Hysteresis OFF threshold
    OFF        : float = 1.0

    # Detection-eval window (for hit-rate; not for CNN cutting)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # ---------- Fixed parameters (tuned for <=6/h) ----------
    Q          : float = 0.997   # tighter quantile
    ALPHA      : float = 1.03    # slightly above 1.0
    REFRACT    : int   = 240     # refractory thinning (seconds)
    NMS_SEC    : int   = 240     # NMS spacing (seconds)
    ADAPT_SCOPE: str   = "hour"  # "hour" or "day"

    # Budget (used for reporting only)
    FPH_BUDGET : float = 6.0

    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- NPZ helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    """Load positives and return (X, y, sid, window_start) filtered to detect_label==1 if available."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))  # FIXED: map(str, ...)
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    Test events = positives within TEST scope.
    Event time = window_start + 20s (classification dataset convention).
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- Build daily CFTs + precompute quantiles --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """
    Read one MiniSEED; resample & detrend; for each band: bandpass -> classic_sta_lta.
    Returns dict(fs, t0, hours, cft_list=[...])
    """
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)

    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)

    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

def precompute_quantiles(entry, q_values):
    """
    Precompute per-day & per-hour quantiles for all bands and given q set.
    Adds keys:
      entry["q_day"][band_index][q] = value
      entry["q_hour"][band_index] = list of dict per hour: [{q:val,...}, ...]
      entry["hour_segments"] = list of (s,e) indices per hour
    """
    fs = entry["fs"]
    T  = len(entry["cft_list"][0])
    H  = int(3600 * fs)
    segs = [(s, min(s+H, T)) for s in range(0, T, H)]
    entry["hour_segments"] = segs
    entry["q_day"]  = []
    entry["q_hour"] = []
    for cft in entry["cft_list"]:
        qd = {float(q): float(np.quantile(cft, q)) for q in q_values}
        entry["q_day"].append(qd)
        qh = []
        for (s,e) in segs:
            seg = cft[s:e]
            if seg.size == 0:
                qh.append({float(q): 0.0 for q in q_values})
            else:
                qh.append({float(q): float(np.quantile(seg, q)) for q in q_values})
        entry["q_hour"].append(qh)

# -------------------- Fusion & metrics helpers --------------------
def to_onoff_array(x):
    """Normalize trigger_onset output to a (N,2) int ndarray."""
    arr = np.asarray(x, dtype=int)
    if arr.size == 0:
        return np.empty((0, 2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0:
            raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    """
    OR fusion (union of ON starts across bands) -> NMS (min spacing) -> refractory.
    Returns list of pandas.Timestamp in ascending time order.
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0:
            continue
        for a, b in arr:
            picks.append(t0 + a / fs)  # UTCDateTime (float seconds internally)
    if not picks:
        return []
    picks.sort()
    # NMS
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    # Refractory
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Boolean hits for each event window; hit if any trigger ∈ [start, end]."""
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build caches & eval windows --------------------
log("Building test events...")
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

log("Building CFTs & quantiles per day...")
cft_cache, total_hours, dates_ok = {}, 0.0, []
for di, dt in enumerate(event_days, 1):
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    entry = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    precompute_quantiles(entry, [CFG.Q])  # single q → fastest
    cft_cache[dt] = entry
    total_hours += entry["hours"]
    dates_ok.append(dt)
    if di % 2 == 0:
        log(f"  {di} day(s) processed...")

if not dates_ok:
    raise SystemExit("No overlapping days between TEST events and waveform files.")

ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

log(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")
log(f"[Params] q={CFG.Q} alpha={CFG.ALPHA} refract={CFG.REFRACT}s nms={CFG.NMS_SEC}s scope={CFG.ADAPT_SCOPE} bands={CFG.BANDS}")

# -------------------- Run once with fixed params --------------------
def run_fixed_params():
    all_picks = []
    for di, dt in enumerate(dates_ok, 1):
        entry = cft_cache[dt]
        fs = entry["fs"]
        onoffs = []
        for b, cft in enumerate(entry["cft_list"]):
            if CFG.ADAPT_SCOPE == "hour":
                arrs = []
                for (seg_idx, (s,e)) in enumerate(entry["hour_segments"]):
                    base = entry["q_hour"][b][seg_idx].get(float(CFG.Q), 0.0)
                    on_val = base * CFG.ALPHA
                    raw = trigger_onset(cft[s:e], on_val, CFG.OFF)
                    _onoff = to_onoff_array(raw)
                    if _onoff.size:
                        _onoff[:,0] += s; _onoff[:,1] += s
                    arrs.append(_onoff)
                onoff = np.vstack(arrs) if len(arrs) else np.empty((0,2), dtype=int)
            elif CFG.ADAPT_SCOPE == "day":
                base = entry["q_day"][b].get(float(CFG.Q), 0.0)
                on_val = base * CFG.ALPHA
                raw = trigger_onset(cft, on_val, CFG.OFF)
                onoff = to_onoff_array(raw)
            else:
                raise ValueError("ADAPT_SCOPE must be 'hour' or 'day'")
            onoffs.append(onoff)

        picks = picks_or_then_nms(onoffs, fs, entry["t0"], nms_sec=CFG.NMS_SEC, refract=CFG.REFRACT)
        all_picks.extend(picks)

        if (di % 1) == 0:
            log(f"  processed {di}/{len(dates_ok)} day(s)")

    # Metrics
    trigs_ns = np.array(sorted(all_picks), dtype="datetime64[ns]")
    fph = trigs_ns.size / max(1e-6, total_hours)
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())

    # Save CSV
    mode_tag = "ornms"
    csv_name = f"triggers_MB_{mode_tag}_q{CFG.Q}_a{CFG.ALPHA}_rf{CFG.REFRACT}_nms{CFG.NMS_SEC}_sc{CFG.ADAPT_SCOPE}.csv"
    csv_out = os.path.join(CFG.OUT_DIR, csv_name)
    if trigs_ns.size == 0:
        pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    else:
        trig_ts = pd.to_datetime(trigs_ns)
        date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
        trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
        trig_df.to_csv(csv_out, index=False)

    log(f"[Final] recall={recall:.3f} | FPH={fph:.3f} | triggers={int(trigs_ns.size)}")
    log(f"[Saved] {csv_out} (rows={trigs_ns.size})")

run_fixed_params()


[13:12:27] Building test events...
[13:12:27] Building CFTs & quantiles per day...
[13:12:28]   2 day(s) processed...
[13:12:28]   4 day(s) processed...
[13:12:28]   6 day(s) processed...
[13:12:28]   8 day(s) processed...
[13:12:29]   10 day(s) processed...
[13:12:29]   12 day(s) processed...
[13:12:29]   14 day(s) processed...
[13:12:30]   16 day(s) processed...
[13:12:30] [Info] events considered: 1392 on 17 days | hours=408.0
[13:12:30] [Params] q=0.997 alpha=1.03 refract=240s nms=240s scope=hour bands=((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))
[13:12:30]   processed 1/17 day(s)
[13:12:30]   processed 2/17 day(s)
[13:12:30]   processed 3/17 day(s)
[13:12:30]   processed 4/17 day(s)
[13:12:30]   processed 5/17 day(s)
[13:12:30]   processed 6/17 day(s)
[13:12:30]   processed 7/17 day(s)
[13:12:30]   processed 8/17 day(s)
[13:12:30]   processed 9/17 day(s)
[13:12:30]   processed 10/17 day(s)
[13:12:30]   processed 11/17 day(s)
[13:12:30]   processed 12/17 day(s)
[13:12:30]   processed 13/17

In [41]:
# ============================================================
# Multiband STA/LTA Detection v2 — Auto-budget (FPH <= 6/h)
# Strategy:
#   - Fusion: OR + NMS  (recall-friendly, then thin)
#   - Hour-level thresholds (robust to drift) + day-level fallback
#   - Small, conservative search over (q, alpha, NMS, refractory, scope)
#   - Pick the HIGHEST recall that satisfies FPH <= budget
#   - If none satisfy, pick the LOWEST-FPH combo and warn
# Logs are verbose so Jupyter won't look "stuck".
# ============================================================

import os, json, time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20

    # Bands: conservative 3-band set to curb low-freq false alarms
    BANDS      : tuple = ((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))
    # If you want more recall later, add the low band back:
    # BANDS      : tuple = ((0.1, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows (seconds) and OFF hysteresis
    STA        : float = 1.5
    LTA        : float = 20.0
    OFF        : float = 1.0

    # Evaluation window for detection hit-rate only
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # -------- Small conservative search space (fast, budget-oriented) -------
    GRID_Q        : tuple = (0.997, 0.998, 0.999)
    GRID_ALPHA    : tuple = (1.03, 1.05)
    GRID_REFRACT  : tuple = (240, 300)
    GRID_NMS_SEC  : tuple = (240, 300, 360)
    ADAPT_SCOPES  : tuple = ("hour", "day")  # try hour first; day can reduce picks

    # Budget
    FPH_BUDGET    : float = 6.0

    # Output
    OUT_DIR       : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- NPZ & split helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))  # FIXED: map(str, ...)
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    Test events = positives in TEST scope; event_time = window_start + 20s.
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- CFT build & quantile precompute --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """
    Read MiniSEED; resample & detrend; per band: bandpass -> classic_sta_lta.
    """
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)
    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)
    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

def precompute_quantiles(entry, q_values):
    """
    Precompute per-day & per-hour quantiles for all bands and given q values.
    Adds:
      entry["q_day"][band_idx][q]  and
      entry["q_hour"][band_idx][hour_idx][q]
      entry["hour_segments"] = [(s,e), ...]
    """
    fs = entry["fs"]
    T  = len(entry["cft_list"][0])
    H  = int(3600 * fs)
    segs = [(s, min(s+H, T)) for s in range(0, T, H)]
    entry["hour_segments"] = segs
    entry["q_day"]  = []
    entry["q_hour"] = []
    for cft in entry["cft_list"]:
        qd = {float(q): float(np.quantile(cft, q)) for q in q_values}
        entry["q_day"].append(qd)
        qh = []
        for (s,e) in segs:
            seg = cft[s:e]
            if seg.size == 0:
                qh.append({float(q): 0.0 for q in q_values})
            else:
                qh.append({float(q): float(np.quantile(seg, q)) for q in q_values})
        entry["q_hour"].append(qh)

# -------------------- Fusion & metric helpers --------------------
def to_onoff_array(x):
    """Normalize trigger_onset output to a (N,2) int ndarray (handles list/array)."""
    arr = np.asarray(x, dtype=int)
    if arr.size == 0:
        return np.empty((0, 2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0:
            raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    """
    OR fusion of ON starts across bands -> NMS (min spacing) -> refractory thinning.
    Returns list of pandas.Timestamp (ascending).
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0:
            continue
        for a, b in arr:
            picks.append(t0 + a / fs)  # UTCDateTime (float seconds internally)
    if not picks: return []
    picks.sort()
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Hit if any trigger ∈ [start, end] for each event (vectorized)."""
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build caches --------------------
log("Building test events...")
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

log("Building CFTs & quantiles per day...")
cft_cache, total_hours, dates_ok = {}, 0.0, []
for di, dt in enumerate(event_days, 1):
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp):
        continue
    entry = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    precompute_quantiles(entry, CFG.GRID_Q)  # precompute for all candidate q
    cft_cache[dt] = entry
    total_hours += entry["hours"]
    dates_ok.append(dt)
    if di % 2 == 0:
        log(f"  {di} day(s) processed...")

if not dates_ok:
    raise SystemExit("No overlapping days between TEST events and waveform files.")

ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

log(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

# -------------------- Core evaluator (uses quantile cache) --------------------
def run_or_nms(q, alpha, refract, nms_sec, adapt_scope):
    picks_all = []
    for di, dt in enumerate(dates_ok, 1):
        entry = cft_cache[dt]
        fs = entry["fs"]
        onoffs = []
        for b, cft in enumerate(entry["cft_list"]):
            if adapt_scope == "hour":
                arrs = []
                for (seg_idx, (s,e)) in enumerate(entry["hour_segments"]):
                    base = entry["q_hour"][b][seg_idx].get(float(q), 0.0)
                    on_val = base * alpha
                    raw = trigger_onset(cft[s:e], on_val, CFG.OFF)
                    _onoff = to_onoff_array(raw)
                    if _onoff.size:
                        _onoff[:,0] += s; _onoff[:,1] += s
                    arrs.append(_onoff)
                onoff = np.vstack(arrs) if len(arrs) else np.empty((0,2), dtype=int)
            elif adapt_scope == "day":
                base = entry["q_day"][b].get(float(q), 0.0)
                on_val = base * alpha
                raw = trigger_onset(cft, on_val, CFG.OFF)
                onoff = to_onoff_array(raw)
            else:
                raise ValueError("adapt_scope must be 'hour' or 'day'")
            onoffs.append(onoff)
        picks = picks_or_then_nms(onoffs, fs, entry["t0"], nms_sec=nms_sec, refract=refract)
        picks_all.extend(picks)

    trigs_ns = np.array(sorted(picks_all), dtype="datetime64[ns]")
    fph = trigs_ns.size / max(1e-6, total_hours)
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    return dict(q=q, alpha=alpha, refract=int(refract), nms_sec=int(nms_sec),
                adapt=adapt_scope, recall=recall, fph=fph, triggers=int(trigs_ns.size), trigs=trigs_ns)

# -------------------- Search small grid (fast) --------------------
rows = []
total_combos = len(CFG.ADAPT_SCOPES)*len(CFG.GRID_Q)*len(CFG.GRID_ALPHA)*len(CFG.GRID_REFRACT)*len(CFG.GRID_NMS_SEC)
log(f"Scanning {total_combos} combos...")
done = 0
for scope in CFG.ADAPT_SCOPES:
    for q in CFG.GRID_Q:
        for a in CFG.GRID_ALPHA:
            for rf in CFG.GRID_REFRACT:
                for nms in CFG.GRID_NMS_SEC:
                    r = run_or_nms(q, a, rf, nms, scope)
                    rows.append({k:v for k,v in r.items() if k!='trigs'})
                    done += 1
                    if done % 10 == 0:
                        log(f"  scanned {done}/{total_combos} combos")

scan = pd.DataFrame(rows).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
log("Top candidates (by recall then lower FPH):")
log(scan.head(12).to_string(index=False))

# Choose best under budget; else lowest-FPH overall
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
if len(cands) > 0:
    chosen = cands.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
    note = "[Chosen UNDER budget]"
else:
    chosen = scan.sort_values(["fph","recall"], ascending=[True, False]).iloc[0]
    note = "[WARN] No combo meets budget; picking lowest-FPH overall"

log(f"{note} q={chosen['q']} a={chosen['alpha']} rf={int(chosen['refract'])} nms={int(chosen['nms_sec'])} sc={chosen['adapt']} "
    f"| recall={chosen['recall']:.3f} FPH={chosen['fph']:.3f} triggers≈{int(chosen['triggers'])}")

# -------------------- Finalize (re-run chosen & save CSV) --------------------
def finalize_and_save(q, a, rf, nms, scope):
    res = run_or_nms(q, a, rf, nms, scope)
    trigs_ns = res["trigs"]
    mode_tag = "ornms"
    csv_name = f"triggers_MB_{mode_tag}_q{q}_a{a}_rf{rf}_nms{nms}_sc{scope}.csv"
    csv_out = os.path.join(CFG.OUT_DIR, csv_name)
    if trigs_ns.size == 0:
        pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    else:
        trig_ts = pd.to_datetime(trigs_ns)
        date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
        trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
        trig_df.to_csv(csv_out, index=False)
    log(f"[FINAL] q={q} a={a} rf={rf} nms={nms} sc={scope} | recall={res['recall']:.3f} FPH={res['fph']:.3f} triggers={int(res['triggers'])}")
    log(f"[Saved] {csv_out} (rows={int(res['triggers'])})")

finalize_and_save(float(chosen["q"]), float(chosen["alpha"]), int(chosen["refract"]), int(chosen["nms_sec"]), str(chosen["adapt"]))


[13:16:20] Building test events...
[13:16:20] Building CFTs & quantiles per day...
[13:16:21]   2 day(s) processed...
[13:16:21]   4 day(s) processed...
[13:16:22]   6 day(s) processed...
[13:16:23]   8 day(s) processed...
[13:16:23]   10 day(s) processed...
[13:16:24]   12 day(s) processed...
[13:16:24]   14 day(s) processed...
[13:16:25]   16 day(s) processed...
[13:16:25] [Info] events considered: 1392 on 17 days | hours=408.0
[13:16:25] Scanning 72 combos...
[13:16:29]   scanned 10/72 combos
[13:16:31]   scanned 20/72 combos
[13:16:34]   scanned 30/72 combos
[13:16:37]   scanned 40/72 combos
[13:16:40]   scanned 50/72 combos
[13:16:43]   scanned 60/72 combos
[13:16:46]   scanned 70/72 combos
[13:16:46] Top candidates (by recall then lower FPH):
[13:16:46]     q  alpha  refract  nms_sec adapt   recall      fph  triggers
0.997   1.03      240      240  hour 0.642960 8.105397      3307
0.997   1.05      240      240  hour 0.624282 7.740201      3158
0.997   1.03      240      240   da

In [42]:
# ============================================================
# Multiband STA/LTA Detection v2 — Auto-budget (FPH <= 6/h)
# Strategy:
#   - Fusion: OR + NMS (recall-friendly, then thin)
#   - Hour-level thresholds (robust to drift) + day-level fallback
#   - Small conservative search over (q, alpha, NMS, refractory, scope)
#   - Pick the HIGHEST recall that satisfies FPH <= budget
#   - If none satisfy, pick the LOWEST-FPH combo and warn
#   - Verbose logs so notebooks won't look "stuck"
# ============================================================

import os, json, time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20

    # Bands: conservative 3-band set to curb low-freq false alarms
    BANDS      : tuple = ((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))
    # If you want more recall later, add the low band back:
    # BANDS      : tuple = ((0.1, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows and OFF hysteresis
    STA        : float = 1.5
    LTA        : float = 20.0
    OFF        : float = 1.0

    # Detection-eval window (for hit-rate; not for CNN cutting)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # -------- Small conservative search (fast, budget-oriented) -------
    GRID_Q        : tuple = (0.997, 0.998, 0.999)
    GRID_ALPHA    : tuple = (1.03, 1.05)
    GRID_REFRACT  : tuple = (240, 300)
    GRID_NMS_SEC  : tuple = (240, 300, 360)
    ADAPT_SCOPES  : tuple = ("hour", "day")  # try hour first; day may reduce picks

    # Budget
    FPH_BUDGET    : float = 6.0

    # Output
    OUT_DIR       : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- NPZ & split helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files: return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    """Load positives and return (X, y, sid, window_start) filtered to detect_label==1 if available."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))  # FIXED: map(str, ...)
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    Test events = positives in TEST scope; event_time = window_start + 20s.
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- CFT build & quantile precompute --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """Read MiniSEED; resample & detrend; per band: bandpass -> classic_sta_lta."""
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6: tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)
    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)
    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

def precompute_quantiles(entry, q_values):
    """
    Precompute per-day & per-hour quantiles for all bands and given q values.
    Adds:
      entry["q_day"][band_idx][q]  and
      entry["q_hour"][band_idx][hour_idx][q]
      entry["hour_segments"] = [(s,e), ...]
    """
    fs = entry["fs"]
    T  = len(entry["cft_list"][0])
    H  = int(3600 * fs)
    segs = [(s, min(s+H, T)) for s in range(0, T, H)]
    entry["hour_segments"] = segs
    entry["q_day"]  = []
    entry["q_hour"] = []
    for cft in entry["cft_list"]:
        qd = {float(q): float(np.quantile(cft, q)) for q in q_values}
        entry["q_day"].append(qd)
        qh = []
        for (s,e) in segs:
            seg = cft[s:e]
            if seg.size == 0:
                qh.append({float(q): 0.0 for q in q_values})
            else:
                qh.append({float(q): float(np.quantile(seg, q)) for q in q_values})
        entry["q_hour"].append(qh)

# -------------------- Fusion & metric helpers --------------------
def to_onoff_array(x):
    """Normalize trigger_onset output to a (N,2) int ndarray (handles list/array)."""
    arr = np.asarray(x, dtype=int)
    if arr.size == 0: return np.empty((0, 2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0: raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    """
    OR fusion of ON starts across bands -> NMS (min spacing) -> refractory thinning.
    Returns list of pandas.Timestamp (ascending).
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0: continue
        for a, b in arr:
            picks.append(t0 + a / fs)  # UTCDateTime (float seconds internally)
    if not picks: return []
    picks.sort()
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Hit if any trigger ∈ [start, end] for each event (vectorized)."""
    if trigs_ns.size == 0: return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build caches --------------------
log("Building test events...")
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

log("Building CFTs & quantiles per day...")
cft_cache, total_hours, dates_ok = {}, 0.0, []
for di, dt in enumerate(event_days, 1):
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp): continue
    entry = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    precompute_quantiles(entry, CFG.GRID_Q)  # precompute for all candidate q
    cft_cache[dt] = entry
    total_hours += entry["hours"]
    dates_ok.append(dt)
    if di % 2 == 0: log(f"  {di} day(s) processed...")

if not dates_ok:
    raise SystemExit("No overlapping days between TEST events and waveform files.")

ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

log(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

# -------------------- Core evaluator (uses quantile cache) --------------------
def run_or_nms(q, alpha, refract, nms_sec, adapt_scope):
    picks_all = []
    for dt in dates_ok:
        entry = cft_cache[dt]; fs = entry["fs"]
        onoffs = []
        for b, cft in enumerate(entry["cft_list"]):
            if adapt_scope == "hour":
                arrs = []
                for (seg_idx, (s,e)) in enumerate(entry["hour_segments"]):
                    base = entry["q_hour"][b][seg_idx].get(float(q), 0.0)
                    on_val = base * alpha
                    raw = trigger_onset(cft[s:e], on_val, CFG.OFF)
                    _onoff = to_onoff_array(raw)
                    if _onoff.size:
                        _onoff[:,0] += s; _onoff[:,1] += s
                    arrs.append(_onoff)
                onoff = np.vstack(arrs) if len(arrs) else np.empty((0,2), dtype=int)
            elif adapt_scope == "day":
                base = entry["q_day"][b].get(float(q), 0.0)
                on_val = base * alpha
                raw = trigger_onset(cft, on_val, CFG.OFF)
                onoff = to_onoff_array(raw)
            else:
                raise ValueError("adapt_scope must be 'hour' or 'day'")
            onoffs.append(onoff)
        picks = picks_or_then_nms(onoffs, fs, entry["t0"], nms_sec=int(nms_sec), refract=int(refract))
        picks_all.extend(picks)

    trigs_ns = np.array(sorted(picks_all), dtype="datetime64[ns]")
    fph = trigs_ns.size / max(1e-6, total_hours)
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    return dict(q=q, alpha=alpha, refract=int(refract), nms_sec=int(nms_sec),
                adapt=adapt_scope, recall=recall, fph=fph, triggers=int(trigs_ns.size), trigs=trigs_ns)

# -------------------- Search small grid (fast) --------------------
rows = []
total_combos = (len(CFG.ADAPT_SCOPES)*len(CFG.GRID_Q)*
                len(CFG.GRID_ALPHA)*len(CFG.GRID_REFRACT)*len(CFG.GRID_NMS_SEC))
log(f"Scanning {total_combos} combos...")
done = 0
for scope in CFG.ADAPT_SCOPES:
    for q in CFG.GRID_Q:
        for a in CFG.GRID_ALPHA:
            for rf in CFG.GRID_REFRACT:
                for nms in CFG.GRID_NMS_SEC:
                    r = run_or_nms(q, a, rf, nms, scope)
                    rows.append({k:v for k,v in r.items() if k!='trigs'})
                    done += 1
                    if done % 10 == 0:
                        log(f"  scanned {done}/{total_combos} combos")

scan = pd.DataFrame(rows).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
log("Top candidates (by recall then lower FPH):")
log(scan.head(12).to_string(index=False))

# Choose best under budget; else lowest-FPH overall
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
if len(cands) > 0:
    chosen = cands.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
    note = "[Chosen UNDER budget]"
else:
    chosen = scan.sort_values(["fph","recall"], ascending=[True, False]).iloc[0]
    note = "[WARN] No combo meets budget; picking lowest-FPH overall"

log(f"{note} q={chosen['q']} a={chosen['alpha']} rf={int(chosen['refract'])} "
    f"nms={int(chosen['nms_sec'])} sc={chosen['adapt']} | "
    f"recall={chosen['recall']:.3f} FPH={chosen['fph']:.3f} triggers≈{int(chosen['triggers'])}")

# -------------------- Finalize (re-run chosen & save CSV) --------------------
def finalize_and_save(q, a, rf, nms, scope):
    res = run_or_nms(q, a, rf, nms, scope)
    trigs_ns = res["trigs"]
    mode_tag = "ornms"
    csv_name = f"triggers_MB_{mode_tag}_q{q}_a{a}_rf{rf}_nms{nms}_sc{scope}.csv"
    csv_out = os.path.join(CFG.OUT_DIR, csv_name)
    if trigs_ns.size == 0:
        pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    else:
        trig_ts = pd.to_datetime(trigs_ns)
        date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
        trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
        trig_df.to_csv(csv_out, index=False)
    log(f"[FINAL] q={q} a={a} rf={rf} nms={nms} sc={scope} | recall={res['recall']:.3f} FPH={res['fph']:.3f} triggers={int(res['triggers'])}")
    log(f"[Saved] {csv_out} (rows={int(res['triggers'])})")

finalize_and_save(float(chosen["q"]), float(chosen["alpha"]), int(chosen["refract"]), int(chosen["nms_sec"]), str(chosen["adapt"]))


[13:24:20] Building test events...
[13:24:20] Building CFTs & quantiles per day...
[13:24:21]   2 day(s) processed...
[13:24:22]   4 day(s) processed...
[13:24:22]   6 day(s) processed...
[13:24:23]   8 day(s) processed...
[13:24:24]   10 day(s) processed...
[13:24:24]   12 day(s) processed...
[13:24:25]   14 day(s) processed...
[13:24:26]   16 day(s) processed...
[13:24:26] [Info] events considered: 1392 on 17 days | hours=408.0
[13:24:26] Scanning 72 combos...
[13:24:29]   scanned 10/72 combos
[13:24:32]   scanned 20/72 combos
[13:24:35]   scanned 30/72 combos
[13:24:38]   scanned 40/72 combos
[13:24:41]   scanned 50/72 combos
[13:24:44]   scanned 60/72 combos
[13:24:47]   scanned 70/72 combos
[13:24:48] Top candidates (by recall then lower FPH):
[13:24:48]     q  alpha  refract  nms_sec adapt   recall      fph  triggers
0.997   1.03      240      240  hour 0.642960 8.105397      3307
0.997   1.05      240      240  hour 0.624282 7.740201      3158
0.997   1.03      240      240   da

In [43]:
# ============================================================
# Cascade v2 (verbose): triggers -> re-center -> cut -> z-score -> strict CNN
# - Verbose progress logs with timestamps
# - FORCE_DEVICE option (cpu/cuda/auto; MPS skipped by default)
# - DRY_RUN_LIMIT for quick smoke tests
# - Robust trigger CSV resolver (or set CFG.TRIG_CSV explicitly)
# - Wider re-centering; TRAIN-only z-score; strict CNN
# - Optional softmax remap to lift M/L recall slightly
# ============================================================

import os, glob, time, json, numpy as np, pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.filter import envelope
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch, torch.nn as nn, torch.nn.functional as F

# -------------------- Config --------------------
@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # If empty, resolver will auto-pick newest triggers_MB_*.csv in OUT_DIR
    TRIG_CSV   : str = ""  # e.g., "runs/cascade_eval/triggers_MB_ornms_q0.997_a1.05_rf300_nms240_scday.csv"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20
    BAND_CUT   : tuple = (0.5, 8.0)  # band for cutting windows

    # CNN window (must match training)
    PRE        : int = 20
    POST       : int = 70

    # Detection-eval window (match detection eval)
    PRE_DET    : int = 20
    POST_DET   : int = 300

    # Wider re-centering window (coarse + fine)
    RC_PRE     : int = 20
    RC_POST    : int = 40

    # Strict CNN checkpoint (same arch as training)
    BEST_PT    : str = "runs/cnn_strict/best.pt"
    OUT_DIR    : str = "runs/cascade_eval"

    # Post-softmax remapping (optional)
    REMAP_ENABLE  : bool  = True
    REMAP_M_THRES : float = 0.33
    REMAP_L_THRES : float = 0.20

    # ---------- Stability & speed ----------
    FORCE_DEVICE  : str = "auto"  # "cpu" | "cuda" | "auto" (auto prefers CUDA; MPS skipped)
    BATCH_SIZE    : int = 256
    DRY_RUN_LIMIT : int = 0       # set to 50 for a quick smoke test
    PRINT_EVERY   : int = 100     # progress print frequency

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

LABEL_MAP = {"S":0, "M":1, "L":2}

# ---------- Robust trigger CSV resolver ----------
def resolve_trig_csv(cfg):
    if cfg.TRIG_CSV and os.path.exists(cfg.TRIG_CSV):
        log(f"[Using provided trigger CSV] {cfg.TRIG_CSV}")
        return cfg.TRIG_CSV
    pattern = os.path.join(cfg.OUT_DIR, "triggers_MB_*.csv")
    cands = sorted(glob.glob(pattern), key=lambda p: os.path.getmtime(p), reverse=True)
    if cands:
        log(f"[Auto-picked latest trigger CSV] {cands[0]}")
        return cands[0]
    raise FileNotFoundError(f"No trigger CSV found in {cfg.OUT_DIR} matching 'triggers_MB_*.csv'")

# -------------------- Data helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files: return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, "s"))
    y_te = pd.Series(y[mte]).reset_index(drop=True)
    order = np.argsort(evt_time.values)
    df_te = pd.DataFrame({"event_time": evt_time.values[order]})
    y_te_sorted = y_te.iloc[order].to_numpy()
    return df_te.reset_index(drop=True), y_te_sorted

def train_mean_std(cfg: Cfg):
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    tr_scope, _ = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mtr = np.isin(sid, list(tr_scope))
    flat = X[mtr].reshape(np.sum(mtr), -1)
    mean, std = float(flat.mean()), float(flat.std() + 1e-8)
    with open(os.path.join(cfg.OUT_DIR, "mean_std_from_train.json"), "w") as f:
        json.dump({"mean": mean, "std": std}, f, indent=2)
    return mean, std

# -------------------- Waveform IO --------------------
_trace_cache = {}
def get_trace_for_day(cfg: Cfg, day_str: str):
    if day_str not in _trace_cache:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=day_str))
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        if abs(tr.stats.sampling_rate - cfg.FS) > 1e-6:
            tr.resample(cfg.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=cfg.BAND_CUT[0], freqmax=cfg.BAND_CUT[1])
        _trace_cache[day_str] = tr
    return _trace_cache[day_str]

def recenter_trigger(tr, t_ts, fs, pre, post):
    """
    Find an envelope peak near the trigger to center the CNN window.
    Coarse search (±pre/post), then fine re-peak in ±5 s window.
    """
    t0 = UTCDateTime(t_ts.to_pydatetime()) - pre
    t1 = UTCDateTime(t_ts.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need:
        return t_ts
    env = envelope(x)
    i = int(np.argmax(env))
    fine = int(5 * fs)
    a = max(0, i - fine); b = min(len(env), i + fine + 1)
    j = a + int(np.argmax(env[a:b]))
    t_pk = t0 + j / fs
    return pd.Timestamp(UTCDateTime(t_pk).datetime)

# -------------------- Strict CNN --------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k // 2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn   = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d):
        super().__init__(); self.W = nn.Linear(d, d); self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch, 32), ConvBlock(32, 64), ConvBlock(64, 128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2 * hidden)
        self.head = nn.Sequential(nn.Linear(2 * hidden, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x):
        z = self.cnn(x)          # [B,C,L]
        z = z.transpose(1, 2)    # [B,L,C]
        H, _ = self.lstm(z)      # [B,L,2H]
        Z, _ = self.attn(H)      # [B,2H]
        return self.head(Z)

def softmax_remap(logits, enable=False, m_th=0.33, l_th=0.20):
    preds = logits.argmax(1).cpu().numpy()
    if not enable: return preds
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    pM, pL = probs[:,1], probs[:,2]
    for n in range(len(preds)):
        if pL[n] >= l_th: preds[n] = 2
        elif (pM[n] >= m_th) and (preds[n] == 0): preds[n] = 1
    return preds

# -------------------- Main --------------------
log("Building test events...")
df_te, y_te_sorted = build_test_events(CFG)

log("Resolving trigger CSV...")
trig_csv_path = resolve_trig_csv(CFG)
trig_df = pd.read_csv(trig_csv_path)
if "trigger_time" not in trig_df.columns:
    raise ValueError(f"'trigger_time' column not found in {trig_csv_path}")
trig_df["trigger_time"] = pd.to_datetime(trig_df["trigger_time"], errors="coerce", utc=False)
trig_df = trig_df.dropna(subset=["trigger_time"]).sort_values("trigger_time").reset_index(drop=True)
log(f"Triggers loaded: {len(trig_df)} (showing first 3)\n{trig_df.head(3)}")

log("Preparing detection-eval windows...")
ev = df_te.copy()
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")

tr       = trig_df["trigger_time"].values.astype("datetime64[ns]")
ev_start = ev["start"].values.astype("datetime64[ns]")
ev_end   = ev["end"].values.astype("datetime64[ns]")

i = np.searchsorted(tr, ev_start, side="left")
j = np.searchsorted(tr, ev_end,   side="right")
hit = (j - i) > 0
first_idx = np.where(hit, i, -1)
log(f"[Info] events={len(ev)} | hits={int(hit.sum())} | hit_rate={hit.mean():.3f}")

# -------------------- Cut windows --------------------
win_len = CFG.FS * (CFG.PRE + CFG.POST)
X_cut, y_hit, kept = [], [], []
hit_indices = np.where(hit)[0]
if CFG.DRY_RUN_LIMIT > 0:
    hit_indices = hit_indices[:CFG.DRY_RUN_LIMIT]
log(f"Cutting windows for {len(hit_indices)} hits (win_len={win_len})...")

_trace_cache = {}
def get_trace_for_day_cached(day_str):
    if day_str not in _trace_cache:
        _trace_cache[day_str] = get_trace_for_day(CFG, day_str)
    return _trace_cache[day_str]

for k, idx in enumerate(hit_indices, 1):
    t_first_ns = tr[first_idx[idx]]
    t_first = pd.Timestamp(t_first_ns)
    day_str = str(t_first.date())
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=day_str))
    if not os.path.exists(fp): 
        continue
    tr_day = get_trace_for_day_cached(day_str)
    t_center = recenter_trigger(tr_day, t_first, fs=CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)
    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr_day.slice(t0, t1).data
    if len(x) >= win_len:
        X_cut.append(x[:win_len].astype(np.float32))
        y_hit.append(y_te_sorted[idx])
        kept.append(idx)
    if (k % CFG.PRINT_EVERY) == 0:
        log(f" Cut {k}/{len(hit_indices)} windows...")

if len(X_cut) == 0:
    raise SystemExit("No windows cut; check waveform paths or time bounds.")

X_cut = np.stack(X_cut, axis=0)
y_hit = np.array(y_hit, dtype=int)
log(f"[Cut] windows={len(X_cut)} | shape={X_cut.shape} | kept={len(kept)}")

# -------------------- Normalize --------------------
log("Computing TRAIN-only mean/std for z-score...")
mean, std = train_mean_std(CFG)
Xn = (X_cut - mean) / (std if std > 0 else 1.0)
Xn = Xn[:, None, :]  # [N,1,T]
log("Z-score done.")

# -------------------- Device & model --------------------
def choose_device(pref: str):
    if pref.lower() == "cpu": return torch.device("cpu")
    if pref.lower() == "cuda" and torch.cuda.is_available(): return torch.device("cuda")
    if torch.cuda.is_available(): return torch.device("cuda")
    return torch.device("cpu")

device = choose_device(CFG.FORCE_DEVICE)
log(f"Using device: {device}")
if not os.path.exists(CFG.BEST_PT):
    raise FileNotFoundError(f"CNN checkpoint not found: {CFG.BEST_PT}")

log("Loading model checkpoint...")
model = CNNBiLSTMAttn(in_ch=1, n_classes=3).to(device)
state = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state, strict=True)
model.eval()
log("Model ready.")

# -------------------- Inference --------------------
log("Running inference...")
y_pred = []
with torch.no_grad():
    for i0 in range(0, len(Xn), CFG.BATCH_SIZE):
        xb = torch.from_numpy(Xn[i0:i0+CFG.BATCH_SIZE]).to(device)
        logits = model(xb)
        preds = softmax_remap(logits, enable=CFG.REMAP_ENABLE,
                              m_th=CFG.REMAP_M_THRES, l_th=CFG.REMAP_L_THRES)
        y_pred.extend(preds)
        if ((i0 // CFG.BATCH_SIZE + 1) % max(1, CFG.PRINT_EVERY // max(1, CFG.BATCH_SIZE))) == 0:
            log(f" Inferred {min(i0+CFG.BATCH_SIZE, len(Xn))}/{len(Xn)}")

y_pred = np.array(y_pred, dtype=int)
log("Inference done.")

# -------------------- Reports --------------------
rep = classification_report(y_hit, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0)
cm = confusion_matrix(y_hit, y_pred, labels=[0,1,2])
macro_f1 = f1_score(y_hit, y_pred, average="macro")

print("\n== CNN (cascade on detected events) ==")
print(rep)
print("Confusion matrix:\n", cm)

# End-to-end per-class recall (missed detections count as errors)
lab = y_te_sorted
correct_mask = np.zeros_like(lab, dtype=bool)
kept_idx = np.array(kept, dtype=int)
for k, idx in enumerate(kept_idx):
    if y_hit[k] == y_pred[k]:
        correct_mask[idx] = True

for c, name in enumerate(["S","M","L"]):
    total_c = (lab == c).sum()
    e2e_c = correct_mask[lab == c].sum() / max(1, total_c)
    print(f"[End-to-End] {name} recall = {e2e_c:.3f} (total {total_c})")

# Save artifacts
out_report = os.path.join(CFG.OUT_DIR, "cascade_v2_report.txt")
with open(out_report, "w") as f:
    f.write(rep + "\n")
    f.write("Confusion matrix:\n" + np.array2string(cm))
np.savez(os.path.join(CFG.OUT_DIR, "cascade_v2_preds.npz"),
         y_true=y_hit, y_pred=y_pred, kept_event_indices=np.array(kept, dtype=int))
log(f"Saved report to {out_report}")


[13:24:57] Building test events...
[13:24:57] Resolving trigger CSV...
[13:24:57] [Auto-picked latest trigger CSV] runs/cascade_eval/triggers_MB_ornms_q0.997_a1.05_rf300_nms240_scday.csv
[13:24:57] Triggers loaded: 2433 (showing first 3)
                trigger_time        date
0 2011-03-01 00:02:45.119500  2011-03-01
1 2011-03-01 00:16:13.769500  2011-03-01
2 2011-03-01 00:28:27.219500  2011-03-01
[13:24:57] Preparing detection-eval windows...
[13:24:57] [Info] events=1392 | hits=743 | hit_rate=0.534
[13:24:57] Cutting windows for 743 hits (win_len=1800)...
[13:24:57]  Cut 100/743 windows...
[13:24:57]  Cut 200/743 windows...
[13:24:57]  Cut 300/743 windows...
[13:24:57]  Cut 400/743 windows...
[13:24:57]  Cut 500/743 windows...
[13:24:58]  Cut 600/743 windows...
[13:24:58]  Cut 700/743 windows...
[13:24:58] [Cut] windows=743 | shape=(743, 1800) | kept=743
[13:24:58] Computing TRAIN-only mean/std for z-score...
[13:24:58] Z-score done.
[13:24:58] Using device: cpu
[13:24:58] Loading m

In [44]:
# ============================================================
# FPH Checker for Multiband STA/LTA Triggers
# - Build the exact day set used by detection:
#   days = (unique test event dates) ∩ (dates with available mseed files)
# - Load triggers CSV (auto-pick latest if not provided), filter to those days
# - Compute total_hours from mseed coverage and FPH = triggers / total_hours
# - Print overall summary and per-day breakdown
# ============================================================

import os, glob, json
from dataclasses import dataclass
import numpy as np
import pandas as pd
from obspy import read

@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"

    TRIG_CSV   : str = ""   # if empty, auto-pick latest triggers_MB_*.csv
    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()

def _first_key(d, candidates):
    for k in candidates:
        if k in d.files: return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_event_days(cfg: Cfg):
    """Return sorted unique dates of TEST positives (window_start + 20s by convention)."""
    _, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    return sorted(set(pd.to_datetime(evt_time).dt.date))

def resolve_trig_csv(cfg: Cfg):
    if cfg.TRIG_CSV and os.path.exists(cfg.TRIG_CSV):
        print(f"[Using provided trigger CSV] {cfg.TRIG_CSV}")
        return cfg.TRIG_CSV
    pattern = os.path.join(cfg.OUT_DIR, "triggers_MB_*.csv")
    cands = sorted(glob.glob(pattern), key=lambda p: os.path.getmtime(p), reverse=True)
    if not cands:
        raise FileNotFoundError(f"No trigger CSV found in {cfg.OUT_DIR} matching 'triggers_MB_*.csv'.")
    print(f"[Auto-picked latest trigger CSV] {cands[0]}")
    return cands[0]

def compute_hours_for_days(cfg: Cfg, days):
    per_day_hours, days_ok, total_hours = {}, [], 0.0
    for dt in days:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=dt))
        if not os.path.exists(fp): continue
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
        per_day_hours[dt] = hours
        total_hours += hours
        days_ok.append(dt)
    return per_day_hours, total_hours, days_ok

def check_fph(cfg: Cfg):
    event_days = build_test_event_days(cfg)
    per_day_hours, total_hours, days_ok = compute_hours_for_days(cfg, event_days)
    if not days_ok:
        raise SystemExit("No overlapping days between TEST events and waveform files.")

    trig_csv = resolve_trig_csv(cfg)
    df = pd.read_csv(trig_csv)
    if "trigger_time" not in df.columns:
        raise ValueError(f"'trigger_time' column not found in {trig_csv}")
    df["trigger_time"] = pd.to_datetime(df["trigger_time"], errors="coerce", utc=False)
    df = df.dropna(subset=["trigger_time"]).sort_values("trigger_time").reset_index(drop=True)
    df["date"] = df["trigger_time"].dt.date

    n_total_trigs = len(df)
    df = df[df["date"].isin(days_ok)].copy()
    n_used_trigs = len(df)
    n_dropped = n_total_trigs - n_used_trigs
    if n_dropped > 0:
        print(f"[Note] Dropped {n_dropped} triggers outside the detection day set (kept {n_used_trigs}).")

    if total_hours <= 0: raise ValueError("Total hours computed as zero; check your mseed files.")
    fph = n_used_trigs / total_hours

    per_day = df.groupby("date")["trigger_time"].count().rename("triggers").reset_index()
    per_day["hours"] = per_day["date"].map(per_day_hours).astype(float)
    per_day["fph"] = per_day["triggers"] / per_day["hours"].replace(0, np.nan)

    print("\n================= FPH SUMMARY =================")
    print(f"Triggers CSV: {trig_csv}")
    print(f"Days considered: {len(days_ok)}  |  Total hours: {total_hours:.3f}")
    print(f"Total triggers (used): {n_used_trigs}  |  Overall FPH: {fph:.3f} per hour")
    print("===============================================")

    if len(per_day) > 0:
        per_day_sorted = per_day.sort_values("fph", ascending=False)
        print("\nTop-10 days by FPH:")
        print(per_day_sorted.head(10).to_string(index=False))
    else:
        print("No triggers within the considered day set; FPH = 0.")

    return dict(fph=fph, total_hours=total_hours, n_trigs=n_used_trigs, per_day=per_day)

if __name__ == "__main__":
    _ = check_fph(CFG)


[Auto-picked latest trigger CSV] runs/cascade_eval/triggers_MB_ornms_q0.997_a1.05_rf300_nms240_scday.csv

================= FPH SUMMARY =================
Triggers CSV: runs/cascade_eval/triggers_MB_ornms_q0.997_a1.05_rf300_nms240_scday.csv
Days considered: 17  |  Total hours: 408.000
Total triggers (used): 2433  |  Overall FPH: 5.963 per hour

Top-10 days by FPH:
      date  triggers     hours      fph
2011-03-04       171 23.999986 7.125004
2011-03-03       167 23.999986 6.958337
2011-03-01       165 23.999986 6.875004
2011-03-07       164 23.999986 6.833337
2011-03-05       154 23.999986 6.416670
2011-03-10       153 23.999986 6.375004
2011-03-13       143 23.999986 5.958337
2011-03-12       140 23.999986 5.833337
2011-03-09       140 23.999986 5.833337
2011-03-14       135 23.999986 5.625003


In [46]:
# ============================================================
# Multiband STA/LTA Detection v2 — Scheme A (Auto-budget, hour-adapt)
# Goal:
#   - Push recall up by using hour-level adaptive thresholds,
#     while keeping FPH <= 6/h by tightening NMS (330/360/390s)
# Choices:
#   - Fusion: OR + NMS (recall-friendly, then thin)
#   - Hour-level quantiles (robust to drift)
#   - Conservative 3-band front-end to curb low-freq false alarms
# Output:
#   runs/cascade_eval/triggers_MB_ornms_q{q}_a{a}_rf{rf}_nms{nms}_schour.csv
# ============================================================

import os, json, time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# -------------------- Config (Scheme A) --------------------
@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20

    # Bands: conservative 3-band set to reduce LF false alarms
    BANDS      : tuple = ((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))
    # If you later want extra recall, add low band back:
    # BANDS      : tuple = ((0.1, 1.0), (0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA windows and OFF hysteresis
    STA        : float = 1.5
    LTA        : float = 20.0
    OFF        : float = 1.0

    # Detection-eval window (for hit-rate; not for CNN cutting)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # -------- Tiny grid for Scheme A (hour-adapt only) --------
    GRID_Q        : tuple = (0.997,)
    GRID_ALPHA    : tuple = (1.03,)
    GRID_REFRACT  : tuple = (300,)            # strong refractory
    GRID_NMS_SEC  : tuple = (360, 390, 420)   # tighten NMS to curb FPH
    ADAPT_SCOPES  : tuple = ("hour",)         # hour-level only

    # Budget
    FPH_BUDGET    : float = 6.0

    # Output directory
    OUT_DIR       : str = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)

def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- NPZ & split helpers --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files:
            return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    """Load positives and return (X, y, sid, window_start) filtered to detect_label==1 if available."""
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))  # fix: map(str, ...)
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """Test events = TEST positives; event_time = window_start + 20s (dataset convention)."""
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    df_te = pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)
    return df_te

# -------------------- CFT build & quantile precompute --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    """Read MiniSEED; resample & detrend; per band: bandpass -> classic_sta_lta."""
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6:
        tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime) / 3600.0)
    cft_list = []
    for (fmin, fmax) in bands:
        tr = tr0.copy()
        tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta * fs), int(lta * fs))
        cft_list.append(cft)
    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

def precompute_quantiles(entry, q_values):
    """
    Precompute per-day & per-hour quantiles for all bands and given q values.
    Adds:
      entry["q_day"][band_idx][q]  and
      entry["q_hour"][band_idx][hour_idx][q]
      entry["hour_segments"] = [(s,e), ...]
    """
    fs = entry["fs"]
    T  = len(entry["cft_list"][0])
    H  = int(3600 * fs)
    segs = [(s, min(s+H, T)) for s in range(0, T, H)]
    entry["hour_segments"] = segs
    entry["q_day"]  = []
    entry["q_hour"] = []
    for cft in entry["cft_list"]:
        qd = {float(q): float(np.quantile(cft, q)) for q in q_values}
        entry["q_day"].append(qd)
        qh = []
        for (s,e) in segs:
            seg = cft[s:e]
            if seg.size == 0:
                qh.append({float(q): 0.0 for q in q_values})
            else:
                qh.append({float(q): float(np.quantile(seg, q)) for q in q_values})
        entry["q_hour"].append(qh)

# -------------------- Fusion & metric helpers --------------------
def to_onoff_array(x):
    """Normalize trigger_onset output to a (N,2) int ndarray (handles list/array)."""
    arr = np.asarray(x, dtype=int)
    if arr.size == 0:
        return np.empty((0, 2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0] % 2 != 0:
            raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1, 2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1, 2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    """
    OR fusion of ON starts across bands -> NMS (min spacing) -> refractory thinning.
    Returns list of pandas.Timestamp (ascending).
    """
    picks = []
    for arr in onoffs:
        if arr is None or len(arr) == 0: continue
        for a, b in arr:
            picks.append(t0 + a / fs)  # UTCDateTime (float seconds internally)
    if not picks: return []
    picks.sort()
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t - last) > nms_sec):
            fused.append(t); last = t
    out, last = [], None
    for t in fused:
        if (last is None) or ((t - last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    """Hit if any trigger ∈ [start, end] for each event (vectorized)."""
    if trigs_ns.size == 0:
        return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build caches --------------------
log("Building test events...")
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

log("Building CFTs & quantiles per day...")
cft_cache, total_hours, dates_ok = {}, 0.0, []
for di, dt in enumerate(event_days, 1):
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp): continue
    entry = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    precompute_quantiles(entry, CFG.GRID_Q)  # precompute only needed q's
    cft_cache[dt] = entry
    total_hours += entry["hours"]
    dates_ok.append(dt)
    if di % 2 == 0:
        log(f"  {di} day(s) processed...")

if not dates_ok:
    raise SystemExit("No overlapping days between TEST events and waveform files.")

ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

log(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")

# -------------------- Core evaluator (hour-adapt only) --------------------
def run_or_nms(q, alpha, refract, nms_sec):
    picks_all = []
    for dt in dates_ok:
        entry = cft_cache[dt]
        fs = entry["fs"]
        onoffs = []
        for b, cft in enumerate(entry["cft_list"]):
            # hour-adapt thresholds using precomputed quantiles
            arrs = []
            for (seg_idx, (s,e)) in enumerate(entry["hour_segments"]):
                base = entry["q_hour"][b][seg_idx].get(float(q), 0.0)
                on_val = base * alpha
                raw = trigger_onset(cft[s:e], on_val, CFG.OFF)
                _onoff = to_onoff_array(raw)
                if _onoff.size:
                    _onoff[:,0] += s; _onoff[:,1] += s
                arrs.append(_onoff)
            onoff = np.vstack(arrs) if len(arrs) else np.empty((0,2), dtype=int)
            onoffs.append(onoff)

        # OR + NMS + refractory
        picks = picks_or_then_nms(onoffs, fs, entry["t0"], nms_sec=int(nms_sec), refract=int(refract))
        picks_all.extend(picks)

    trigs_ns = np.array(sorted(picks_all), dtype="datetime64[ns]")
    fph = trigs_ns.size / max(1e-6, total_hours)
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())
    return dict(q=q, alpha=alpha, refract=int(refract), nms_sec=int(nms_sec),
                adapt="hour", recall=recall, fph=fph, triggers=int(trigs_ns.size), trigs=trigs_ns)

# -------------------- Scan tiny grid (3 combos) --------------------
rows = []
total_combos = len(CFG.GRID_Q)*len(CFG.GRID_ALPHA)*len(CFG.GRID_REFRACT)*len(CFG.GRID_NMS_SEC)
log(f"Scanning {total_combos} combos...")
done = 0
for q in CFG.GRID_Q:
    for a in CFG.GRID_ALPHA:
        for rf in CFG.GRID_REFRACT:
            for nms in CFG.GRID_NMS_SEC:
                r = run_or_nms(q, a, rf, nms)
                rows.append({k:v for k,v in r.items() if k!='trigs'})
                done += 1
                log(f"  scanned {done}/{total_combos} combos")

scan = pd.DataFrame(rows).sort_values(["recall","fph"], ascending=[False, True]).reset_index(drop=True)
log("Top candidates (by recall then lower FPH):")
log(scan.to_string(index=False))

# Choose best under budget; else lowest-FPH overall
cands = scan[scan["fph"] <= CFG.FPH_BUDGET]
if len(cands) > 0:
    chosen = cands.sort_values(["recall","fph"], ascending=[False, True]).iloc[0]
    note = "[Chosen UNDER budget]"
else:
    chosen = scan.sort_values(["fph","recall"], ascending=[True, False]).iloc[0]
    note = "[WARN] No combo meets budget; picking lowest-FPH overall"

log(f"{note} q={chosen['q']} a={chosen['alpha']} rf={int(chosen['refract'])} "
    f"nms={int(chosen['nms_sec'])} sc=hour | "
    f"recall={chosen['recall']:.3f} FPH={chosen['fph']:.3f} triggers≈{int(chosen['triggers'])}")

# -------------------- Finalize (re-run chosen & save CSV) --------------------
def finalize_and_save(q, a, rf, nms):
    res = run_or_nms(q, a, rf, nms)
    trigs_ns = res["trigs"]
    mode_tag = "ornms"
    csv_name = f"triggers_MB_{mode_tag}_q{q}_a{a}_rf{rf}_nms{nms}_schour.csv"
    csv_out = os.path.join(CFG.OUT_DIR, csv_name)
    if trigs_ns.size == 0:
        pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    else:
        trig_ts = pd.to_datetime(trigs_ns)
        date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
        trig_df = pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").reset_index(drop=True)
        trig_df.to_csv(csv_out, index=False)
    log(f"[FINAL] q={q} a={a} rf={rf} nms={nms} sc=hour | recall={res['recall']:.3f} FPH={res['fph']:.3f} triggers={int(res['triggers'])}")
    log(f"[Saved] {csv_out} (rows={int(res['triggers'])})")

finalize_and_save(float(chosen["q"]), float(chosen["alpha"]), int(chosen["refract"]), int(chosen["nms_sec"]))


[13:36:31] Building test events...
[13:36:31] Building CFTs & quantiles per day...
[13:36:31]   2 day(s) processed...
[13:36:32]   4 day(s) processed...
[13:36:32]   6 day(s) processed...
[13:36:32]   8 day(s) processed...
[13:36:32]   10 day(s) processed...
[13:36:33]   12 day(s) processed...
[13:36:33]   14 day(s) processed...
[13:36:33]   16 day(s) processed...
[13:36:34] [Info] events considered: 1392 on 17 days | hours=408.0
[13:36:34] Scanning 3 combos...
[13:36:34]   scanned 1/3 combos
[13:36:34]   scanned 2/3 combos
[13:36:34]   scanned 3/3 combos
[13:36:34] Top candidates (by recall then lower FPH):
[13:36:34]     q  alpha  refract  nms_sec adapt   recall      fph  triggers
0.997   1.03      300      360  hour 0.540230 6.367651      2598
0.997   1.03      300      390  hour 0.515805 6.056376      2471
0.997   1.03      300      420  hour 0.496408 5.747552      2345
[13:36:34] [Chosen UNDER budget] q=0.997 a=1.03 rf=300 nms=420 sc=hour | recall=0.496 FPH=5.748 triggers≈2345
[13

In [47]:
# ============================================================
# Multiband STA/LTA — Scheme A (FIXED)  FPB <= ~6/h
# Fixed params (from your best run under budget):
#   q=0.997, alpha=1.03, refractory=300s, NMS=420s, adapt=hour
#   Bands: (0.5–2), (1–5), (5–8) Hz  (conservative 3-band)
# Output CSV: runs/cascade_eval/triggers_MB_ornms_q0.997_a1.03_rf300_nms420_schour.csv
# ============================================================

import os, json, time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset

@dataclass
class Cfg:
    # Project artifacts
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # Continuous waveforms
    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20

    # Bands (3-band conservative front-end)
    BANDS      : tuple = ((0.5, 2.0), (1.0, 5.0), (5.0, 8.0))

    # STA/LTA + hysteresis
    STA        : float = 1.5
    LTA        : float = 20.0
    OFF        : float = 1.0

    # Detection-eval window (for recall computation)
    PRE_DET    : int   = 20
    POST_DET   : int   = 300

    # -------- FIXED params (Scheme A) --------
    Q          : float = 0.997
    ALPHA      : float = 1.03
    REFRACT    : int   = 300
    NMS_SEC    : int   = 420
    ADAPT_SCOPE: str   = "hour"   # "hour" only

    OUT_DIR    : str   = "runs/cascade_eval"

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)
def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- NPZ + splits --------------------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files: return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id","sample_ids"])
    y_key   = _first_key(d, ["mag_class","labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos = (d["detect_label"].astype(int)==1) if "detect_label" in d.files else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """Test positives in TEST scope; event_time = window_start + 20s."""
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    return pd.DataFrame({"event_time": evt_time}).sort_values("event_time").reset_index(drop=True)

# -------------------- CFT build + quantiles --------------------
def build_cfts_for_day_multiband(fp, fs, bands, sta, lta):
    st = read(fp).merge(method=1, fill_value="interpolate")
    tr0 = st[0]
    if abs(tr0.stats.sampling_rate - fs) > 1e-6: tr0.resample(fs)
    tr0.detrend("demean")
    hours = float((tr0.stats.endtime - tr0.stats.starttime)/3600.0)
    cft_list = []
    for (fmin,fmax) in bands:
        tr = tr0.copy(); tr.filter("bandpass", freqmin=fmin, freqmax=fmax)
        x = tr.data.astype(np.float32, copy=False)
        cft = classic_sta_lta(x, int(sta*fs), int(lta*fs))
        cft_list.append(cft)
    return dict(fs=fs, t0=tr0.stats.starttime, hours=hours, cft_list=cft_list)

def precompute_quantiles(entry, q_values):
    fs = entry["fs"]; T = len(entry["cft_list"][0]); H = int(3600*fs)
    segs = [(s, min(s+H, T)) for s in range(0, T, H)]
    entry["hour_segments"] = segs; entry["q_day"] = []; entry["q_hour"] = []
    for cft in entry["cft_list"]:
        entry["q_day"].append({float(q): float(np.quantile(cft,q)) for q in q_values})
        qh=[]
        for (s,e) in segs:
            seg = cft[s:e]
            qh.append({float(q): float(np.quantile(seg,q)) if seg.size else 0.0 for q in q_values})
        entry["q_hour"].append(qh)

# -------------------- Fusion + metrics --------------------
def to_onoff_array(x):
    arr = np.asarray(x, dtype=int)
    if arr.size == 0: return np.empty((0,2), dtype=int)
    if arr.ndim == 1:
        if arr.shape[0]%2!=0: raise ValueError("Trigger on/off array length must be even.")
        arr = arr.reshape(-1,2)
    elif arr.shape[1] != 2:
        arr = arr.reshape(-1,2)
    return arr

def picks_or_then_nms(onoffs, fs, t0, nms_sec=30, refract=60):
    picks=[]
    for arr in onoffs:
        if arr is None or len(arr)==0: continue
        for a,b in arr: picks.append(t0 + a/fs)
    if not picks: return []
    picks.sort()
    fused, last = [], None
    for t in picks:
        if (last is None) or ((t-last) > nms_sec):
            fused.append(t); last = t
    out, last = [], None
    for t in fused:
        if (last is None) or ((t-last) > refract):
            out.append(pd.Timestamp(UTCDateTime(t).datetime)); last = t
    return out

def vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns):
    if trigs_ns.size == 0: return np.zeros(len(ev_start_ns), dtype=bool)
    i = np.searchsorted(trigs_ns, ev_start_ns, side="left")
    j = np.searchsorted(trigs_ns, ev_end_ns,   side="right")
    return (j - i) > 0

# -------------------- Build caches + run --------------------
log("Building test events...")
df_te = build_test_events(CFG)
event_days = sorted(set(df_te["event_time"].dt.date))

log("Building CFTs & quantiles per day...")
cft_cache, total_hours, dates_ok = {}, 0.0, []
for di, dt in enumerate(event_days, 1):
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=dt))
    if not os.path.exists(fp): continue
    entry = build_cfts_for_day_multiband(fp, CFG.FS, CFG.BANDS, CFG.STA, CFG.LTA)
    precompute_quantiles(entry, [CFG.Q])
    cft_cache[dt] = entry
    total_hours += entry["hours"]; dates_ok.append(dt)
    if di % 2 == 0: log(f"  {di} day(s) processed...")

if not dates_ok:
    raise SystemExit("No overlapping days between TEST events and waveform files.")

ev = df_te[df_te["event_time"].dt.date.isin(dates_ok)].copy().sort_values("event_time").reset_index(drop=True)
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")
ev_start_ns = ev["start"].to_numpy("datetime64[ns]")
ev_end_ns   = ev["end"].to_numpy("datetime64[ns]")

log(f"[Info] events considered: {len(ev)} on {len(dates_ok)} days | hours={total_hours:.1f}")
log(f"[Params] q={CFG.Q} alpha={CFG.ALPHA} rf={CFG.REFRACT}s nms={CFG.NMS_SEC}s scope={CFG.ADAPT_SCOPE}")

def run_fixed():
    all_picks=[]
    for di, dt in enumerate(dates_ok, 1):
        entry = cft_cache[dt]; fs = entry["fs"]; onoffs=[]
        for b, cft in enumerate(entry["cft_list"]):
            # hour-adapt threshold using precomputed quantiles
            arrs=[]
            for (seg_idx,(s,e)) in enumerate(entry["hour_segments"]):
                base = entry["q_hour"][b][seg_idx].get(float(CFG.Q), 0.0)
                on_val = base * CFG.ALPHA
                raw = trigger_onset(cft[s:e], on_val, CFG.OFF)
                _onoff = to_onoff_array(raw)
                if _onoff.size:
                    _onoff[:,0]+=s; _onoff[:,1]+=s
                arrs.append(_onoff)
            onoff = np.vstack(arrs) if len(arrs) else np.empty((0,2), dtype=int)
            onoffs.append(onoff)
        picks = picks_or_then_nms(onoffs, fs, entry["t0"], nms_sec=CFG.NMS_SEC, refract=CFG.REFRACT)
        all_picks.extend(picks)
        if di % 1 == 0: log(f"  processed {di}/{len(dates_ok)} day(s)")

    trigs_ns = np.array(sorted(all_picks), dtype="datetime64[ns]")
    fph = trigs_ns.size / max(1e-6, total_hours)
    hit = vectorized_any_hit(trigs_ns, ev_start_ns, ev_end_ns)
    recall = float(hit.mean())

    csv_out = os.path.join(CFG.OUT_DIR, f"triggers_MB_ornms_q{CFG.Q}_a{CFG.ALPHA}_rf{CFG.REFRACT}_nms{CFG.NMS_SEC}_schour.csv")
    if trigs_ns.size == 0:
        pd.DataFrame(columns=["trigger_time","date"]).to_csv(csv_out, index=False)
    else:
        trig_ts = pd.to_datetime(trigs_ns)
        date_str = pd.Series(trig_ts).dt.strftime('%Y-%m-%d')
        pd.DataFrame({"trigger_time": trig_ts, "date": date_str}).sort_values("trigger_time").to_csv(csv_out, index=False)

    log(f"[Final] recall={recall:.3f} | FPH={fph:.3f} | triggers={int(trigs_ns.size)}")
    log(f"[Saved] {csv_out} (rows={trigs_ns.size})")

run_fixed()


[13:40:18] Building test events...
[13:40:18] Building CFTs & quantiles per day...
[13:40:19]   2 day(s) processed...
[13:40:19]   4 day(s) processed...
[13:40:19]   6 day(s) processed...
[13:40:20]   8 day(s) processed...
[13:40:20]   10 day(s) processed...
[13:40:20]   12 day(s) processed...
[13:40:20]   14 day(s) processed...
[13:40:21]   16 day(s) processed...
[13:40:21] [Info] events considered: 1392 on 17 days | hours=408.0
[13:40:21] [Params] q=0.997 alpha=1.03 rf=300s nms=420s scope=hour
[13:40:21]   processed 1/17 day(s)
[13:40:21]   processed 2/17 day(s)
[13:40:21]   processed 3/17 day(s)
[13:40:21]   processed 4/17 day(s)
[13:40:21]   processed 5/17 day(s)
[13:40:21]   processed 6/17 day(s)
[13:40:21]   processed 7/17 day(s)
[13:40:21]   processed 8/17 day(s)
[13:40:21]   processed 9/17 day(s)
[13:40:21]   processed 10/17 day(s)
[13:40:21]   processed 11/17 day(s)
[13:40:21]   processed 12/17 day(s)
[13:40:21]   processed 13/17 day(s)
[13:40:21]   processed 14/17 day(s)
[13:

In [49]:
# ============================================================
# Cascade (FIXED): triggers -> re-center -> cut -> z-score -> strict CNN
# Fixed to your chosen detection CSV:
#   triggers_MB_ornms_q0.997_a1.03_rf300_nms420_schour.csv
# Remap: keep defaults (M=0.33, L=0.20) to match your current version
# ============================================================

import os, time, json, glob
import numpy as np
import pandas as pd
from dataclasses import dataclass
from obspy import read, UTCDateTime
from obspy.signal.filter import envelope
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch, torch.nn as nn, torch.nn.functional as F

@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    # FIX to the CSV we just produced (edit if you rename)
    TRIG_CSV   : str = "runs/cascade_eval/triggers_MB_ornms_q0.997_a1.03_rf300_nms420_schour.csv"

    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"
    FS         : int = 20
    BAND_CUT   : tuple = (0.5, 8.0)

    PRE        : int = 20
    POST       : int = 70
    PRE_DET    : int = 20
    POST_DET   : int = 300

    RC_PRE     : int = 20
    RC_POST    : int = 40

    BEST_PT    : str = "runs/cnn_strict/best.pt"
    OUT_DIR    : str = "runs/cascade_eval"

    # Keep the same remap as your previous version
    REMAP_ENABLE  : bool  = True
    REMAP_M_THRES : float = 0.33
    REMAP_L_THRES : float = 0.20

    FORCE_DEVICE  : str = "auto"  # "cpu" | "cuda" | "auto"
    BATCH_SIZE    : int = 256
    PRINT_EVERY   : int = 100

CFG = Cfg()
os.makedirs(CFG.OUT_DIR, exist_ok=True)
def log(msg): print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# ------ dataset helpers ------
def _first_key(d, candidates):
    for k in candidates:
        if k in d.files: return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id","sample_ids"])
    y_key   = _first_key(d, ["mag_class","labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos = (d["detect_label"].astype(int)==1) if "detect_label" in d.files else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path); df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_events(cfg: Cfg):
    """
    FIXED: use pd.to_timedelta for seconds offset; do NOT wrap Timedelta with to_datetime.
    """
    X, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = wst[mte].reset_index(drop=True) + pd.to_timedelta(cfg.PRE, unit="s")
    y_te = pd.Series(y[mte]).reset_index(drop=True)
    order = np.argsort(evt_time.values)
    df_te = pd.DataFrame({"event_time": evt_time.values[order]})
    y_te_sorted = y_te.iloc[order].to_numpy()
    return df_te.reset_index(drop=True), y_te_sorted

# ------ waveform IO ------
_trace_cache = {}
def get_trace_for_day(cfg: Cfg, day_str: str):
    if day_str not in _trace_cache:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=day_str))
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        if abs(tr.stats.sampling_rate - cfg.FS) > 1e-6: tr.resample(cfg.FS)
        tr.detrend("demean")
        tr.filter("bandpass", freqmin=cfg.BAND_CUT[0], freqmax=cfg.BAND_CUT[1])
        _trace_cache[day_str] = tr
    return _trace_cache[day_str]

def recenter_trigger(tr, t_ts, fs, pre, post):
    """Envelope peak near trigger, with fine search ±5s."""
    t0 = UTCDateTime(t_ts.to_pydatetime()) - pre
    t1 = UTCDateTime(t_ts.to_pydatetime()) + post
    x = tr.slice(t0, t1).data.astype(np.float32, copy=False)
    need = int((pre + post) * fs)
    if len(x) < need: return t_ts
    env = envelope(x)
    i = int(np.argmax(env)); fine = int(5*fs); a = max(0, i-fine); b = min(len(env), i+fine+1)
    j = a + int(np.argmax(env[a:b])); t_pk = t0 + j / fs
    return pd.Timestamp(UTCDateTime(t_pk).datetime)

# ------ strict CNN ------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=9, p=None, pool=2):
        super().__init__()
        if p is None: p = k//2
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, padding=p)
        self.bn = nn.BatchNorm1d(out_ch)
        self.pool = nn.MaxPool1d(kernel_size=pool)
    def forward(self, x):
        x = self.conv(x); x = self.bn(x); x = F.gelu(x); x = self.pool(x); return x

class AdditiveAttention(nn.Module):
    def __init__(self, d):
        super().__init__(); self.W = nn.Linear(d, d); self.v = nn.Linear(d, 1, bias=False)
    def forward(self, H):
        U = torch.tanh(self.W(H)); a = self.v(U).squeeze(-1); a = torch.softmax(a, dim=1)
        Z = torch.bmm(a.unsqueeze(1), H).squeeze(1); return Z, a

class CNNBiLSTMAttn(nn.Module):
    def __init__(self, in_ch=1, hidden=96, layers=2, n_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(ConvBlock(in_ch,32), ConvBlock(32,64), ConvBlock(64,128))
        self.lstm = nn.LSTM(128, hidden, num_layers=layers, batch_first=True, bidirectional=True, dropout=0.1)
        self.attn = AdditiveAttention(2*hidden)
        self.head = nn.Sequential(nn.Linear(2*hidden,128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128,n_classes))
    def forward(self, x):
        z = self.cnn(x); z = z.transpose(1,2); H,_ = self.lstm(z); Z,_ = self.attn(H); return self.head(Z)

def softmax_remap(logits, enable=False, m_th=0.33, l_th=0.20):
    preds = logits.argmax(1).cpu().numpy()
    if not enable: return preds
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    pM, pL = probs[:,1], probs[:,2]
    for n in range(len(preds)):
        if pL[n] >= l_th: preds[n] = 2
        elif (pM[n] >= m_th) and (preds[n] == 0): preds[n] = 1
    return preds

# ------ misc helpers ------
def log_head(df, n=3): return df.head(n).to_string(index=False)

def choose_device(pref: str):
    if pref.lower()=="cpu": return torch.device("cpu")
    if pref.lower()=="cuda" and torch.cuda.is_available(): return torch.device("cuda")
    if torch.cuda.is_available(): return torch.device("cuda")
    return torch.device("cpu")

# -------------------- MAIN --------------------
log("Building test events...")
df_te, y_te_sorted = build_test_events(CFG)

if not os.path.exists(CFG.TRIG_CSV):
    raise FileNotFoundError(f"Trigger CSV not found: {CFG.TRIG_CSV}")
log(f"[Using fixed trigger CSV] {CFG.TRIG_CSV}")
trig_df = pd.read_csv(CFG.TRIG_CSV)
if "trigger_time" not in trig_df.columns:
    raise ValueError("'trigger_time' column missing in CSV.")
trig_df["trigger_time"] = pd.to_datetime(trig_df["trigger_time"], errors="coerce", utc=False)
trig_df = trig_df.dropna(subset=["trigger_time"]).sort_values("trigger_time").reset_index(drop=True)
log(f"Triggers loaded: {len(trig_df)} (first 3)\n{log_head(trig_df[['trigger_time','date']])}")

log("Preparing detection-eval windows...")
ev = df_te.copy()
ev["start"] = ev["event_time"] - pd.to_timedelta(CFG.PRE_DET, unit="s")
ev["end"]   = ev["event_time"] + pd.to_timedelta(CFG.POST_DET, unit="s")

tr = trig_df["trigger_time"].values.astype("datetime64[ns]")
ev_start = ev["start"].values.astype("datetime64[ns]")
ev_end   = ev["end"].values.astype("datetime64[ns]")

i = np.searchsorted(tr, ev_start, side="left")
j = np.searchsorted(tr, ev_end,   side="right")
hit = (j - i) > 0
first_idx = np.where(hit, i, -1)
log(f"[Info] events={len(ev)} | hits={int(hit.sum())} | hit_rate={hit.mean():.3f}")

# Cut windows
win_len = CFG.FS * (CFG.PRE + CFG.POST)
X_cut, y_hit, kept = [], [], []
hit_indices = np.where(hit)[0]
log(f"Cutting windows for {len(hit_indices)} hits (win_len={win_len})...")
_cache = {}
def get_tr_day(day_str):
    if day_str not in _cache: _cache[day_str] = get_trace_for_day(CFG, day_str)
    return _cache[day_str]

for k, idx in enumerate(hit_indices, 1):
    t_first = pd.Timestamp(tr[first_idx[idx]])
    day_str = str(t_first.date())
    fp = os.path.join(CFG.MSEED_DIR, CFG.MSEED_FMT.format(date=day_str))
    if not os.path.exists(fp): continue
    tr_day = get_tr_day(day_str)
    t_center = recenter_trigger(tr_day, t_first, fs=CFG.FS, pre=CFG.RC_PRE, post=CFG.RC_POST)
    t0 = UTCDateTime(t_center.to_pydatetime()) - CFG.PRE
    t1 = UTCDateTime(t_center.to_pydatetime()) + CFG.POST
    x = tr_day.slice(t0, t1).data
    if len(x) >= win_len:
        X_cut.append(x[:win_len].astype(np.float32))
        y_hit.append(y_te_sorted[idx])
        kept.append(idx)
    if (k % CFG.PRINT_EVERY) == 0:
        log(f"  Cut {k}/{len(hit_indices)}")

if len(X_cut)==0: raise SystemExit("No windows cut; check files or bounds.")
X_cut = np.stack(X_cut, axis=0); y_hit = np.array(y_hit, dtype=int)
log(f"[Cut] windows={len(X_cut)} | shape={X_cut.shape} | kept={len(kept)}")

# z-score using TRAIN-only stats
def train_mean_std(cfg: Cfg):
    X, y, sid, _ = load_npz_pos(cfg.NPZ_PATH)
    tr_scope, _ = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mtr = np.isin(sid, list(tr_scope))
    flat = X[mtr].reshape(np.sum(mtr), -1)
    mean, std = float(flat.mean()), float(flat.std() + 1e-8)
    with open(os.path.join(cfg.OUT_DIR, "mean_std_from_train.json"), "w") as f:
        json.dump({"mean":mean,"std":std}, f, indent=2)
    return mean, std

log("Computing TRAIN-only mean/std...")
mean, std = train_mean_std(CFG)
Xn = (X_cut - mean) / (std if std>0 else 1.0)
Xn = Xn[:, None, :]

# device + model
def choose_device(pref: str):
    if pref.lower()=="cpu": return torch.device("cpu")
    if pref.lower()=="cuda" and torch.cuda.is_available(): return torch.device("cuda")
    if torch.cuda.is_available(): return torch.device("cuda")
    return torch.device("cpu")

device = choose_device(CFG.FORCE_DEVICE); log(f"Using device: {device}")
if not os.path.exists(CFG.BEST_PT):
    raise FileNotFoundError(f"CNN checkpoint not found: {CFG.BEST_PT}")
log("Loading model...")
class Model(CNNBiLSTMAttn): pass
model = Model(in_ch=1, n_classes=3).to(device)
state = torch.load(CFG.BEST_PT, map_location=device)
model.load_state_dict(state, strict=True); model.eval(); log("Model ready.")

# inference
log("Running inference...")
y_pred=[]
with torch.no_grad():
    for i0 in range(0, len(Xn), CFG.BATCH_SIZE):
        xb = torch.from_numpy(Xn[i0:i0+CFG.BATCH_SIZE]).to(device)
        logits = model(xb)
        preds = softmax_remap(logits, enable=CFG.REMAP_ENABLE,
                              m_th=CFG.REMAP_M_THRES, l_th=CFG.REMAP_L_THRES)
        y_pred.extend(preds)
        if ((i0 // CFG.BATCH_SIZE + 1) % max(1, CFG.PRINT_EVERY // max(1, CFG.BATCH_SIZE))) == 0:
            log(f"  Inferred {min(i0+CFG.BATCH_SIZE, len(Xn))}/{len(Xn)}")
y_pred = np.array(y_pred, dtype=int); log("Inference done.")

# reports
rep = classification_report(y_hit, y_pred, labels=[0,1,2], target_names=["S","M","L"], digits=4, zero_division=0)
cm = confusion_matrix(y_hit, y_pred, labels=[0,1,2])
macro_f1 = f1_score(y_hit, y_pred, average="macro")

print("\n== CNN (cascade on detected events) ==")
print(rep); print("Confusion matrix:\n", cm)

# end-to-end per-class recall
lab = y_te_sorted; correct_mask = np.zeros_like(lab, dtype=bool)
kept_idx = np.array(kept, dtype=int)
for k, idx in enumerate(kept_idx):
    if y_hit[k] == y_pred[k]: correct_mask[idx] = True

for c, name in enumerate(["S","M","L"]):
    total_c = (lab==c).sum()
    e2e_c = correct_mask[lab==c].sum() / max(1,total_c)
    print(f"[End-to-End] {name} recall = {e2e_c:.3f} (total {total_c})")

# save artifacts
out_report = os.path.join(CFG.OUT_DIR, "cascade_fixed_report.txt")
with open(out_report, "w") as f:
    f.write(rep + "\n"); f.write("Confusion matrix:\n" + np.array2string(cm))
np.savez(os.path.join(CFG.OUT_DIR, "cascade_fixed_preds.npz"),
         y_true=y_hit, y_pred=y_pred, kept_event_indices=np.array(kept, dtype=int))
log(f"Saved report to {out_report}")


[13:44:03] Building test events...
[13:44:03] [Using fixed trigger CSV] runs/cascade_eval/triggers_MB_ornms_q0.997_a1.03_rf300_nms420_schour.csv
[13:44:03] Triggers loaded: 2345 (first 3)
              trigger_time       date
2011-03-01 00:02:45.069500 2011-03-01
2011-03-01 00:16:13.619500 2011-03-01
2011-03-01 00:23:19.819500 2011-03-01
[13:44:03] Preparing detection-eval windows...
[13:44:03] [Info] events=1392 | hits=691 | hit_rate=0.496
[13:44:03] Cutting windows for 691 hits (win_len=1800)...
[13:44:03]   Cut 100/691
[13:44:04]   Cut 200/691
[13:44:04]   Cut 300/691
[13:44:04]   Cut 400/691
[13:44:04]   Cut 500/691
[13:44:04]   Cut 600/691
[13:44:04] [Cut] windows=691 | shape=(691, 1800) | kept=691
[13:44:04] Computing TRAIN-only mean/std...
[13:44:04] Using device: cpu
[13:44:04] Loading model...
[13:44:04] Model ready.
[13:44:04] Running inference...
[13:44:10]   Inferred 256/691
[13:44:20]   Inferred 512/691
[13:44:27]   Inferred 691/691
[13:44:27] Inference done.

== CNN (casc

In [50]:
# ============================================================
# FPH Checker for Multiband STA/LTA Triggers (fixed pipeline)
# ============================================================

import os, glob, json
from dataclasses import dataclass
import numpy as np
import pandas as pd
from obspy import read

@dataclass
class Cfg:
    NPZ_PATH   : str = "data/wave_mag_dataset.npz"
    CSV_PATH   : str = "data/features_from_npz_mag.csv"
    SPLIT_PATH : str = "runs/frozen_splits.json"

    MSEED_DIR  : str = "waveforms"
    MSEED_FMT  : str = "MAJO_{date}.mseed"

    TRIG_CSV   : str = "runs/cascade_eval/triggers_MB_ornms_q0.997_a1.03_rf300_nms420_schour.csv"
    OUT_DIR    : str = "runs/cascade_eval"

CFG = Cfg()

def _first_key(d, candidates):
    for k in candidates:
        if k in d.files: return k
    raise KeyError(f"None of {candidates} found in NPZ.")

def load_npz_pos(npz_path):
    d = np.load(npz_path, allow_pickle=True, mmap_mode="r")
    X = d["waveforms"]
    sid_key = _first_key(d, ["sample_id", "sample_ids"])
    y_key   = _first_key(d, ["mag_class", "labels"])
    sid = np.array([str(s) for s in d[sid_key]])
    y   = d[y_key].astype(int)
    wst = pd.Series(pd.to_datetime(d["window_start"].astype(object), utc=True).tz_localize(None))
    pos_key = "detect_label" if "detect_label" in d.files else None
    pos = (d[pos_key].astype(int) == 1) if pos_key else np.ones(len(y), dtype=bool)
    return X[pos], y[pos], sid[pos], wst[pos].reset_index(drop=True)

def get_ids_split(csv_path, split_path):
    with open(split_path, "r") as f:
        splits = json.load(f)
    tr_ids = set(map(str, splits["magcls"]["train_ids"]))
    te_ids = set(map(str, splits["magcls"]["test_ids"]))
    df = pd.read_csv(csv_path)
    df["sample_id"] = df["sample_id"].astype(str)
    tr_scope = set(df[df["sample_id"].isin(tr_ids)]["sample_id"])
    te_scope = set(df[df["sample_id"].isin(te_ids)]["sample_id"])
    return tr_scope, te_scope

def build_test_event_days(cfg: Cfg):
    _, y, sid, wst = load_npz_pos(cfg.NPZ_PATH)
    _, te_scope = get_ids_split(cfg.CSV_PATH, cfg.SPLIT_PATH)
    mte = np.isin(sid, list(te_scope))
    evt_time = (wst[mte].reset_index(drop=True) + pd.to_timedelta(20, "s"))
    return sorted(set(pd.to_datetime(evt_time).dt.date))

def compute_hours_for_days(cfg: Cfg, days):
    per_day_hours, days_ok, total_hours = {}, [], 0.0
    for dt in days:
        fp = os.path.join(cfg.MSEED_DIR, cfg.MSEED_FMT.format(date=dt))
        if not os.path.exists(fp): continue
        st = read(fp).merge(method=1, fill_value="interpolate")
        tr = st[0]
        hours = float((tr.stats.endtime - tr.stats.starttime) / 3600.0)
        per_day_hours[dt] = hours
        total_hours += hours
        days_ok.append(dt)
    return per_day_hours, total_hours, days_ok

def check_fph(cfg: Cfg):
    event_days = build_test_event_days(cfg)
    per_day_hours, total_hours, days_ok = compute_hours_for_days(cfg, event_days)
    if not days_ok:
        raise SystemExit("No overlapping days between TEST events and waveform files.")
    if not os.path.exists(cfg.TRIG_CSV):
        raise FileNotFoundError(f"Trigger CSV not found: {cfg.TRIG_CSV}")

    df = pd.read_csv(cfg.TRIG_CSV)
    if "trigger_time" not in df.columns:
        raise ValueError(f"'trigger_time' column not found in {cfg.TRIG_CSV}")
    df["trigger_time"] = pd.to_datetime(df["trigger_time"], errors="coerce", utc=False)
    df = df.dropna(subset=["trigger_time"]).sort_values("trigger_time").reset_index(drop=True)
    df["date"] = df["trigger_time"].dt.date

    n_total_trigs = len(df)
    df = df[df["date"].isin(days_ok)].copy()
    n_used_trigs = len(df)
    if total_hours <= 0: raise ValueError("Total hours computed as zero; check your mseed files.")
    fph = n_used_trigs / total_hours

    per_day = df.groupby("date")["trigger_time"].count().rename("triggers").reset_index()
    per_day["hours"] = per_day["date"].map(per_day_hours).astype(float)
    per_day["fph"]   = per_day["triggers"] / per_day["hours"].replace(0, np.nan)

    print("\n================= FPH SUMMARY =================")
    print(f"Triggers CSV: {cfg.TRIG_CSV}")
    print(f"Days considered: {len(days_ok)}  |  Total hours: {total_hours:.3f}")
    print(f"Total triggers (used): {n_used_trigs}  |  Overall FPH: {fph:.3f} per hour")
    print("===============================================")

    if len(per_day) > 0:
        print("\nTop-10 days by FPH:")
        print(per_day.sort_values("fph", ascending=False).head(10).to_string(index=False))
    else:
        print("No triggers within the considered day set; FPH = 0.")

if __name__ == "__main__":
    check_fph(CFG)



================= FPH SUMMARY =================
Triggers CSV: runs/cascade_eval/triggers_MB_ornms_q0.997_a1.03_rf300_nms420_schour.csv
Days considered: 17  |  Total hours: 408.000
Total triggers (used): 2345  |  Overall FPH: 5.748 per hour

Top-10 days by FPH:
      date  triggers     hours      fph
2011-03-03       166 23.999986 6.916671
2011-03-04       165 23.999986 6.875004
2011-03-07       165 23.999986 6.875004
2011-03-05       162 23.999986 6.750004
2011-03-01       159 23.999986 6.625004
2011-03-10       143 23.999986 5.958337
2011-03-09       131 23.999986 5.458336
2011-03-11       131 23.999986 5.458336
2011-03-13       128 23.999986 5.333336
2011-03-15       128 23.999986 5.333336
